# PSI

In [2]:
import math
from pyecharts.components import Table
from pyecharts.options import ComponentTitleOpts
from pyecharts.charts import Bar, Grid, Line, Pie, Tab
from impala.util import as_pandas
import pandas as pd
from impala.dbapi import connect
import numpy as np
import time
import functools

def auto_run_psi(user,password,time1,time2,time3,time4,inputdate1,inputdate2,inputdate3,inputdate4):
    ###PSI计算公式(奇怪的点在于放入spyder中以公式形式调用时，会不管用？？？)
    def make_psi(SampleA,SampleB,bins = 'num'):
        Max = max(max(SampleA),max(SampleB))
        Min = min(min(SampleA),min(SampleB))
        result = pd.DataFrame()
        def get_result(bins,dropna=True):       
            Cnt_SampleA = pd.cut(SampleA,bins).value_counts(dropna=False)
            Cnt_SampleB = pd.cut(SampleB,bins).value_counts(dropna=False)
            Cnt_All_SampleA = len(SampleA)
            Cnt_All_SampleB = len(SampleB)
            result['Cnt_SampleA'] = Cnt_SampleA
            result['Cnt_SampleB'] = Cnt_SampleB
            result['Cnt_All_SampleA'] = len(SampleA)
            result['Cnt_All_SampleB'] = len(SampleB)
            result['SampleA_Rate'] = Cnt_SampleA/Cnt_All_SampleA
            result['SampleB_Rate'] = Cnt_SampleB/Cnt_All_SampleB
            #c=(result.SampleA_Rate-result.SampleB_Rate)*np.log(result.SampleA_Rate/result.SampleB_Rate)
            result['PSI'] = (result.SampleA_Rate-result.SampleB_Rate)*np.log(result.SampleA_Rate/result.SampleB_Rate)
            where_are_inf = np.isinf(result['PSI'])  ##查找是inf的PSI
            result['PSI'][where_are_inf] = 0         ##将inf转化成0
            result['PSI_Sum'] = sum(result['PSI'])
            return result
        if bins == 'num':
            bins1 = np.linspace(Min,Max,11)
            bins = [int(x) for x in bins1[1:10]]
            bins.insert(0,int(Min-1))
            bins.append(math.ceil(Max))
            result = get_result(bins,dropna=False)
        else:
            result = get_result(bins,dropna=False)
        return result
    ## 输入数据库账号
    conn = connect(host='10.1.57.8', 
                   port=10000, 
                   auth_mechanism='PLAIN',
                   user=user,
                   password = password,
                   database='fk')
    #连接数据库
    cursor=conn.cursor()

    
    ##确定需要监控的模型
    scores=['creditScore46','creditScore48','creditScore49','creditScore50','creditScore51','creditScore52','creditScore54','creditScore55',
       'creditScore56','creditScore57','creditScore58','creditScore59','creditScore60','creditScore61','creditScore62','creditScore63',
       'creditScore64','creditScore65']
    ##需要监控模型对应的中文名（注意：要一一对应）
    chinese_names= ['借呗阿波罗模型V2',
                    '借呗联合模型V2',
                    '借呗阿波罗模型V3（XGB）',
                    '借呗阿波罗模型V3（LGB）',
                    '借呗阿波罗模型V3（权重版）',
                    '京东A卡',
                    '借呗三方征信模型',
                    '借呗阿波罗入催模型',
                    '借呗负债压力模型',
                    '借呗人行二代模型',
                    '借呗融合模型-全变量',
                    '借呗融合模型-去除score56',
                    '京东plusA卡V2',
                    '借呗阿波罗V4',
                    '借呗三方征信V2',
                    '借呗阿波罗模型V1',
                    '京东plus卡A卡V3.1',
                    '京东plus卡A卡V3.2']
    ##借呗
    score_1010=['creditScore46','creditScore48','creditScore49','creditScore50','creditScore51','creditScore54','creditScore55','creditScore56',
                'creditScore57','creditScore58','creditScore59','creditScore61','creditScore62','creditScore63']
    ##京东
    score_1006=['creditScore52']
    tab = Tab()
    psi_dic={}
    for i, score in enumerate(scores):
        for a, chinese_name in enumerate(chinese_names):
            if i==a:        
                if score in score_1010:
                    businesstype="'"'1010'"'"
                elif score in score_1006:
                    businesstype="'"'1006'"'"
                else:
                    businesstype="'"'2006'"'"
                # 定义  显示运行时间 的小功能
                def runtime(func):
                    @functools.wraps(func)
                    def inner(sql_Strip):
                        t1 = time.time()
                        data = func(sql_Strip)
                        t2 = time.time()
                        run_time = t2 - t1
                        print(score+'运行时间为:',run_time)
                        return data
                    return inner
                # 输出SQL运行结果
                @runtime # 能展示时间
                def run_sql(sql_Strip):
                    cursor.execute(sql_Strip) #运行SQL
                    data = as_pandas(cursor)  ##  输出跑SQL结果
                    return data
                
                ###申请客群对照组sql
                apply_sql_control='''
                SELECT {score} as score
                FROM fk.v_r_risk_fico_xml_format
                WHERE  data_dt BETWEEN  {time1} AND {time2}
                AND left(applicationdate,10) BETWEEN  {time1} AND {time2}
                AND {score}>0
                AND producttype={businesstype}
                and decisionstage='Apply'
                '''
                apply_control_sql_end = apply_sql_control.format(
                time1=time1,
                time2=time2,
                score=score,
                businesstype=businesstype
                )
                apply_control=run_sql(apply_control_sql_end)
            
                ##判断是否没有数据
                if len(apply_control)<=50:
                    print(score+'apply_control没有数据')
                else:
                    ##申请客群实验组的sql
                    apply_sql_experiment='''
                    SELECT {score} as score
                    FROM fk.v_r_risk_fico_xml_format
                    WHERE  data_dt BETWEEN  {time3} AND {time4}
                    AND left(applicationdate,10) BETWEEN  {time3} AND {time4}
                    AND {score}>0
                    AND producttype={businesstype}
                    and decisionstage='Apply'
                    '''
                    apply_experiment_sql_end = apply_sql_experiment.format(
                    time3=time3
                    ,time4=time4
                    ,score=score
                    ,businesstype=businesstype
                    )
                    apply_experiment=run_sql(apply_experiment_sql_end)
                    if len(apply_experiment)==0:
                        print(score+'apply_experiment没有数据')
                    else:
                        ##通过客群对照组的sql
                        pass_sql_control='''
                        SELECT score
                        FROM 
                        (select serialno
                        from fk.v_g_aprvadt_appl_bsc_inf
                        where data_dt={time2}
                        and inputdate between {inputdate1} AND {inputdate2}
                        and decisionstage='Apply'--申请
                        and businesstype={businesstype} --限制渠道 借呗
                        AND approvestatus='03'--通过的
                        )a
                        JOIN 
                        (SELECT {score} as score,objectno
                        FROM fk.v_r_risk_fico_xml_format
                        where data_dt BETWEEN  {time1} AND {time2}
                        AND left(applicationdate,10) BETWEEN  {time1} AND {time2}
                        AND {score}>0
                        AND producttype={businesstype}
                        and decisionstage='Apply'
                        )b
                        ON a.serialno=b.objectno
                        '''
                        pass_control_sql_end = pass_sql_control.format(
                        time1=time1
                        ,time2=time2
                        ,inputdate1=inputdate1
                        ,inputdate2=inputdate2
                        ,score=score
                        ,businesstype=businesstype
                        )
                        pass_control=run_sql(pass_control_sql_end)
                        if len(pass_control)==0:
                            print(score+'pass_control没有数据')
                        else:
                            ##通过客群实验组的Sql
                            pass_sql_experiment='''
                            SELECT score
                            FROM 
                            (select serialno
                            from fk.v_g_aprvadt_appl_bsc_inf
                            where data_dt={time4}
                            and inputdate between {inputdate3} AND {inputdate4}
                            and decisionstage='Apply'
                            and businesstype={businesstype} 
                            AND approvestatus='03'
                            )a
                            JOIN 
                            (SELECT {score} as score ,objectno
                            FROM fk.v_r_risk_fico_xml_format
                            where data_dt BETWEEN  {time3} AND {time4}
                            AND left(applicationdate,10) BETWEEN  {time3} AND {time4}
                            AND {score}>0
                            AND producttype={businesstype}
                            and decisionstage='Apply'
                            )b
                            ON a.serialno=b.objectno
                            '''
                            ##运行sql
                            pass_experiment_sql_end = pass_sql_experiment.format(
                            time3=time3
                            ,time4=time4
                            ,inputdate3=inputdate3
                            ,inputdate4=inputdate4
                            ,score=score
                            ,businesstype=businesstype
                            )
                            pass_experiment=run_sql(pass_experiment_sql_end)
                            if len(pass_experiment)==0:
                                print(score+'pass_experiment没有数据')
                            else:
                                if score!='creditScore48':
                                    ##计算psi以及格式转化
                                    apply_a=apply_control['score'].astype('float').tolist()
                                    apply_b=apply_experiment['score'].astype('float').tolist()

                                    pass_a=pass_control['score'].astype(float).tolist()
                                    pass_b=pass_experiment['score'].astype(float).tolist()
                                else:
                                    ##注意：由于48模型的分数是小数，本身无法带入PSI公式，需要特殊处理，在此采用将数据乘以10000的方式
                                    L_a_c=apply_control['score'].astype('float').tolist()
                                    apply_a=[]
                                    for i in L_a_c:
                                        apply_a.append(i*10000)

                                    L_a_e=apply_experiment['score'].astype('float').tolist()
                                    apply_b=[]
                                    for i in L_a_e:
                                        apply_b.append(i*10000)

                                    L_p_c=pass_control['score'].astype('float').tolist()
                                    pass_a=[]
                                    for i in L_p_c:
                                        pass_a.append(i*10000)

                                    L_p_e=pass_experiment['score'].astype('float').tolist()
                                    pass_b=[]
                                    for i in L_p_e:
                                        pass_b.append(i*10000)

                                ##计算PSI，并进行各种展示形式上的变化
                                apply_psi= make_psi(apply_a,apply_b)
                                apply_psi.index.name = 'score' ##给第一列加上列名
                                apply_psi.drop(['Cnt_All_SampleA','Cnt_All_SampleB'],axis=1, inplace=True)##去掉某两列
                                f=lambda x: '%.2f%%' % (x*100)
                                apply_psi[['SampleA_Rate']]=apply_psi[['SampleA_Rate']].applymap(f)##转变成百分数并保留两位小数
                                apply_psi[['SampleB_Rate']]=apply_psi[['SampleB_Rate']].applymap(f)##转变成百分数并保留两位小数
                                apply_psi[['PSI']]=round(apply_psi[['PSI']],4)##保留四位
                                apply_psi[['PSI_Sum']]=round(apply_psi[['PSI_Sum']],4)##保留四位
                                apply_psi=apply_psi.reset_index()##将index列转化成普通列
                                apply_psi['状态']=['申请psi','申请psi','申请psi','申请psi','申请psi','申请psi','申请psi','申请psi','申请psi','申请psi']
                                cols = apply_psi.columns.tolist()
                                cols = cols[-1:] + cols[:-1]
                                apply_psi=apply_psi[cols]
                                column_list=['状态', 'score', '对照组', '实验组', '对照组占比','实验组占比', 'PSI', 'PSI_Sum']
                                apply_psi.columns=column_list

                                pass_psi= make_psi(pass_a,pass_b)
                                pass_psi.index.name = 'score'##给第一列加上列名
                                pass_psi.drop(['Cnt_All_SampleA','Cnt_All_SampleB'],axis=1, inplace=True)##去掉某两列
                                pass_psi[['SampleA_Rate']]=pass_psi[['SampleA_Rate']].applymap(f)##转变成百分数并保留两位小数
                                pass_psi[['SampleB_Rate']]=pass_psi[['SampleB_Rate']].applymap(f)##转变成百分数并保留两位小数
                                pass_psi[['PSI']]=round(pass_psi[['PSI']],4)##保留四位
                                pass_psi[['PSI_Sum']]=round(pass_psi[['PSI_Sum']],4)##保留四位
                                pass_psi=pass_psi.reset_index()##将index列转化成普通列
                                pass_psi['状态']=['通过psi','通过psi','通过psi','通过psi','通过psi','通过psi','通过psi','通过psi','通过psi','通过psi']
                                cols = pass_psi.columns.tolist()
                                cols = cols[-1:] + cols[:-1]
                                pass_psi=pass_psi[cols]
                                column_list=['状态', 'score', '对照组', '实验组', '对照组占比','实验组占比', 'PSI', 'PSI_Sum']
                                pass_psi.columns=column_list

                                ##字典加数
                                psi_dic.update({str(score):[apply_psi['PSI_Sum'][1],pass_psi['PSI_Sum'][1],businesstype]})

                                ##添加该部分用于分割apply_psi以及pass_psi
                                kong={
                                '状态':['','状态'],
                                'score':['','score'], 
                                '对照组':['','对照组'],
                                '实验组':['','实验组'], 
                                '对照组占比':['','对照组占比'],
                                '实验组占比':['','实验组占比'], 
                                'PSI':['','PSI'], 
                                'PSI_Sum':['','PSI_Sum']
                                }
                                kong=pd.DataFrame(kong)
                                psi=pd.concat([apply_psi,kong,pass_psi],axis=0)
                                
                                line_psi=list(range(0,22,1))
                                for i in line_psi:
                                    line_psi[i]=psi.iloc[i].tolist()
                                headers_psi=['状态', 'score', '对照组', '实验组', '对照组占比', '实验组占比', 'PSI', 'PSI_Sum']
                                rows_psi=[line_psi[0],line_psi[1],line_psi[2],line_psi[3],line_psi[4],line_psi[5],line_psi[6],line_psi[7],line_psi[8],line_psi[9],
                                          line_psi[10],line_psi[11],line_psi[12],line_psi[13],line_psi[14],line_psi[15],line_psi[16],line_psi[17],line_psi[18],
                                          line_psi[19],line_psi[20],line_psi[21]]
                                def psi():
                                    c=(Table().add(headers_psi,rows_psi)  
                                    .set_global_opts(title_opts=ComponentTitleOpts(title=score+"-"+"对照组"+"("+time1+time2+")"+"实验组"+"("+time3+time4+")"
                                                     ,subtitle=chinese_name)))  
                                    return c
                                tab.add(psi(),score)
    
    return tab.render("PSI.html"),psi_dic

## 运行部分

In [3]:
html,psi_dic=auto_run_psi(user='fk_yanjintong'##用户
                    ,password = 'Yjt@0828$$'##密码
                    ,time1="'"'2021-04-19'"'" ##控制组授信起始时间，inputdate1-4与time1-4对应
                    ,time2="'"'2021-04-25'"'" ##控制组授信终止时间
                    ,time3="'"'2021-04-26'"'" ##实验组授信起始时间
                    ,time4="'"'2021-05-02'"'" ##实验组授信终止时间
                    ,inputdate1="'"'20210419'"'"
                    ,inputdate2="'"'20210425'"'" 
                    ,inputdate3="'"'20210426'"'"
                    ,inputdate4="'"'20210502'"'"
                    )

creditScore46运行时间为: 299.0796673297882
creditScore46运行时间为: 249.7176158428192
creditScore46运行时间为: 277.78392601013184
creditScore46运行时间为: 288.48068475723267


D:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


creditScore48运行时间为: 232.99937200546265
creditScore48运行时间为: 217.0727243423462
creditScore48运行时间为: 182.83449935913086
creditScore48运行时间为: 146.8711178302765
creditScore49运行时间为: 123.04779028892517
creditScore49运行时间为: 122.76220846176147
creditScore49运行时间为: 141.06771802902222
creditScore49运行时间为: 135.60168528556824
creditScore50运行时间为: 148.64898085594177
creditScore50运行时间为: 117.55281376838684
creditScore50运行时间为: 134.5396807193756
creditScore50运行时间为: 124.23184561729431
creditScore51运行时间为: 122.1300675868988
creditScore51运行时间为: 119.011794090271
creditScore51运行时间为: 124.62114787101746
creditScore51运行时间为: 249.73115730285645
creditScore52运行时间为: 196.62289714813232
creditScore52apply_control没有数据
creditScore54运行时间为: 188.38737201690674
creditScore54运行时间为: 146.50323367118835
creditScore54运行时间为: 148.31501626968384
creditScore54运行时间为: 175.205171585083


D:\Program Files\Anaconda3\lib\site-packages\pandas\core\series.py:853: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


creditScore55运行时间为: 151.2114496231079
creditScore55运行时间为: 176.15007734298706
creditScore55运行时间为: 217.441339969635
creditScore55运行时间为: 216.97909021377563
creditScore56运行时间为: 190.62439012527466
creditScore56运行时间为: 155.66805934906006
creditScore56运行时间为: 166.37810730934143
creditScore56运行时间为: 161.2830512523651
creditScore57运行时间为: 229.65696573257446
creditScore57运行时间为: 353.24700474739075
creditScore57运行时间为: 426.20433139801025
creditScore57运行时间为: 384.79162883758545
creditScore58运行时间为: 294.74263739585876
creditScore58运行时间为: 506.5366656780243
creditScore58运行时间为: 250.0174376964569
creditScore58运行时间为: 271.79657196998596
creditScore59运行时间为: 282.55895495414734
creditScore59运行时间为: 330.1243727207184
creditScore59运行时间为: 607.298050403595
creditScore59运行时间为: 324.4996590614319
creditScore60运行时间为: 249.2023799419403
creditScore60运行时间为: 365.3001048564911
creditScore60运行时间为: 351.77348947525024
creditScore60运行时间为: 289.7197644710541
creditScore61运行时间为: 258.31740045547485
creditScore61运行时间为: 684.9573457241058


# PSI与汇总(最终使用)

In [15]:
import math
from pyecharts.components import Table
from pyecharts.options import ComponentTitleOpts
from pyecharts.charts import Bar, Grid, Line, Pie, Tab
from impala.util import as_pandas
import pandas as pd
from impala.dbapi import connect
import numpy as np
import time
import functools

def auto_run_psi(user,password,time1,time2,time3,time4,inputdate1,inputdate2,inputdate3,inputdate4):
    ###PSI计算公式(奇怪的点在于放入spyder中以公式形式调用时，会不管用？？？)
    def make_psi(SampleA,SampleB,bins = 'num'):
        Max = max(max(SampleA),max(SampleB))
        Min = min(min(SampleA),min(SampleB))
        result = pd.DataFrame()
        def get_result(bins,dropna=True):       
            Cnt_SampleA = pd.cut(SampleA,bins).value_counts(dropna=False)
            Cnt_SampleB = pd.cut(SampleB,bins).value_counts(dropna=False)
            Cnt_All_SampleA = len(SampleA)
            Cnt_All_SampleB = len(SampleB)
            result['Cnt_SampleA'] = Cnt_SampleA
            result['Cnt_SampleB'] = Cnt_SampleB
            result['Cnt_All_SampleA'] = len(SampleA)
            result['Cnt_All_SampleB'] = len(SampleB)
            result['SampleA_Rate'] = Cnt_SampleA/Cnt_All_SampleA
            result['SampleB_Rate'] = Cnt_SampleB/Cnt_All_SampleB
            #c=(result.SampleA_Rate-result.SampleB_Rate)*np.log(result.SampleA_Rate/result.SampleB_Rate)
            result['PSI'] = (result.SampleA_Rate-result.SampleB_Rate)*np.log(result.SampleA_Rate/result.SampleB_Rate)
            where_are_inf = np.isinf(result['PSI'])  ##查找是inf的PSI
            result['PSI'][where_are_inf] = 0         ##将inf转化成0
            result['PSI_Sum'] = sum(result['PSI'])
            return result
        if bins == 'num':
            bins1 = np.linspace(Min,Max,11)
            bins = [int(x) for x in bins1[1:10]]
            bins.insert(0,int(Min-1))
            bins.append(math.ceil(Max))
            result = get_result(bins,dropna=False)
        else:
            result = get_result(bins,dropna=False)
        return result
    ## 输入数据库账号
    conn = connect(host='10.1.57.8', 
                   port=10000, 
                   auth_mechanism='PLAIN',
                   user=user,
                   password = password,
                   database='fk')
    #连接数据库
    cursor=conn.cursor()
 
    ##确定需要监控的模型
    scores=['creditScore46','creditScore48','creditScore49','creditScore50','creditScore51','creditScore52','creditScore54','creditScore55',
       'creditScore56','creditScore57','creditScore58','creditScore59','creditScore60','creditScore61','creditScore62','creditScore63',
       'creditScore64','creditScore65']

    ##需要监控模型对应的中文名（注意：要一一对应）
    chinese_names= ['借呗阿波罗模型V2',
                    '借呗联合模型V2',
                    '借呗阿波罗模型V3（XGB）',
                    '借呗阿波罗模型V3（LGB）',
                    '借呗阿波罗模型V3（权重版）',
                    '京东A卡',
                    '借呗三方征信模型',
                    '借呗阿波罗入催模型',
                    '借呗负债压力模型',
                    '借呗人行二代模型',
                    '借呗融合模型-全变量',
                    '借呗融合模型-去除score56',
                    '京东plusA卡V2',
                    '借呗阿波罗V4',
                    '借呗三方征信V2',
                    '借呗阿波罗模型V1',
                    '京东plus卡A卡V3.1',
                    '京东plus卡A卡V3.2']
    ##借呗
    score_1010=['creditScore46','creditScore48','creditScore49','creditScore50','creditScore51','creditScore54','creditScore55','creditScore56',
                'creditScore57','creditScore58','creditScore59','creditScore61','creditScore62','creditScore63']
    ##京东
    score_1006=['creditScore52']
    tab = Tab()
    psi_dic={}
    
    for i, score in enumerate(scores):
        for a, chinese_name in enumerate(chinese_names):
            if i==a:        
                if score in score_1010:
                    businesstype="'"'1010'"'"
                    businesstype_china='借呗渠道'
                elif score in score_1006:
                    businesstype="'"'1006'"'"
                    businesstype_china='京东渠道'
                else:
                    businesstype="'"'2006'"'"
                    businesstype_china='京东PLUS渠道'
                # 定义  显示运行时间 的小功能
                def runtime(func):
                    @functools.wraps(func) ##标准库 functools 中的 wrap 函数用于包装函数, 不改变原有函数的功能, 仅改变原有函数的一些属性
                    def inner(sql_Strip):
                        t1 = time.time()
                        data = func(sql_Strip)
                        t2 = time.time()
                        run_time = t2 - t1
                        print(score+'运行时间为:',run_time)
                        return data
                    return inner
                # 输出SQL运行结果
                @runtime # 能展示时间
                def run_sql(sql_Strip):
                    cursor.execute(sql_Strip) #运行SQL
                    data = as_pandas(cursor)  ##  输出跑SQL结果
                    return data
                
                ###申请客群对照组sql
                apply_sql_control='''
                SELECT {score} as score
                FROM fk.v_r_risk_fico_xml_format
                WHERE  data_dt BETWEEN {time1} AND {time2}
                AND left(applicationdate,10) BETWEEN  {time1} AND {time2}
                AND {score}>0
                AND producttype={businesstype}
                and decisionstage='Apply'
                '''
                apply_control_sql_end = apply_sql_control.format(
                time1=time1,
                time2=time2,
                score=score,
                businesstype=businesstype
                )
                apply_control=run_sql(apply_control_sql_end)
            
                ##判断是否没有数据
                if len(apply_control)<=50:##可以再做调整
                    print(score+'apply_control没有数据')
                else:
                    ##申请客群实验组的sql
                    apply_sql_experiment='''
                    SELECT {score} as score
                    FROM fk.v_r_risk_fico_xml_format
                    WHERE  data_dt BETWEEN  {time3} AND {time4}
                    AND left(applicationdate,10) BETWEEN  {time3} AND {time4}
                    AND {score}>0
                    AND producttype={businesstype}
                    and decisionstage='Apply'
                    '''
                    apply_experiment_sql_end = apply_sql_experiment.format(
                    time3=time3
                    ,time4=time4
                    ,score=score
                    ,businesstype=businesstype
                    )
                    apply_experiment=run_sql(apply_experiment_sql_end)
                    if len(apply_experiment)<=50:
                        print(score+'apply_experiment没有数据')
                    else:
                        ##通过客群对照组的sql
                        pass_sql_control='''
                        SELECT score
                        FROM 
                        (select serialno
                        from fk.v_g_aprvadt_appl_bsc_inf
                        where data_dt={time2}
                        and inputdate between {inputdate1} AND {inputdate2}
                        and decisionstage='Apply'--申请
                        and businesstype={businesstype} --限制渠道 借呗
                        AND approvestatus='03'--通过的
                        )a
                        JOIN 
                        (SELECT {score} as score,objectno
                        FROM fk.v_r_risk_fico_xml_format
                        where data_dt BETWEEN  {time1} AND {time2}
                        AND left(applicationdate,10) BETWEEN  {time1} AND {time2}
                        AND {score}>0
                        AND producttype={businesstype}
                        and decisionstage='Apply'
                        )b
                        ON a.serialno=b.objectno
                        '''
                        pass_control_sql_end = pass_sql_control.format(
                        time1=time1
                        ,time2=time2
                        ,inputdate1=inputdate1
                        ,inputdate2=inputdate2
                        ,score=score
                        ,businesstype=businesstype
                        )
                        pass_control=run_sql(pass_control_sql_end)
                        if len(pass_control)<=50:
                            print(score+'pass_control没有数据')
                        else:
                            ##通过客群实验组的Sql
                            pass_sql_experiment='''
                            SELECT score
                            FROM 
                            (select serialno
                            from fk.v_g_aprvadt_appl_bsc_inf
                            where data_dt={time4}
                            and inputdate between {inputdate3} AND {inputdate4}
                            and decisionstage='Apply'
                            and businesstype={businesstype} 
                            AND approvestatus='03'
                            )a
                            JOIN 
                            (SELECT {score} as score ,objectno
                            FROM fk.v_r_risk_fico_xml_format
                            where data_dt BETWEEN  {time3} AND {time4}
                            AND left(applicationdate,10) BETWEEN  {time3} AND {time4}
                            AND {score}>0
                            AND producttype={businesstype}
                            and decisionstage='Apply'
                            )b
                            ON a.serialno=b.objectno
                            '''
                            ##运行sql
                            pass_experiment_sql_end = pass_sql_experiment.format(
                            time3=time3
                            ,time4=time4
                            ,inputdate3=inputdate3
                            ,inputdate4=inputdate4
                            ,score=score
                            ,businesstype=businesstype
                            )
                            pass_experiment=run_sql(pass_experiment_sql_end)
                            if len(pass_experiment)<=50:
                                print(score+'pass_experiment没有数据')
                            else:
                                if score!='creditScore48':
                                    ##计算psi以及格式转化
                                    apply_a=apply_control['score'].astype('float').tolist()
                                    apply_b=apply_experiment['score'].astype('float').tolist()

                                    pass_a=pass_control['score'].astype(float).tolist()
                                    pass_b=pass_experiment['score'].astype(float).tolist()
                                else:
                                    ##注意：由于48模型的分数是小数，本身无法带入PSI公式，需要特殊处理，在此采用将数据乘以10000的方式
                                    L_a_c=apply_control['score'].astype('float').tolist()
                                    apply_a=[]
                                    for i in L_a_c:
                                        apply_a.append(i*10000)

                                    L_a_e=apply_experiment['score'].astype('float').tolist()
                                    apply_b=[]
                                    for i in L_a_e:
                                        apply_b.append(i*10000)

                                    L_p_c=pass_control['score'].astype('float').tolist()
                                    pass_a=[]
                                    for i in L_p_c:
                                        pass_a.append(i*10000)

                                    L_p_e=pass_experiment['score'].astype('float').tolist()
                                    pass_b=[]
                                    for i in L_p_e:
                                        pass_b.append(i*10000)

                                ##计算PSI，并进行各种展示形式上的变化
                                apply_psi= make_psi(apply_a,apply_b)
                                apply_psi.index.name = 'score' ##给第一列加上列名
                                apply_psi.drop(['Cnt_All_SampleA','Cnt_All_SampleB'],axis=1, inplace=True)##去掉某两列
                                f=lambda x: '%.2f%%' % (x*100)
                                apply_psi[['SampleA_Rate']]=apply_psi[['SampleA_Rate']].applymap(f)##转变成百分数并保留两位小数
                                apply_psi[['SampleB_Rate']]=apply_psi[['SampleB_Rate']].applymap(f)##转变成百分数并保留两位小数
                                apply_psi[['PSI']]=round(apply_psi[['PSI']],4)##保留四位
                                apply_psi[['PSI_Sum']]=round(apply_psi[['PSI_Sum']],4)##保留四位
                                apply_psi=apply_psi.reset_index()##将index列转化成普通列
                                apply_psi['状态']=['申请psi','申请psi','申请psi','申请psi','申请psi','申请psi','申请psi','申请psi','申请psi','申请psi']
                                cols = apply_psi.columns.tolist()
                                cols = cols[-1:] + cols[:-1]
                                apply_psi=apply_psi[cols]
                                column_list=['状态', 'score', '对照组', '实验组', '对照组占比','实验组占比', 'PSI', 'PSI_Sum']
                                apply_psi.columns=column_list

                                pass_psi= make_psi(pass_a,pass_b)
                                pass_psi.index.name = 'score'##给第一列加上列名
                                pass_psi.drop(['Cnt_All_SampleA','Cnt_All_SampleB'],axis=1, inplace=True)##去掉某两列
                                pass_psi[['SampleA_Rate']]=pass_psi[['SampleA_Rate']].applymap(f)##转变成百分数并保留两位小数
                                pass_psi[['SampleB_Rate']]=pass_psi[['SampleB_Rate']].applymap(f)##转变成百分数并保留两位小数
                                pass_psi[['PSI']]=round(pass_psi[['PSI']],4)##保留四位
                                pass_psi[['PSI_Sum']]=round(pass_psi[['PSI_Sum']],4)##保留四位
                                pass_psi=pass_psi.reset_index()##将index列转化成普通列
                                pass_psi['状态']=['通过psi','通过psi','通过psi','通过psi','通过psi','通过psi','通过psi','通过psi','通过psi','通过psi']
                                cols = pass_psi.columns.tolist()
                                cols = cols[-1:] + cols[:-1]
                                pass_psi=pass_psi[cols]
                                column_list=['状态', 'score', '对照组', '实验组', '对照组占比','实验组占比', 'PSI', 'PSI_Sum']
                                pass_psi.columns=column_list

                                ##字典加数
                                psi_dic.update({str(score):[chinese_name,apply_psi['PSI_Sum'][1],pass_psi['PSI_Sum'][1],businesstype_china]})

                                ##添加该部分用于分割apply_psi以及pass_psi
                                kong={
                                '状态':['','状态'],
                                'score':['','score'], 
                                '对照组':['','对照组'],
                                '实验组':['','实验组'], 
                                '对照组占比':['','对照组占比'],
                                '实验组占比':['','实验组占比'], 
                                'PSI':['','PSI'], 
                                'PSI_Sum':['','PSI_Sum']
                                }
                                kong=pd.DataFrame(kong)
                                psi=pd.concat([apply_psi,kong,pass_psi],axis=0)
                                
                                line_psi=list(range(0,22,1))
                                for i in line_psi:
                                    line_psi[i]=psi.iloc[i].tolist()
                                headers_psi=['状态', 'score', '对照组', '实验组', '对照组占比', '实验组占比', 'PSI', 'PSI_Sum']
                                rows_psi=[line_psi[0],line_psi[1],line_psi[2],line_psi[3],line_psi[4],line_psi[5],line_psi[6],line_psi[7],line_psi[8],line_psi[9],
                                          line_psi[10],line_psi[11],line_psi[12],line_psi[13],line_psi[14],line_psi[15],line_psi[16],line_psi[17],line_psi[18],
                                          line_psi[19],line_psi[20],line_psi[21]]
                                def psi():
                                    c=(Table().add(headers_psi,rows_psi)  
                                    .set_global_opts(title_opts=ComponentTitleOpts(title=score+"-"+"对照组"+"("+time1+time2+")"+"实验组"+"("+time3+time4+")"
                                                     ,subtitle=chinese_name)))  
                                    return c
                                tab.add(psi(),score)
    
    return tab,psi_dic

#定义新函数psi(),用以合并所有creditscore的PSI和汇总的PSI，其中用到了之前定义的auto_run_psi()函数
def psi(user,password,time1,time2,time3,time4,inputdate1,inputdate2,inputdate3,inputdate4):
    tab,psi_dic=auto_run_psi(user=user##用户
                    ,password = password##密码
                    ,time1=time1 ##控制组授信起始时间，inputdate1-4与time1-4对应
                    ,time2=time2 ##控制组授信终止时间
                    ,time3=time3 ##实验组授信起始时间
                    ,time4=time4 ##实验组授信终止时间
                    ,inputdate1=inputdate1
                    ,inputdate2=inputdate2 
                    ,inputdate3=inputdate3
                    ,inputdate4=inputdate4
                    )
    ##将psi_dic字典转化为dataframe形式，方便处理
    psi_df=pd.DataFrame.from_dict(psi_dic,orient='index',columns=['Chinese_name','apply_psi','pass_psi','businesstype'])
    psi_df = psi_df.reset_index().rename(columns = {'index':'id'})
    
    ##所有模型汇总的PSI以table形式展示
    headers=['id','Chinese_name','apply_psi','pass_psi','businesstype']
    rows=[]
    line=list(range(0,len(psi_df),1))
    for i in line:
        line[i]=psi_df.iloc[i].tolist()
        rows.append(line[i])
    def psi():
        c=(Table().add(headers,rows)  
        .set_global_opts(title_opts=ComponentTitleOpts(title="PSI汇总"
                         ,subtitle="对照组"+"("+time1+"-"+time2+")"+"实验组"+"("+time3+"-"+time4+")")))                                                   
        return c
    tab.add(psi(),'PSI_all')
    return tab.render("PSI+汇总.html")

## 运行部分

In [2]:
psi(user='fk_yanjintong'##用户
    ,password = 'Yjt@0828$$'##密码
    ,time1="'"'2021-04-19'"'" ##控制组授信起始时间，inputdate1-4与time1-4对应
    ,time2="'"'2021-04-25'"'" ##控制组授信终止时间
    ,time3="'"'2021-04-26'"'" ##实验组授信起始时间
    ,time4="'"'2021-05-02'"'" ##实验组授信终止时间
    ,inputdate1="'"'20210419'"'"
    ,inputdate2="'"'20210425'"'" 
    ,inputdate3="'"'20210426'"'"
    ,inputdate4="'"'20210502'"'")

creditScore46运行时间为: 428.4075472354889
creditScore46运行时间为: 182.30483317375183
creditScore46运行时间为: 167.8858458995819
creditScore46运行时间为: 149.13893294334412


D:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


creditScore48运行时间为: 143.99682545661926
creditScore48运行时间为: 140.0156741142273
creditScore48运行时间为: 222.34501886367798
creditScore48运行时间为: 150.22818613052368
creditScore49运行时间为: 134.23974418640137
creditScore49运行时间为: 127.42193460464478
creditScore49运行时间为: 139.68558621406555
creditScore49运行时间为: 147.2909116744995
creditScore50运行时间为: 135.75010442733765
creditScore50运行时间为: 129.4161515235901
creditScore50运行时间为: 141.34859347343445
creditScore50运行时间为: 165.47335076332092
creditScore51运行时间为: 131.6489131450653
creditScore51运行时间为: 175.29038524627686
creditScore51运行时间为: 176.30559134483337
creditScore51运行时间为: 206.74385261535645
creditScore52运行时间为: 156.47385835647583
creditScore52apply_control没有数据
creditScore54运行时间为: 229.5456337928772
creditScore54运行时间为: 215.00532054901123
creditScore54运行时间为: 206.24541568756104
creditScore54运行时间为: 218.4026551246643


D:\Program Files\Anaconda3\lib\site-packages\pandas\core\series.py:853: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


creditScore55运行时间为: 171.3692536354065
creditScore55运行时间为: 171.09042763710022
creditScore55运行时间为: 189.94351887702942
creditScore55运行时间为: 217.94101071357727
creditScore56运行时间为: 140.7139129638672
creditScore56运行时间为: 157.34640836715698
creditScore56运行时间为: 137.70485281944275
creditScore56运行时间为: 137.29851412773132
creditScore57运行时间为: 128.50862669944763
creditScore57运行时间为: 131.55891227722168
creditScore57运行时间为: 141.84241223335266
creditScore57运行时间为: 152.00431513786316
creditScore58运行时间为: 134.70198273658752
creditScore58运行时间为: 144.9572970867157
creditScore58运行时间为: 196.8605194091797
creditScore58运行时间为: 136.13179230690002
creditScore59运行时间为: 151.88271164894104
creditScore59运行时间为: 144.06315207481384
creditScore59运行时间为: 137.5684905052185
creditScore59运行时间为: 156.456711769104
creditScore60运行时间为: 198.62990427017212
creditScore60运行时间为: 136.94036960601807
creditScore60运行时间为: 139.18981051445007
creditScore60运行时间为: 150.73002576828003
creditScore61运行时间为: 144.96510219573975
creditScore61运行时间为: 121.24898242

'C:\\Users\\jintong.yan\\PSI+汇总.html'

# 模型预警（IV值，没有分箱，有问题）

In [13]:
##包的加载以及连接数据库做跑数准备
import scorecardpy as sc
from pyecharts.components import Table
from pyecharts.options import ComponentTitleOpts
from pyecharts.charts import Bar, Tab
from impala.util import as_pandas
import pandas as pd
from impala.dbapi import connect
import time
import functools
from pyecharts import options as opts
from pyecharts.faker import Faker


## 输入数据库账号
conn = connect(host='10.1.57.8', 
               port=10000, 
               auth_mechanism='PLAIN',
               user='fk_yanjintong',
               password ='Yjt@0828$$' ,
               database='fk')
#连接数据库
cursor=conn.cursor()
# 定义  显示运行时间 的小功能
def runtime(func):
    @functools.wraps(func)
    def inner(sql_Strip):
        t1 = time.time()
        data = func(sql_Strip)
        t2 = time.time()
        run_time = t2 - t1
        print('运行时间为:',run_time)
        return data
    return inner
# 输出SQL运行结果
@runtime # 能展示时间
def run_sql(sql_Strip):
    cursor.execute(sql_Strip) #运行SQL
    data = as_pandas(cursor)  ##  输出跑SQL结果
    return data

In [19]:
##预警代码（以warm公式形式呈现）
def warn(psi_dic,threshold,time1,time2,time3,time4,inputdate1,inputdate2,inputdate3,inputdate4):
    ##将psi_dic字典转化为dataframe形式，方便处理
    psi_df=pd.DataFrame.from_dict(psi_dic,orient='index',columns=['apply_psi','pass_psi','businesstype'])
    psi_df = psi_df.reset_index().rename(columns = {'index':'id'})
    
    ##建立tab
    tab=Tab()
    
    ##所有模型汇总的PSI以table形式展示
    headers=['id','apply_psi','pass_psi','businesstype']
    rows=[]
    line=list(range(0,len(psi_df),1))
    for i in line:
        line[i]=psi_df.iloc[i].tolist()
        rows.append(line[i])
    def psi():
        c=(Table().add(headers,rows)  
        .set_global_opts(title_opts=ComponentTitleOpts(title="PSI汇总"
                         ,subtitle="对照组"+"("+time1+"-"+time2+")"+"实验组"+"("+time3+"-"+time4+")")))                                                   
        return c
    tab.add(psi(),'PSI_all')
    
    ##分渠道，当某一渠道的任何一个模型下apply_psi大于阈值时，触发预警机制，将对该渠道所有重要变量进行IV值预警
    jb=psi_df[(psi_df.businesstype=="'1010'")]
    jd=psi_df[(psi_df.businesstype=="'1006'")]
    jd_plus=psi_df[(psi_df.businesstype=="'2006'")]
    
    ##sql取数,取所有需要监测的变量
    control_warn_sql='''
    SELECT DISTINCT 
    a.customerid,bdmultiscorenew,txriskscore,txisauthenticatefail,tdloanorgin1m,tdloanorgin3m,tdloanorgin7d,rhmultiscore,bd_age,bd_sex_id,
    zmJoinRisk,zmRiskRating,zmSolvencyRatings,zmCustSeg,zmMobileFixedGrade,zmAdrStabilityGrade,zmDevStabilityGrade,zmTotPayAmt6mGrade,zmXfdcIndex,zmLast6mAvgAssetTotalGrade,
    zmHaveCarProbGrade,zmHaveFangProbGrade,zmOvdOrderCnt6mGrade,zmOvdOrderAmt6mGrade,zmOvdOrderDays6mGrade,zmPositiveBizCnt1yGrade,zmRepayAmt6mGrade,zmFirstLoanLengthGrade,zmRiskScore,
    zxBadBebtsCnt,zxRestDisposalCnt,zxIsCarryRecord,zxAccountNowMaxPeriod,zxRcy6MAvgUseCreditAmt,zxAccountNowTotalDelqAmt,zxTotalCreditAmt,zxHasG,zxCreCardAcctNowMaxPeriod,zxCreCardAcctNowTotalDelqAmt
    ,zxIsBadCreCardAcct,zxCreCardStatus4,zxCreCardStatus5,zxCreCardStatus3,zxCreCardStatus1,zxWarrantLoanType,zxMaxMonths,zxIsBadAccount,zxRcy1MLoanQueryCnt,zxRcy1MCreCardQueryCnt,zxRcy3MCreCardQueryCnt,zxRcy3MLoanQueryCnt,
    zxAccountCnt,zxRcy3MLoanQueryCompanyCnt,zxPersonalLoanCnt,zxCreCardUsedAmtSum,zxHouseLoan1MAvgPayAmt,zxRcy6MAvgPayAmt,zxLgctASSITEvt,zxLgctMISSTAXEvt,zxLgctENFRCEvt,zxLgctADMPENEvt,zxLgctCVLJUDEvt,zxActAtfwrnif,zxActRc,
    zxDac22Gte24MlCCPr,zxDac21Gte24MlCCPr,zxDteMAXSt12MlPLPr,zxDteMAXSt24MlPLPr,zxDteMAXSt12MlCCPr,zxDteMAXSt24MlCCPr,zxDtiMAXSt3MlCCPr,zxDtiMAXSt6MlCCPr,zxDtiMAXSt12MlCCPr,zxDtiMAXSt6MlPLPr,zxDteMAXSt60MlALLPr
    FROM 
    (SELECT customerid, serialno
    FROM fk.v_g_aprvadt_appl_bsc_inf
    WHERE data_dt={time2}
    AND  businesstype={businesstype}
    AND decisionstage='Apply'
    AND approvestatus='03'
    AND inputdate BETWEEN {inputdate1} AND {inputdate2} 
    ) a
    JOIN 
    (SELECT customer_no, report_code
    FROM fk.v_f_cv_v_report_info
    WHERE  left(update_time,10) BETWEEN {time1} AND {time2}
    )b
    ON a.customerid=b.customer_no
    JOIN 
    (SELECT report_code
    ,zxBadBebtsCnt,zxRestDisposalCnt,zxIsCarryRecord,zxAccountNowMaxPeriod,zxRcy6MAvgUseCreditAmt,zxAccountNowTotalDelqAmt,zxTotalCreditAmt,zxHasG,zxCreCardAcctNowMaxPeriod,zxCreCardAcctNowTotalDelqAmt
    ,zxIsBadCreCardAcct,zxCreCardStatus4,zxCreCardStatus5,zxCreCardStatus3,zxCreCardStatus1,zxWarrantLoanType,zxMaxMonths,zxIsBadAccount,zxRcy1MLoanQueryCnt,zxRcy1MCreCardQueryCnt,zxRcy3MCreCardQueryCnt,zxRcy3MLoanQueryCnt,
    zxAccountCnt,zxRcy3MLoanQueryCompanyCnt,zxPersonalLoanCnt,zxCreCardUsedAmtSum,zxHouseLoan1MAvgPayAmt,zxRcy6MAvgPayAmt
    FROM fk.v_f_cv_v_xingfu_phase_2
    WHERE left(update_time,10) BETWEEN {time1} AND {time2}
    )c
    ON b.report_code=c.report_code
    JOIN
    (SELECT report_code
    ,zxLgctASSITEvt,zxLgctMISSTAXEvt,zxLgctENFRCEvt,zxLgctADMPENEvt,zxLgctCVLJUDEvt
    FROM fk.v_f_cv_v_antifraud_1
    WHERE left(update_time,10) BETWEEN {time1} AND {time2}
    )d
    ON c.report_code=d.report_code
    JOIN
    (SELECT report_code
    ,zxActAtfwrnif,zxActRc
    FROM fk.v_f_cv_v_xingfu_phase_3
    WHERE left(update_time,10) BETWEEN {time1} AND {time2}
    )e
    ON c.report_code=e.report_code
    JOIN
    (SELECT report_code,zxDac22Gte24MlCCPr,zxDac21Gte24MlCCPr
    FROM fk.v_f_cv_v_repay_history_8
    WHERE left(update_time,10) BETWEEN {time1} AND {time2}
    )f
    ON c.report_code=f.report_code
    JOIN
    (SELECT report_code
    ,zxDac22Gte24MlPLPr,zxDac21Gte24MlPLPr
    FROM fk.v_f_cv_v_repay_history_5
    WHERE left(update_time,10) BETWEEN {time1} AND {time2}
    )g
    ON c.report_code=g.report_code
    JOIN
    (SELECT report_code
    ,zxDteMAXSt12MlPLPr,zxDteMAXSt24MlPLPr,zxDteMAXSt12MlCCPr,zxDteMAXSt24MlCCPr,zxDtiMAXSt3MlCCPr,zxDtiMAXSt6MlCCPr,zxDtiMAXSt12MlCCPr,zxDtiMAXSt6MlPLPr,zxDteMAXSt60MlALLPr
    FROM fk.v_f_cv_v_repay_history_1
    WHERE left(update_time,10) BETWEEN {time1} AND {time2}
    ) h
    ON c.report_code=h.report_code
    JOIN 
    (SELECT objectno,bdmultiscorenew,txriskscore,txisauthenticatefail,tdloanorgin1m,tdloanorgin3m,tdloanorgin7d,rhmultiscore,
    zmJoinRisk,zmRiskRating,zmSolvencyRatings,zmCustSeg,zmMobileFixedGrade,zmAdrStabilityGrade,zmDevStabilityGrade,zmTotPayAmt6mGrade,zmXfdcIndex,zmLast6mAvgAssetTotalGrade,
    zmHaveCarProbGrade,zmHaveFangProbGrade,zmOvdOrderCnt6mGrade,zmOvdOrderAmt6mGrade,zmOvdOrderDays6mGrade,zmPositiveBizCnt1yGrade,zmRepayAmt6mGrade,zmFirstLoanLengthGrade,zmRiskScore
    FROM fk.v_r_risk_fico_xml_format
    WHERE  data_dt BETWEEN  {time1} AND {time2}
    AND left(applicationdate,10) BETWEEN  {time1} AND {time2}
    AND producttype={businesstype}
    and decisionstage='Apply'
    )i
    ON a.serialno=i.objectno
    JOIN
    (SELECT customerid, bd_age,bd_sex_id
    FROM fk.v_g_cust_cst_bsc_inf
    WHERE data_dt={time2}
    )j
    ON a.customerid=j.customerid
    '''
    experiment_warn_sql='''
    SELECT DISTINCT 
    a.customerid,bdmultiscorenew,txriskscore,txisauthenticatefail,tdloanorgin1m,tdloanorgin3m,tdloanorgin7d,rhmultiscore,bd_age,bd_sex_id,
    zmJoinRisk,zmRiskRating,zmSolvencyRatings,zmCustSeg,zmMobileFixedGrade,zmAdrStabilityGrade,zmDevStabilityGrade,zmTotPayAmt6mGrade,zmXfdcIndex,zmLast6mAvgAssetTotalGrade,
    zmHaveCarProbGrade,zmHaveFangProbGrade,zmOvdOrderCnt6mGrade,zmOvdOrderAmt6mGrade,zmOvdOrderDays6mGrade,zmPositiveBizCnt1yGrade,zmRepayAmt6mGrade,zmFirstLoanLengthGrade,zmRiskScore,
    zxBadBebtsCnt,zxRestDisposalCnt,zxIsCarryRecord,zxAccountNowMaxPeriod,zxRcy6MAvgUseCreditAmt,zxAccountNowTotalDelqAmt,zxTotalCreditAmt,zxHasG,zxCreCardAcctNowMaxPeriod,zxCreCardAcctNowTotalDelqAmt
    ,zxIsBadCreCardAcct,zxCreCardStatus4,zxCreCardStatus5,zxCreCardStatus3,zxCreCardStatus1,zxWarrantLoanType,zxMaxMonths,zxIsBadAccount,zxRcy1MLoanQueryCnt,zxRcy1MCreCardQueryCnt,zxRcy3MCreCardQueryCnt,zxRcy3MLoanQueryCnt,
    zxAccountCnt,zxRcy3MLoanQueryCompanyCnt,zxPersonalLoanCnt,zxCreCardUsedAmtSum,zxHouseLoan1MAvgPayAmt,zxRcy6MAvgPayAmt,zxLgctASSITEvt,zxLgctMISSTAXEvt,zxLgctENFRCEvt,zxLgctADMPENEvt,zxLgctCVLJUDEvt,zxActAtfwrnif,zxActRc,
    zxDac22Gte24MlCCPr,zxDac21Gte24MlCCPr,zxDteMAXSt12MlPLPr,zxDteMAXSt24MlPLPr,zxDteMAXSt12MlCCPr,zxDteMAXSt24MlCCPr,zxDtiMAXSt3MlCCPr,zxDtiMAXSt6MlCCPr,zxDtiMAXSt12MlCCPr,zxDtiMAXSt6MlPLPr,zxDteMAXSt60MlALLPr
    FROM 
    (SELECT customerid, serialno
    FROM fk.v_g_aprvadt_appl_bsc_inf
    WHERE data_dt={time4}
    AND  businesstype={businesstype}
    AND decisionstage='Apply'
    AND approvestatus='03'
    AND inputdate BETWEEN {inputdate3} AND {inputdate4} 
    ) a
    JOIN 
    (SELECT customer_no, report_code
    FROM fk.v_f_cv_v_report_info
    WHERE  left(update_time,10) BETWEEN {time3} AND {time4}
    )b
    ON a.customerid=b.customer_no
    JOIN 
    (SELECT report_code
    ,zxBadBebtsCnt,zxRestDisposalCnt,zxIsCarryRecord,zxAccountNowMaxPeriod,zxRcy6MAvgUseCreditAmt,zxAccountNowTotalDelqAmt,zxTotalCreditAmt,zxHasG,zxCreCardAcctNowMaxPeriod,zxCreCardAcctNowTotalDelqAmt
    ,zxIsBadCreCardAcct,zxCreCardStatus4,zxCreCardStatus5,zxCreCardStatus3,zxCreCardStatus1,zxWarrantLoanType,zxMaxMonths,zxIsBadAccount,zxRcy1MLoanQueryCnt,zxRcy1MCreCardQueryCnt,zxRcy3MCreCardQueryCnt,zxRcy3MLoanQueryCnt,
    zxAccountCnt,zxRcy3MLoanQueryCompanyCnt,zxPersonalLoanCnt,zxCreCardUsedAmtSum,zxHouseLoan1MAvgPayAmt,zxRcy6MAvgPayAmt
    FROM fk.v_f_cv_v_xingfu_phase_2
    WHERE left(update_time,10) BETWEEN {time3} AND {time4}
    )c
    ON b.report_code=c.report_code
    JOIN
    (SELECT report_code
    ,zxLgctASSITEvt,zxLgctMISSTAXEvt,zxLgctENFRCEvt,zxLgctADMPENEvt,zxLgctCVLJUDEvt
    FROM fk.v_f_cv_v_antifraud_1
    WHERE left(update_time,10) BETWEEN {time3} AND {time4}
    )d
    ON c.report_code=d.report_code
    JOIN
    (SELECT report_code
    ,zxActAtfwrnif,zxActRc
    FROM fk.v_f_cv_v_xingfu_phase_3
    WHERE left(update_time,10) BETWEEN {time3} AND {time4}
    )e
    ON c.report_code=e.report_code
    JOIN
    (SELECT report_code,zxDac22Gte24MlCCPr,zxDac21Gte24MlCCPr
    FROM fk.v_f_cv_v_repay_history_8
    WHERE left(update_time,10) BETWEEN {time3} AND {time4}
    )f
    ON c.report_code=f.report_code
    JOIN
    (SELECT report_code
    ,zxDac22Gte24MlPLPr,zxDac21Gte24MlPLPr
    FROM fk.v_f_cv_v_repay_history_5
    WHERE left(update_time,10) BETWEEN {time3} AND {time4}
    )g
    ON c.report_code=g.report_code
    JOIN
    (SELECT report_code
    ,zxDteMAXSt12MlPLPr,zxDteMAXSt24MlPLPr,zxDteMAXSt12MlCCPr,zxDteMAXSt24MlCCPr,zxDtiMAXSt3MlCCPr,zxDtiMAXSt6MlCCPr,zxDtiMAXSt12MlCCPr,zxDtiMAXSt6MlPLPr,zxDteMAXSt60MlALLPr
    FROM fk.v_f_cv_v_repay_history_1
    WHERE left(update_time,10) BETWEEN {time3} AND {time4}
    ) h
    ON c.report_code=h.report_code
    JOIN 
    (SELECT objectno,bdmultiscorenew,txriskscore,txisauthenticatefail,tdloanorgin1m,tdloanorgin3m,tdloanorgin7d,rhmultiscore,
    zmJoinRisk,zmRiskRating,zmSolvencyRatings,zmCustSeg,zmMobileFixedGrade,zmAdrStabilityGrade,zmDevStabilityGrade,zmTotPayAmt6mGrade,zmXfdcIndex,zmLast6mAvgAssetTotalGrade,
    zmHaveCarProbGrade,zmHaveFangProbGrade,zmOvdOrderCnt6mGrade,zmOvdOrderAmt6mGrade,zmOvdOrderDays6mGrade,zmPositiveBizCnt1yGrade,zmRepayAmt6mGrade,zmFirstLoanLengthGrade,zmRiskScore
    FROM fk.v_r_risk_fico_xml_format
    WHERE  data_dt BETWEEN  {time3} AND {time4}
    AND left(applicationdate,10) BETWEEN  {time3} AND {time4}
    AND producttype={businesstype}
    and decisionstage='Apply'
    )i
    ON a.serialno=i.objectno
    JOIN
    (SELECT customerid, bd_age,bd_sex_id
    FROM fk.v_g_cust_cst_bsc_inf
    WHERE data_dt={time4}
    )j
    ON a.customerid=j.customerid
    '''
    ##借呗渠道，如果该渠道某一模型apply_psi>阈值，则出发预警机制，进行变量iv值查看
    if any(jb['apply_psi']>=threshold):
        businesstype="'"'1010'"'"
        control_warn_sql_end=control_warn_sql.format(
                            inputdate1=inputdate1
                            ,inputdate2=inputdate1
                            ,time1=time1
                            ,time2=time2
                            ,businesstype=businesstype
                            )
        control_warn=run_sql(control_warn_sql_end)
        experiment_warn_sql_end=experiment_warn_sql.format(
                            inputdate3=inputdate3
                            ,inputdate4=inputdate4
                            ,time3=time3
                            ,time4=time4
                            ,businesstype=businesstype
                            )
        experiment_warn=run_sql(experiment_warn_sql_end)
        
        ##给控制组和实验组数据打标，并将其进行合并
        control_warn['diff']=0
        experiment_warn['diff']=1
        warn=pd.concat([control_warn,experiment_warn],axis=0)
        warn.drop(['customerid'],inplace=True,axis=1)
        
        ##计算IV值
        K =sc.iv(warn, y='diff')
        K_0=K.sort_values(by='info_value',ascending=False)##按照IV值由大到小顺序排列，下同
        i=round(K_0['info_value'],4).tolist()[:10]##选取前十个变量，下同
        v=K_0['variable'].tolist()[:10]
        def bar1():
            c = (
                Bar(init_opts=opts.InitOpts(width="2000px"))##以防图中横坐标没有办法全部展示，如果仍无法展示全，可以改成2500px甚至更大
                .add_xaxis(v)
                .add_yaxis("importance", i, color=Faker.rand_color())
                .set_global_opts(xaxis_opts=opts.AxisOpts(axislabel_opts={"interval":"0"}),
                                 title_opts=opts.TitleOpts(title=businesstype+"渠道预警"),
                                 datazoom_opts=opts.DataZoomOpts(orient="vertical")
                                  )
                 )
            return c
        tab.add(bar1(), "IV预警—借呗（1010）")
    else:
        print("借呗渠道PSI均小于阈值")
    
    ##京东渠道，如果该渠道某一模型apply_psi>阈值，则出发预警机制，进行变量iv值查看 
    if any(jd['apply_psi']>=threshold):
        businesstype="'"'1006'"'"
        control_warn_sql_end=control_warn_sql.format(
                            inputdate1=inputdate1
                            ,inputdate2=inputdate1
                            ,time1=time1
                            ,time2=time2
                            ,businesstype=businesstype
                            )
        control_warn=run_sql(control_warn_sql_end)
        experiment_warn_sql_end=experiment_warn_sql.format(
                            inputdate3=inputdate3
                            ,inputdate4=inputdate4
                            ,time3=time3
                            ,time4=time4
                            ,businesstype=businesstype
                            )
        experiment_warn=run_sql(experiment_warn_sql_end)
        
        ##给控制组和实验组数据打标，并将其进行合并
        control_warn['diff']=0
        experiment_warn['diff']=1
        warn=pd.concat([control_warn,experiment_warn],axis=0)
        warn.drop(['customerid'],inplace=True,axis=1)
        ##计算IV值
        K =sc.iv(warn, y='diff')
        K_0=K[K['info_value']>=0.01].sort_values(by='info_value')
        i=round(K_0['info_value'],4).tolist()[:10]##选取前十个变量，下同
        v=K_0['variable'].tolist()[:10]
        def bar2():
            c = (
                Bar(init_opts=opts.InitOpts(width="2000px"))
                .add_xaxis(v)
                .add_yaxis("importance", i, color=Faker.rand_color())
                .set_global_opts(xaxis_opts=opts.AxisOpts(axislabel_opts={"interval":"0"}),
                                 title_opts=opts.TitleOpts(title=businesstype+"渠道预警"),
                                 datazoom_opts=opts.DataZoomOpts(orient="vertical")
                                  )
                 )
            return c
        tab.add(bar2(), "IV预警—京东（1006）")
    else:
        print("京东渠道PSI均小于阈值")
    
    ##京东PLUS渠道，如果该渠道某一模型apply_psi>阈值，则出发预警机制，进行变量iv值查看   
    if any(jd_plus['apply_psi']>=threshold):
        businesstype="'"'2006'"'"
        control_warn_sql_end=control_warn_sql.format(
                            inputdate1=inputdate1
                            ,inputdate2=inputdate1
                            ,time1=time1
                            ,time2=time2
                            ,businesstype=businesstype
                            )
        control_warn=run_sql(control_warn_sql_end)
        experiment_warn_sql_end=experiment_warn_sql.format(
                            inputdate3=inputdate3
                            ,inputdate4=inputdate4
                            ,time3=time3
                            ,time4=time4
                            ,businesstype=businesstype
                            )
        experiment_warn=run_sql(experiment_warn_sql_end)
        ##给控制组和实验组数据打标，并将其进行合并
        control_warn['diff']=0
        experiment_warn['diff']=1
        warn=pd.concat([control_warn,experiment_warn],axis=0)
        warn.drop(['customerid'],inplace=True,axis=1)
        ##计算Iv值
        K =sc.iv(warn, y='diff')
        K_0=K[K['info_value']>=0.01].sort_values(by='info_value')
        i=round(K_0['info_value'],4).tolist()[:10]##选取前十个变量，下同
        v=K_0['variable'].tolist()[:10]
        def bar3():
            c = (
                Bar(init_opts=opts.InitOpts(width="2000px"))
                .add_xaxis(v)
                .add_yaxis("importance", i, color=Faker.rand_color())
                .set_global_opts(xaxis_opts=opts.AxisOpts(axislabel_opts={"interval":"0"}),
                                 title_opts=opts.TitleOpts(title=businesstype+"渠道预警"),
                                 datazoom_opts=opts.DataZoomOpts(orient="vertical")
                                  )
                 )
            return c
        tab.add(bar3(), "IV预警—京东PLUS(2006)")
    else:
        print("京东PLUS渠道PSI均小于阈值")
    return tab.render("IV预警.html")

## 运行部分

In [20]:
warn(psi_dic
    ,threshold=0.1
    ,time1="'"'2021-03-29'"'"
    ,time2="'"'2021-04-04'"'" 
    ,time3="'"'2021-04-05'"'"
    ,time4="'"'2021-04-11'"'"
    ,inputdate1="'"'20210329'"'"
    ,inputdate2="'"'20210404'"'" 
    ,inputdate3="'"'20210405'"'"
    ,inputdate4="'"'20210411'"'"
    )

借呗渠道PSI均小于阈值
京东渠道PSI均小于阈值
京东PLUS渠道PSI均小于阈值


'C:\\Users\\jintong.yan\\IV预警.html'

# Lift值

In [6]:
import math
from public.Score import make_lift
from pyecharts.components import Table
from pyecharts.options import ComponentTitleOpts
from pyecharts.charts import Bar, Grid, Line, Pie, Tab
from impala.util import as_pandas
import pandas as pd
from impala.dbapi import connect
import numpy as np
import time
import functools

def auto_run_lift(user,password,inputdate1,inputdate2,time1,time55_1):
    ## 输入数据库账号
    conn = connect(host='10.1.57.8', 
                   port=10000, 
                   auth_mechanism='PLAIN',
                   user=user,
                   password = password,
                   database='fk')
    #连接数据库
    cursor=conn.cursor()
    
    tab = Tab()
    ##确定需要监控的模型
#     scores=['creditScore46','creditScore48','creditScore49','creditScore50','creditScore51','creditScore52','creditScore54','creditScore55',
#        'creditScore56','creditScore57','creditScore58','creditScore59','creditScore60','creditScore61','creditScore62','creditScore63',
#        'creditScore64','creditScore65']
    scores=['creditScore55']
    chinese_names= ['借呗阿波罗模型V2']
    #需要监控模型对应的中文名（注意：要一一对应）
#     chinese_names= ['借呗阿波罗模型V2',
#                     '借呗联合模型V2',
#                     '借呗阿波罗模型V3（XGB）',
#                     '借呗阿波罗模型V3（LGB）',
#                     '借呗阿波罗模型V3（权重版）',
#                     '京东A卡',
#                     '借呗三方征信模型',
#                     '借呗阿波罗入催模型',
#                     '借呗负债压力模型',
#                     '借呗人行二代模型',
#                     '借呗融合模型-全变量',
#                     '借呗融合模型-去除score56',
#                     '京东plusA卡V2',
#                     '借呗阿波罗V4',
#                     '借呗三方征信V2',
#                     '借呗阿波罗模型V1',
#                     '京东plus卡A卡V3.1',
#                     '京东plus卡A卡V3.2']
    ##借呗
    score_1010=['creditScore46','creditScore48','creditScore49','creditScore50','creditScore51','creditScore54','creditScore55','creditScore56',
                'creditScore57','creditScore58','creditScore59','creditScore61','creditScore62','creditScore63']
    ##京东
    score_1006=['creditScore52']
    tab = Tab()
    for i, score in enumerate(scores):
        for a, chinese_name in enumerate(chinese_names):
            if i==a:        
                if score in score_1010:
                    businesstype="'"'1010'"'"
                elif score in score_1006:
                    businesstype="'"'1006'"'"
                else:
                    businesstype="'"'2006'"'"
                # 定义  显示运行时间 的小功能
                def runtime(func):
                    @functools.wraps(func)
                    def inner(sql_Strip):
                        t1 = time.time()
                        data = func(sql_Strip)
                        t2 = time.time()
                        run_time = t2 - t1
                        print(score+'运行时间为:',run_time)
                        return data
                    return inner
                # 输出SQL运行结果
                @runtime # 能展示时间
                def run_sql(sql_Strip):
                    cursor.execute(sql_Strip) #运行SQL
                    data = as_pandas(cursor)  ##  输出跑SQL结果
                    return data
                ###sql
                bad_sql='''
                SELECT a.customerid,score,y
                FROM 
                (SELECT customerid ,serialno 
                FROM fk.v_g_aprvadt_appl_bsc_inf 
                WHERE businesstype={businesstype} 
                AND inputdate BETWEEN {inputdate1} and {inputdate2}
                AND approvestatus='03'
                AND decisionstage='Apply'
                AND data_dt= {time1}
                ) a
                JOIN 
                (SELECT customerid,max(bd_hist_high_odue_days) as y
                FROM fk.v_g_loanlend_loan_inf_his 
                WHERE  {time1}> date(bd_fst_maturity_dt)+{days} 
                AND data_dt={time1} 
                GROUP BY customerid
                )b
                ON  a.customerid=b.customerid
                JOIN 
                (SELECT objectno,{score} as score
                from fk.v_r_risk_fico_xml_format
                where data_dt<={time1}
                AND {score}>=0
                )c
                on a.serialno=c.objectno
                '''
                if score=='creditScore55':
                    bad_sql_end=bad_sql.format(
                    businesstype=businesstype
                    ,inputdate1=inputdate1
                    ,inputdate2=inputdate2
                    ,time1=time55_1
                    ,score=score
                    ,days=7
                    )
                    bad=run_sql(bad_sql_end)
                else:
                    bad_sql_end=bad_sql.format(
                    businesstype=businesstype
                    ,inputdate1=inputdate1
                    ,inputdate2=inputdate2
                    ,time1=time1
                    ,score=score
                    ,days=90
                    )
                    bad=run_sql(bad_sql_end)
                if len(bad['score'])<=50:##判断是否没有数据
                    print(score+'没有数据')
                else:
                    a=bad['score'].astype('float').tolist()
                    c=sum(a)
                    if c==0:##判断数据是否全为0
                        print(score+'所有score都是0')
                    else:
                        if score=='creditScore55':
                            bad['y'] = bad['y'].apply(lambda x: 1 if x>=1 else 0 )
                        else:
                            bad['y'] = bad['y'].apply(lambda x: 1 if x>29 else 0 )
                        if score!='creditScore48':
                            ##计算lift前的格式转化
                            SCORE=bad['score'].astype(float).tolist()
                        else:
                            ##注意：由于48模型的分数是小数，经特殊处理，在此采用将数据乘以10000的方式
                            score1=bad['score'].astype('float').tolist()
                            SCORE=[]
                            for i in score1:
                                SCORE.append(i*10000)
                                
                        ##计算LIFT
                        lift = make_lift(bad['y'],SCORE) # 全体y的woe,全体数据分数
                        def compose(a,b):
                            return (a,b)
                        lift['Score']=lift.apply(lambda row:compose(row['最低分'],row['最高分']),axis=1)
                        lift.drop(['分位数','最低分','最高分','Y=1累计占比','累计提升率'],axis=1, inplace=True)##去掉某几列
                        f=lambda x: '%.2f%%' % (x*100)
                        lift[['Y=1占比']]=lift[['Y=1占比']].applymap(f)##转变成百分数并保留两位小数
                        lift[['提升率']]=round(lift[['提升率']],4)##保留四位
                        cols = lift.columns.tolist()
                        cols = cols[-1:] + cols[:-1]
                        lift=lift[cols]
                        
                        line=list(range(0,10,1))
                        for i in line:
                            line[i]=lift.iloc[i].tolist()
                        headers = ['Score', '全体数', 'Y=1观测数', 'Y=1占比', '提升率']
                        rows=[line[0],
                          line[1],
                          line[2],
                          line[3],
                          line[4],
                          line[5],
                          line[6],
                          line[7],
                          line[8],
                          line[9]
                             ]
                        def lift():
                            c=(Table().add(headers,rows)  
                            .set_global_opts(title_opts=ComponentTitleOpts(title=score+"("+inputdate1+"-"+inputdate2+")"
                                             ,subtitle=chinese_name)))
                            return c
                        tab.add(lift(),score)
            else:
                pass
    return tab.render("LIFT.html")

## 运行部分

In [7]:
auto_run_lift(user='fk_yanjintong'##用户
              ,password = 'Yjt@0828$$'##密码
              ,inputdate1="'"'20201101'"'"
              ,inputdate2="'"'20201201'"'"
              ,time1="'"'2021-04-01'"'"
              ,time55_1="'"'2021-01-08'"'")

creditScore55运行时间为: 557.31001496315


'C:\\Users\\jintong.yan\\LIFT.html'

# Lift模型效力（每月）

In [ ]:
from public.Score import make_lift
from impala.util import as_pandas
from impala.dbapi import connect
import time
import functools
from pyecharts import options as opts
from pyecharts.charts import Line, Tab

def auto_run_effect(user,password,inputdate1_1,inputdate1_2,inputdate2_1,inputdate2_2,inputdate3_1,inputdate3_2,time1,time2,time3):
    ## 输入数据库账号
    conn = connect(host='10.1.57.8', 
                   port=10000, 
                   auth_mechanism='PLAIN',
                   user=user,
                   password = password,
                   database='fk')
    #连接数据库
    cursor=conn.cursor()
    
    tab = Tab()
    ##确定需要监控的模型
    scores=['creditScore46','creditScore48','creditScore49','creditScore50','creditScore51','creditScore52','creditScore54','creditScore55',
       'creditScore56','creditScore57','creditScore58','creditScore59','creditScore60','creditScore61','creditScore62','creditScore63',
       'creditScore64','creditScore65']
    #需要监控模型对应的中文名（注意：要一一对应）
    chinese_names= ['借呗阿波罗模型V2',
                    '借呗联合模型V2',
                    '借呗阿波罗模型V3（XGB）',
                    '借呗阿波罗模型V3（LGB）',
                    '借呗阿波罗模型V3（权重版）',
                    '京东A卡',
                    '借呗三方征信模型',
                    '借呗阿波罗入催模型',
                    '借呗负债压力模型',
                    '借呗人行二代模型',
                    '借呗融合模型-全变量',
                    '借呗融合模型-去除score56',
                    '京东plusA卡V2',
                    '借呗阿波罗V4',
                    '借呗三方征信V2',
                    '借呗阿波罗模型V1',
                    '京东plus卡A卡V3.1',
                    '京东plus卡A卡V3.2']
    ##借呗
    score_1010=['creditScore46','creditScore48','creditScore49','creditScore50','creditScore51','creditScore54','creditScore55','creditScore56',
                'creditScore57','creditScore58','creditScore59','creditScore61','creditScore62','creditScore63']
    ##京东
    score_1006=['creditScore52']
    tab = Tab()
    for i, score in enumerate(scores):
        for a, chinese_name in enumerate(chinese_names):
            if i==a:        
                if score in score_1010:
                    businesstype="'"'1010'"'"
                elif score in score_1006:
                    businesstype="'"'1006'"'"
                else:
                    businesstype="'"'2006'"'"
                # 定义  显示运行时间 的小功能
                def runtime(func):
                    @functools.wraps(func)
                    def inner(sql_Strip):
                        t1 = time.time()
                        data = func(sql_Strip)
                        t2 = time.time()
                        run_time = t2 - t1
                        print(score+'运行时间为:',run_time)
                        return data
                    return inner
                # 输出SQL运行结果
                @runtime # 能展示时间
                def run_sql(sql_Strip):
                    cursor.execute(sql_Strip) #运行SQL
                    data = as_pandas(cursor)  ##  输出跑SQL结果
                    return data
                ###sql
                sql_1='''
                SELECT DISTINCT  a.customerid,score, y
                FROM 
                (SELECT customerid, serialno
                FROM fk.v_g_aprvadt_appl_bsc_inf
                WHERE data_dt={time1}
                AND businesstype={businesstype}
                AND decisionstage='Apply'
                AND approvestatus='03'
                AND inputdate BETWEEN {inputdate1_1} AND {inputdate1_2}
                ) a
                JOIN
                (SELECT customerid, max(bd_hist_high_odue_days) AS y
                FROM fk.v_g_loanlend_loan_inf_his
                WHERE data_dt={time1}
                AND businesstype={businesstype}
                AND {time1}>date_add(tdh_todate(bd_fst_maturity_dt),90) 
                GROUP BY 1
                )c
                ON a.customerid=c.customerid
                JOIN 
                (SELECT objectno,{score} AS score
                FROM fk.v_r_risk_fico_xml_format
                WHERE data_dt<={time1}
                AND producttype={businesstype}
                AND {score}>0
                )b
                ON a.serialno=b.objectno
                '''
                sql_1_end=sql_1.format(
                businesstype=businesstype
                ,inputdate1_1=inputdate1_1
                ,inputdate1_2=inputdate1_2
                ,time1=time1
                ,score=score
                )
                number1=run_sql(sql_1_end)
                if len(number1)<=50:##判断是否没有数据
                    print(score+'('+inputdate1_1+'-'+inputdate1_2+')'+'没有数据')
                else:
                    a=number1['score'].astype('float').tolist()
                    c=sum(a)
                    if c==0:##判断数据是否全为0
                        print(score+'('+inputdate1_1+'-'+inputdate1_2+')'+'中的所有score都是0')
                    else:
                        sql_2='''
                        SELECT DISTINCT a.customerid, score, y
                        FROM 
                        (SELECT customerid, serialno
                        FROM fk.v_g_aprvadt_appl_bsc_inf
                        WHERE data_dt={time2}
                        AND businesstype={businesstype}
                        AND decisionstage='Apply'
                        AND approvestatus='03'
                        AND inputdate BETWEEN {inputdate2_1} AND {inputdate2_2}
                        ) a
                        JOIN
                        (SELECT customerid, max(bd_hist_high_odue_days) AS y
                        FROM fk.v_g_loanlend_loan_inf_his
                        WHERE data_dt={time2}
                        AND businesstype={businesstype}
                        AND {time2}>date_add(tdh_todate(bd_fst_maturity_dt),90) 
                        GROUP BY 1
                        )c
                        ON a.customerid=c.customerid
                        JOIN 
                        (SELECT objectno,{score} AS score
                        FROM fk.v_r_risk_fico_xml_format
                        WHERE data_dt<={time2}
                        AND producttype={businesstype}
                        AND {score}>0
                        )b
                        ON a.serialno=b.objectno
                        '''
                        sql_2_end=sql_2.format(
                        businesstype=businesstype
                        ,inputdate2_1=inputdate2_1
                        ,inputdate2_2=inputdate2_2
                        ,time2=time2
                        ,score=score
                        )
                        number2=run_sql(sql_2_end)
                        if len(number2)<=50:##判断是否没有数据
                            print(score+'('+inputdate2_1+'-'+inputdate2_2+')'+'没有数据')
                        else:
                            a=number2['score'].astype('float').tolist()
                            c=sum(a)
                            if c==0:##判断数据是否全为0
                                print(score+'('+inputdate2_1+'-'+inputdate2_2+')'+'中的所有score都是0')
                            else:
                                sql_3='''
                                SELECT DISTINCT a.customerid, score, y
                                FROM 
                                (SELECT customerid, serialno
                                FROM fk.v_g_aprvadt_appl_bsc_inf
                                WHERE data_dt={time3}
                                AND businesstype={businesstype}
                                AND decisionstage='Apply'
                                AND approvestatus='03'
                                AND inputdate BETWEEN {inputdate3_1} AND {inputdate3_2}
                                ) a
                                JOIN
                                (SELECT customerid, max(bd_hist_high_odue_days) AS y
                                FROM fk.v_g_loanlend_loan_inf_his
                                WHERE data_dt={time3}
                                AND businesstype={businesstype}
                                AND {time3}>date_add(tdh_todate(bd_fst_maturity_dt),90) 
                                GROUP BY 1
                                )c
                                ON a.customerid=c.customerid
                                JOIN 
                                (SELECT objectno,{score} AS score
                                FROM fk.v_r_risk_fico_xml_format
                                WHERE data_dt<={time3}
                                AND producttype={businesstype}
                                AND {score}>0
                                )b
                                ON a.serialno=b.objectno
                                '''

                                sql_3_end=sql_3.format(
                                businesstype=businesstype
                                ,inputdate3_1=inputdate3_1
                                ,inputdate3_2=inputdate3_2
                                ,time3=time3
                                ,score=score
                                )
                                number3=run_sql(sql_3_end)
                                if len(number3)<=50:##判断是否没有数据
                                    print(score+'('+inputdate3_1+'-'+inputdate3_2+')'+'没有数据')
                                else:
                                    a=number3['score'].astype('float').tolist()
                                    c=sum(a)
                                    if c==0:##判断数据是否全为0
                                        print(score+'('+inputdate3_1+'-'+inputdate3_2+')'+'中的所有score都是0')
                                    else:
                                        number1['y'] = number1['y'].apply(lambda x: 1 if x>29 else 0 )
                                        number2['y'] = number2['y'].apply(lambda x: 1 if x>29 else 0 )
                                        number3['y'] = number3['y'].apply(lambda x: 1 if x>29 else 0 )
                                        if score!='creditScore48':
                                            ##计算lift前的格式转化
                                            SCORE1=number1['score'].astype(float).tolist()
                                            SCORE2=number2['score'].astype(float).tolist()
                                            SCORE3=number3['score'].astype(float).tolist()
                                        else:
                                            ##注意：由于48模型的分数是小数，经特殊处理，在此采用将数据乘以10000的方式
                                            score1=number1['score'].astype('float').tolist()
                                            SCORE1=[]
                                            for i in score1:
                                                SCORE1.append(i*10000)
                                            score2=number2['score'].astype('float').tolist()
                                            SCORE2=[]
                                            for i in score2:
                                                SCORE2.append(i*10000)
                                            score3=number3['score'].astype('float').tolist()
                                            SCORE3=[]
                                            for i in score3:
                                                SCORE3.append(i*10000)

                                        ##计算LIFT
                                        lift1= make_lift(number1['y'],SCORE1) 
                                        lift2= make_lift(number2['y'],SCORE2)
                                        lift3= make_lift(number3['y'],SCORE3)
                                        lift1[['提升率']]=round(lift1[['提升率']],4)##保留四位
                                        Lift1=lift1['提升率'].tolist()
                                        lift2[['提升率']]=round(lift2[['提升率']],4)##保留四位
                                        Lift2=lift2['提升率'].tolist()
                                        lift3[['提升率']]=round(lift3[['提升率']],4)##保留四位
                                        Lift3=lift3['提升率'].tolist()
                                        
                                        def line() -> Line:
                                            c = (
                                                Line()
                                                .add_xaxis(xaxis_data=[0,1,2,3,4,5,6,7,8,9])
                                                .add_yaxis(
                                                    series_name="Lift1"+"("+inputdate1_1+"-"+inputdate1_2+")",
                                                    y_axis=Lift1,is_smooth=True
                                                    )
                                                .add_yaxis(
                                                    series_name="Lift2"+"("+inputdate2_1+"-"+inputdate2_2+")",
                                                    y_axis=Lift2,is_smooth=True
                                                    )
                                                .add_yaxis(
                                                    series_name="Lift3"+"("+inputdate3_1+"-"+inputdate3_2+")",
                                                    y_axis=Lift3,is_smooth=True
                                                    )
                                                .set_global_opts(title_opts=opts.TitleOpts(title="模型效力图"
                                                         ,subtitle=score+"("+chinese_name+")")))
                                            return c                                    
                                        tab.add(line(), score)

    return tab.render("效力图.html")

## 运行部分

In [ ]:
auto_run_effect(user='fk_yanjintong'##用户
                    ,password = 'Yjt@0828$$'##密码
                    ,inputdate1_1="'"'20200901'"'"
                    ,inputdate1_2="'"'20201001'"'"
                    ,inputdate2_1="'"'20201001'"'" 
                    ,inputdate2_2="'"'20201101'"'"
                    ,inputdate3_1="'"'20201101'"'"
                    ,inputdate3_2="'"'20201201'"'"
                    ,time1="'"'2021-02-01'"'"
                    ,time2="'"'2021-03-01'"'" 
                    ,time3="'"'2021-04-01'"'"
                    )

# Badrate监控

In [5]:
from pyecharts.components import Table
from pyecharts.options import ComponentTitleOpts
from pyecharts.charts import Tab
from impala.util import as_pandas
from impala.dbapi import connect
import time
import functools

def auto_run_badrate(user,password,inputdate1,inputdate2,time1,time55):  
    ## 输入数据库账号
    conn = connect(host='10.1.57.8', 
                   port=10000, 
                   auth_mechanism='PLAIN',
                   user=user,
                   password = password,
                   database='fk')
    #连接数据库
    cursor=conn.cursor()
    
    tab = Tab()
    ##确定需要监控的模型
    creditscores=['creditScore46','creditScore48','creditScore49','creditScore50','creditScore51','creditScore52','creditScore54',
                  'creditScore55','creditScore56','creditScore57','creditScore58','creditScore59','creditScore60','creditScore61',
                  'creditScore62','creditScore63','creditScore64','creditScore65']
    
    score_cuts=["when score >0  and score <=456 then '(0,456]' when score >456 and score <=477 then '(456,477]' when score >477 and score <=498 then '(477,498]' when score >498 and score <=514 then '(498,514]' when score >514 and score <=529 then '(514,529]' when score >529 and score <=544 then '(529,544]' when score >544 and score <=561 then '(544,561]' when score >561 and score <=579 then '(561,579]' when score >579 and score <=601 then '(579,601]' when score >601  then '(601,-)'" 
    ,"when score >0  and score <=0.0031 then  '(0,0.0031]' when score >0.0031 and score <=0.0061 then '(0.0031,0.0061]' when score >0.0061 and score <=0.0091 then '(0.0061,0.0091]' when score >0.0091 and score <=0.0121 then '(0.0091,0.0121]' when score >0.0121 and score <=0.0151 then '(0.0121,0.0151]' when score >0.0151 and score <=0.0181 then '(0.0151,0.0181]' when score >0.0181 and score <=0.0211 then '(0.0181,0.0211]' when score >0.0211 and score <=0.0241 then '(0.0211,0.0241]' when score >0.0241 and score <=0.0271 then '(0.0241,0.0271]' when score >0.0271 then '(0.0271,-]'"
    ,"when score >0  and score <=555 then  '(0,555]' when score >555 and score <=569 then '(555,569]' when score >569 and score <=578 then '(569,578]' when score >578 and score <=582 then '(578,582]' when score >582 and score <=587 then '(582,587]' when score >587 and score <=595 then '(587,595]' when score >595 and score <=605 then '(595,605]' when score >605 and score <=619 then '(605,619]' when score >619 and score <=629 then '(619,629]' when score >629  then '(629,-)'"
    ,"when score >0  and score <=470 then  '(0,470]' when score >470 and score <=484 then '(470,484]' when score >484 and score <=495 then '(484,495]' when score >495 and score <=503 then '(495,503]' when score >503 and score <=528 then '(503,528]' when score >528 and score <=539 then '(528,539]' when score >539 and score <=546 then '(539,546]' when score >546 and score <=612 then '(546,612]' when score >612 and score <=632 then '(612,632]' when score >632  then '(632,-)'"
    ,"when score >0  and score <=425 then  '(0,425]' when score >425 and score <=441 then '(425,441]' when score >441 and score <=449 then '(441,449]' when score >449 and score <=461 then '(449,461]' when score >461 and score <=474 then '(461,474]' when score >474 and score <=490 then '(474,490]' when score >490 and score <=511 then '(490,511]' when score >511 and score <=532 then '(511,532]' when score >532 and score <=557 then '(532,557]' when score >557  then '(557,-)'"
    ,"when score >0  and score <=672 then  '(0,672]' when score >672 and score <=685 then '(672,685]' when score >685 and score <=695 then '(685,695]' when score >695 and score <=704 then '(695,704]' when score >704 and score <=713 then '(704,713]' when score >713 and score <=723 then '(713,723]' when score >723 and score <=735 then '(723,735]' when score >735 and score <=750 then '(735,750]' when score >750 and score <=771 then '(750,771]' when score >771  then '(771,-)'"
    ,"when score >0  and score <=462 then  '(0,462]' when score >462 and score <=469 then '(462,469]' when score >469 and score <=475 then '(469,475]' when score >475 and score <=480 then '(475,480]' when score >480 and score <=485 then '(480,485]' when score >485 and score <=489 then '(485,489]' when score >489 and score <=496 then '(489,496]' when score >496 and score <=503 then '(496,503]' when score >503 and score <=514 then '(503,514]' when score >514  then '(514,-)'"
    ,"when score >0  and score <=379 then  '(0,379]' when score >379 and score <=390 then '(379,390]' when score >390 and score <=398 then '(390,398]' when score >398 and score <=409 then '(398,409]' when score >409 and score <=415 then '(409,415]' when score >415 and score <=425 then '(415,425]' when score >425 and score <=437 then '(425,437]' when score >437 and score <=451 then '(437,451]' when score >451 and score <=472 then '(451,472]' when score >472  then '(472,-)'"
    ,"when score >0  and score <=511 then  '(0,511]' when score >511 and score <=520 then '(511,520]' when score >520 and score <=525 then '(520,525]' when score >525 and score <=535 then '(525,435]' when score >535 and score <=541 then '(535,541]' when score >541 and score <=551 then '(541,551]' when score >551 and score <=558 then '(551,558]' when score >558 and score <=568 then '(558,568]' when score >568 and score <=578 then '(568,578]' when score >578  then '(578,-)'"
    ,"when score >0  and score <=539 then  '(0,539]' when score >539 and score <=563 then '(539,563]' when score >563 and score <=587 then '(563,587]' when score >587 and score <=611 then '(587,611]' when score >611 and score <=635 then '(611,635]' when score >635 and score <=659 then '(635,659]' when score >659 and score <=683 then '(659,683]' when score >683 and score <=707 then '(683,707]' when score >707 and score <=731 then '(707,731]' when score >731  then '(731,-)'"
    ,"when score >0  and score <=412 then  '(0,412]' when score >412 and score <=460 then '(412,460]' when score >460 and score <=508 then '(460,508]' when score >508 and score <=556 then '(508,556]' when score >556 and score <=604 then '(556,604]' when score >604 and score <=652 then '(604,652]' when score >652 and score <=700 then '(652,700]' when score >700 and score <=748 then '(700,748]' when score >748 and score <=796 then '(748,796]' when score >796  then '(796,-)'"
    ,"when score >0  and score <=413 then  '(0,413]' when score >413 and score <=463 then '(413,463]' when score >463 and score <=513 then '(463,513]' when score >513 and score <=562 then '(513,562]' when score >562 and score <=612 then '(562,612]' when score >612 and score <=662 then '(612,662]' when score >662 and score <=712 then '(662,712]' when score >712 and score <=761 then '(712,761]' when score >761 and score <=796 then '(761,796]' when score >796  then '(796,-)'"
    ,"when score >0  and score <=584 then  '(0,584]' when score >584 and score <=592 then '(584,592]' when score >592 and score <=596 then '(592,596]' when score >596 and score <=600 then '(596,600]' when score >600 and score <=604 then '(600,604]' when score >604 and score <=608 then '(604,608]' when score >608 and score <=613 then '(608,613]' when score >613 and score <=618 then '(613,618]' when score >618 and score <=624 then '(618,624]' when score >624  then '(624,-)'"
    ,"when score >0  and score <=430 then  '(0,430]' when score >430 and score <=442 then '(430,442]' when score >442 and score <=454 then '(442,454]' when score >454 and score <=461 then '(454,461]' when score >461 and score <=466 then '(461,466]' when score >466 and score <=476 then '(466,476]' when score >476 and score <=498 then '(476,498]' when score >498 and score <=521 then '(498,521]' when score >521 and score <=536 then '(521,536]' when score >536  then '(536,-)'"
    ,"when score >0  and score <=153 then  '(0,153]' when score >153 and score <=170 then '(153,170]' when score >170 and score <=184 then '(170,184]' when score >184 and score <=197 then '(184,197]' when score >197 and score <=209 then '(197,209]' when score >209 and score <=220 then '(209,220]' when score >220 and score <=234 then '(220,234]' when score >234 and score <=249 then '(234,249]' when score >249 and score <=269 then '(249,269]' when score >269  then '(269,-)'"
    ,"when score >0  and score <=545 then  '(0,545]' when score >545 and score <=579 then '(545,579]' when score >579 and score <=603 then '(579,603]' when score >603 and score <=629 then '(603,629]' when score >629 and score <=647 then '(629,647]' when score >647 and score <=673 then '(647,673]' when score >673 and score <=693 then '(673,693]' when score >693 and score <=723 then '(693,723]' when score >723 and score <=751 then '(723,751]' when score >751  then '(751,-)'"
    ,"when score >0  and score <=583 then  '(0,583]' when score >583 and score <=590 then '(583,590]' when score >590 and score <=595 then '(590,595]' when score >595 and score <=600 then '(595,600]' when score >600 and score <=604 then '(600,604]' when score >604 and score <=608 then '(604,608]' when score >608 and score <=612 then '(608,612]' when score >612 and score <=617 then '(612,617]' when score >617 and score <=626 then '(617,626]' when score >626  then '(626,-)'"
    ,"when score >0  and score <=581 then  '(0,581]' when score >581 and score <=589 then '(581,589]' when score >589 and score <=595 then '(589,595]' when score >595 and score <=600 then '(595,600]' when score >600 and score <=604 then '(600,604]' when score >604 and score <=609 then '(604,609]' when score >609 and score <=614 then '(609,614]' when score >614 and score <=619 then '(614,619]' when score >619 and score <=627 then '(619,627]' when score >627  then '(627,-)'"
                   ]
     #需要监控模型对应的中文名（注意：要一一对应）
    chinese_names= ['借呗阿波罗模型V2','借呗联合模型V2','借呗阿波罗模型V3（XGB）','借呗阿波罗模型V3（LGB）','借呗阿波罗模型V3（权重版）',
                   '京东A卡','借呗三方征信模型','借呗阿波罗入催模型','借呗负债压力模型','借呗人行二代模型','借呗融合模型-全变量',
                   '借呗融合模型-去除score56','京东plusA卡V2','借呗阿波罗V4','借呗三方征信V2','借呗阿波罗模型V1','京东plus卡A卡V3.1',
                   '京东plus卡A卡V3.2']
    
  
    ##借呗
    score_1010=['creditScore46','creditScore48','creditScore49','creditScore50','creditScore51','creditScore54','creditScore55','creditScore56',
                'creditScore57','creditScore58','creditScore59','creditScore61','creditScore62','creditScore63']
    ##京东
    score_1006=['creditScore52']
    tab = Tab()
    for i,creditscore in enumerate(creditscores):
        score_cut=score_cuts[i]
        chinese_name=chinese_names[i]
        if creditscore in score_1010:
            businesstype="'"'1010'"'"
        elif creditscore in score_1006:
            businesstype="'"'1006'"'"
        else:
            businesstype="'"'2006'"'"
        # 定义  显示运行时间 的小功能
        def runtime(func):
            @functools.wraps(func)
            def inner(sql_Strip):
                t1 = time.time()
                data = func(sql_Strip)
                t2 = time.time()
                run_time = t2 - t1
                print(creditscore+'运行时间为:',run_time)
                return data
            return inner
        # 输出SQL运行结果

        @runtime # 能展示时间
        def run_sql(sql_Strip):
            cursor.execute(sql_Strip) #运行SQL
            data = as_pandas(cursor)  ##  输出跑SQL结果
            return data
        ###sql
        sql_badrate='''
        SELECT 
        case 
        {score_cut}
        end as sco ,
        count( DISTINCT case when b.th>={bad} THEN b.customerid  END ) AS '坏样本',
        count( DISTINCT case when b.th>=0 THEN b.customerid  END ) AS '总数'
        FROM 
        (select customerid,serialno
        from fk.v_g_aprvadt_appl_bsc_inf
        where data_dt={time1}
        and inputdate between {inputdate1} and {inputdate2}
        and decisionstage='Apply'--申请
        and businesstype={businesstype}
        )a
        JOIN
        (SELECT customerid, max(bd_hist_high_odue_days) AS th
        FROM fk.v_g_loanlend_loan_inf_his
        WHERE data_dt={time1}
        AND {time1}> date(bd_fst_maturity_dt)+{days}
        GROUP BY 1
        ) b
        ON a.customerid=b.customerid
        Right JOIN
        (SELECT objectno,{creditscore} AS score
        FROM fk.v_r_risk_fico_xml_format
        WHERE data_dt<={time1}
        AND {creditscore}>0
        ) c
        ON a.serialno=c.objectno
        GROUP BY sco
        ORDER BY sco
        '''
        if creditscore=='creditScore55':
            sql_badrate_end = sql_badrate.format(
            score_cut=score_cut,
            time1=time55,
            inputdate1=inputdate1,
            inputdate2=inputdate2 ,
            creditscore=creditscore,
            businesstype=businesstype,
            bad='1',
            days='7'
            )
            badrate=run_sql(sql_badrate_end)
            print(badrate)
        else:
            sql_badrate_end = sql_badrate.format(
            score_cut=score_cut,
            time1=time1,
            inputdate1=inputdate1,
            inputdate2=inputdate2 ,
            creditscore=creditscore,
            businesstype=businesstype,
            bad='30',
            days='90'
            )
            badrate=run_sql(sql_badrate_end)
            print(badrate)      
        if badrate['总数'].sum()<=50:
            print(creditscore+'没有数据')
        else: 
            ##计算Badrate
            badrate['badrate']=badrate['坏样本']/badrate['总数']
            f=lambda x: '%.2f%%' % (x*100)
            badrate[['badrate']]=badrate[['badrate']].applymap(f)##转变成百分数并保留两位小数

            rows=[]
            line=list(range(0,len(badrate),1))
            for i in line:
                line[i]=badrate.iloc[i].tolist()
                rows.append(line[i])
            headers = ['Score', '坏样本', '总数', 'badrate']
            def badrate():
                c=(Table().add(headers,rows)  
                .set_global_opts(title_opts=ComponentTitleOpts(title=creditscore,subtitle=chinese_name)))               
                return c
            tab.add(badrate(),creditscore)
    return tab.render("Badrate.html")

## 运行部分

In [6]:
auto_run_badrate(user='fk_yanjintong'##用户
                    ,password = 'Yjt@0828$$'##密码
                    ,inputdate1="'"'20201201'"'"
                    ,inputdate2="'"'20210101'"'"
                    ,time1="'"'2021-05-01'"'"
                    ,time55="'"'2021-02-08'"'"
                )

creditScore46运行时间为: 223.81172156333923
         sco  坏样本    总数
0    (0,456]    0     7
1  (456,477]    2    98
2  (477,498]    3   333
3  (498,514]    7   591
4  (514,529]    9  1080
5  (529,544]   16  1181
6  (544,561]   23  2208
7  (561,579]   13  2318
8  (579,601]   35  4796
9    (601,-)   16  3022
creditScore48运行时间为: 270.67353558540344
               sco  坏样本   总数
0       (0,0.0031]    3  375
1  (0.0031,0.0061]    5  988
2  (0.0061,0.0091]    4  949
3  (0.0091,0.0121]    6  683
4  (0.0121,0.0151]    6  433
5  (0.0151,0.0181]    3  294
6  (0.0181,0.0211]    2  183
7  (0.0211,0.0241]    3  112
8  (0.0241,0.0271]    0   58
9       (0.0271,-]    1  109
creditScore49运行时间为: 244.08248209953308
         sco  坏样本    总数
0    (0,555]    0     0
1  (555,569]    0     6
2  (569,578]    1   111
3  (578,582]    2   162
4  (582,587]    4   410
5  (587,595]    7   540
6  (595,605]   28  2574
7  (605,619]   14  1985
8  (619,629]   34  3239
9    (629,-)   34  6607
creditScore50运行时间为: 615.018550157547

'C:\\Users\\jintong.yan\\Badrate.html'

# Badrate模型效力-添加趋势线（最终使用）

In [13]:
##所用到的包
import numpy as np
from scipy.optimize import leastsq
from impala.util import as_pandas
from impala.dbapi import connect
import time
import functools
from pyecharts import options as opts
from pyecharts.charts import Line,Tab

##定义函数
def effect(user,password,inputdate1_1,inputdate1_2,inputdate2_1,inputdate2_2,inputdate3_1,inputdate3_2,time1,time2,time3,time55_1,time55_2,time55_3):
    ## 输入数据库账号
    conn = connect(host='10.1.57.8', 
                   port=10000, 
                   auth_mechanism='PLAIN',
                   user=user,
                   password = password,
                   database='fk')
    #连接数据库
    cursor=conn.cursor()
    
    tab = Tab()
    #确定需要监控的模型
#     creditscores=['creditScore46','creditScore48','creditScore49','creditScore50','creditScore51',
#                   'creditScore52','creditScore54','creditScore55' ,'creditScore56','creditScore57',
#                   'creditScore58','creditScore59','creditScore60','creditScore61','creditScore62',
#                   'creditScore63','creditScore64','creditScore65'
#                  ]

#     #需要监控模型对应的中文名（注意：要一一对应）
#     chinese_names= ['借呗阿波罗模型V2',
#                     '借呗联合模型V2',
#                     '借呗阿波罗模型V3（XGB）',
#                     '借呗阿波罗模型V3（LGB）',
#                     '借呗阿波罗模型V3（权重版）',
#                     '京东A卡',
#                     '借呗三方征信模型',
#                     '借呗阿波罗入催模型',
#                     '借呗负债压力模型',
#                     '借呗人行二代模型',
#                     '借呗融合模型-全变量'
#                     ,'借呗融合模型-去除score56',
#                     '京东plusA卡V2',
#                     '借呗阿波罗V4',
#                     '借呗三方征信V2',
#                     '借呗阿波罗模型V1',
#                     '京东plus卡A卡V3.1',
#                     '京东plus卡A卡V3.2'
#                    ]
    
    creditscores=['creditScore55']
    chinese_names= ['借呗阿波罗模型V2']
    score_cuts=["when score >0  and score <=456 then '(0,456]' when score >456 and score <=477 then '(456,477]' when score >477 and score <=498 then '(477,498]' when score >498 and score <=514 then '(498,514]' when score >514 and score <=529 then '(514,529]' when score >529 and score <=544 then '(529,544]' when score >544 and score <=561 then '(544,561]' when score >561 and score <=579 then '(561,579]' when score >579 and score <=601 then '(579,601]' when score >601  then '(601,-)'" ]
    time_controls=["'"'2020-10-27'"'"]
    inputdate_control1s=["'"'20200820'"'"]
    inputdate_control2s=["'"'20200920'"'"]
#     score_cuts=["when score >0  and score <=456 then '(0,456]' when score >456 and score <=477 then '(456,477]' when score >477 and score <=498 then '(477,498]' when score >498 and score <=514 then '(498,514]' when score >514 and score <=529 then '(514,529]' when score >529 and score <=544 then '(529,544]' when score >544 and score <=561 then '(544,561]' when score >561 and score <=579 then '(561,579]' when score >579 and score <=601 then '(579,601]' when score >601  then '(601,-)'" 
#     ,"when score >0  and score <=0.0031 then  '(0,0.0031]' when score >0.0031 and score <=0.0061 then '(0.0031,0.0061]' when score >0.0061 and score <=0.0091 then '(0.0061,0.0091]' when score >0.0091 and score <=0.0121 then '(0.0091,0.0121]' when score >0.0121 and score <=0.0151 then '(0.0121,0.0151]' when score >0.0151 and score <=0.0181 then '(0.0151,0.0181]' when score >0.0181 and score <=0.0211 then '(0.0181,0.0211]' when score >0.0211 and score <=0.0241 then '(0.0211,0.0241]' when score >0.0241 and score <=0.0271 then '(0.0241,0.0271]' when score >0.0271 then '(0.0271,-]'"
#     ,"when score >0  and score <=555 then  '(0,555]' when score >555 and score <=569 then '(555,569]' when score >569 and score <=578 then '(569,578]' when score >578 and score <=582 then '(578,582]' when score >582 and score <=587 then '(582,587]' when score >587 and score <=595 then '(587,595]' when score >595 and score <=605 then '(595,605]' when score >605 and score <=619 then '(605,619]' when score >619 and score <=629 then '(619,629]' when score >629  then '(629,-)'"
#     ,"when score >0  and score <=470 then  '(0,470]' when score >470 and score <=484 then '(470,484]' when score >484 and score <=495 then '(484,495]' when score >495 and score <=503 then '(495,503]' when score >503 and score <=528 then '(503,528]' when score >528 and score <=539 then '(528,539]' when score >539 and score <=546 then '(539,546]' when score >546 and score <=612 then '(546,612]' when score >612 and score <=632 then '(612,632]' when score >632  then '(632,-)'"
#     ,"when score >0  and score <=425 then  '(0,425]' when score >425 and score <=441 then '(425,441]' when score >441 and score <=449 then '(441,449]' when score >449 and score <=461 then '(449,461]' when score >461 and score <=474 then '(461,474]' when score >474 and score <=490 then '(474,490]' when score >490 and score <=511 then '(490,511]' when score >511 and score <=532 then '(511,532]' when score >532 and score <=557 then '(532,557]' when score >557  then '(557,-)'"
#     ,"when score >0  and score <=672 then  '(0,672]' when score >672 and score <=685 then '(672,685]' when score >685 and score <=695 then '(685,695]' when score >695 and score <=704 then '(695,704]' when score >704 and score <=713 then '(704,713]' when score >713 and score <=723 then '(713,723]' when score >723 and score <=735 then '(723,735]' when score >735 and score <=750 then '(735,750]' when score >750 and score <=771 then '(750,771]' when score >771  then '(771,-)'"
#     ,"when score >0  and score <=462 then  '(0,462]' when score >462 and score <=469 then '(462,469]' when score >469 and score <=475 then '(469,475]' when score >475 and score <=480 then '(475,480]' when score >480 and score <=485 then '(480,485]' when score >485 and score <=489 then '(485,489]' when score >489 and score <=496 then '(489,496]' when score >496 and score <=503 then '(496,503]' when score >503 and score <=514 then '(503,514]' when score >514  then '(514,-)'"
#     ,"when score >0  and score <=379 then  '(0,379]' when score >379 and score <=390 then '(379,390]' when score >390 and score <=398 then '(390,398]' when score >398 and score <=409 then '(398,409]' when score >409 and score <=415 then '(409,415]' when score >415 and score <=425 then '(415,425]' when score >425 and score <=437 then '(425,437]' when score >437 and score <=451 then '(437,451]' when score >451 and score <=472 then '(451,472]' when score >472  then '(472,-)'"
#     ,"when score >0  and score <=511 then  '(0,511]' when score >511 and score <=520 then '(511,520]' when score >520 and score <=525 then '(520,525]' when score >525 and score <=535 then '(525,435]' when score >535 and score <=541 then '(535,541]' when score >541 and score <=551 then '(541,551]' when score >551 and score <=558 then '(551,558]' when score >558 and score <=568 then '(558,568]' when score >568 and score <=578 then '(568,578]' when score >578  then '(578,-)'"
#     ,"when score >0  and score <=539 then  '(0,539]' when score >539 and score <=563 then '(539,563]' when score >563 and score <=587 then '(563,587]' when score >587 and score <=611 then '(587,611]' when score >611 and score <=635 then '(611,635]' when score >635 and score <=659 then '(635,659]' when score >659 and score <=683 then '(659,683]' when score >683 and score <=707 then '(683,707]' when score >707 and score <=731 then '(707,731]' when score >731  then '(731,-)'"
#     ,"when score >0  and score <=412 then  '(0,412]' when score >412 and score <=460 then '(412,460]' when score >460 and score <=508 then '(460,508]' when score >508 and score <=556 then '(508,556]' when score >556 and score <=604 then '(556,604]' when score >604 and score <=652 then '(604,652]' when score >652 and score <=700 then '(652,700]' when score >700 and score <=748 then '(700,748]' when score >748 and score <=796 then '(748,796]' when score >796  then '(796,-)'"
#     ,"when score >0  and score <=413 then  '(0,413]' when score >413 and score <=463 then '(413,463]' when score >463 and score <=513 then '(463,513]' when score >513 and score <=562 then '(513,562]' when score >562 and score <=612 then '(562,612]' when score >612 and score <=662 then '(612,662]' when score >662 and score <=712 then '(662,712]' when score >712 and score <=761 then '(712,761]' when score >761 and score <=796 then '(761,796]' when score >796  then '(796,-)'"
#     ,"when score >0  and score <=584 then  '(0,584]' when score >584 and score <=592 then '(584,592]' when score >592 and score <=596 then '(592,596]' when score >596 and score <=600 then '(596,600]' when score >600 and score <=604 then '(600,604]' when score >604 and score <=608 then '(604,608]' when score >608 and score <=613 then '(608,613]' when score >613 and score <=618 then '(613,618]' when score >618 and score <=624 then '(618,624]' when score >624  then '(624,-)'"
#     ,"when score >0  and score <=430 then  '(0,430]' when score >430 and score <=442 then '(430,442]' when score >442 and score <=454 then '(442,454]' when score >454 and score <=461 then '(454,461]' when score >461 and score <=466 then '(461,466]' when score >466 and score <=476 then '(466,476]' when score >476 and score <=498 then '(476,498]' when score >498 and score <=521 then '(498,521]' when score >521 and score <=536 then '(521,536]' when score >536  then '(536,-)'"
#     ,"when score >0  and score <=153 then  '(0,153]' when score >153 and score <=170 then '(153,170]' when score >170 and score <=184 then '(170,184]' when score >184 and score <=197 then '(184,197]' when score >197 and score <=209 then '(197,209]' when score >209 and score <=220 then '(209,220]' when score >220 and score <=234 then '(220,234]' when score >234 and score <=249 then '(234,249]' when score >249 and score <=269 then '(249,269]' when score >269  then '(269,-)'"
#     ,"when score >0  and score <=545 then  '(0,545]' when score >545 and score <=579 then '(545,579]' when score >579 and score <=603 then '(579,603]' when score >603 and score <=629 then '(603,629]' when score >629 and score <=647 then '(629,647]' when score >647 and score <=673 then '(647,673]' when score >673 and score <=693 then '(673,693]' when score >693 and score <=723 then '(693,723]' when score >723 and score <=751 then '(723,751]' when score >751  then '(751,-)'"
#     ,"when score >0  and score <=583 then  '(0,583]' when score >583 and score <=590 then '(583,590]' when score >590 and score <=595 then '(590,595]' when score >595 and score <=600 then '(595,600]' when score >600 and score <=604 then '(600,604]' when score >604 and score <=608 then '(604,608]' when score >608 and score <=612 then '(608,612]' when score >612 and score <=617 then '(612,617]' when score >617 and score <=626 then '(617,626]' when score >626  then '(626,-)'"
#     ,"when score >0  and score <=581 then  '(0,581]' when score >581 and score <=589 then '(581,589]' when score >589 and score <=595 then '(589,595]' when score >595 and score <=600 then '(595,600]' when score >600 and score <=604 then '(600,604]' when score >604 and score <=609 then '(604,609]' when score >609 and score <=614 then '(609,614]' when score >614 and score <=619 then '(614,619]' when score >619 and score <=627 then '(619,627]' when score >627  then '(627,-)'"
#                    ]
    
    ##借呗
    score_1010=['creditScore46','creditScore48','creditScore49','creditScore50','creditScore51','creditScore54','creditScore55','creditScore56',
                'creditScore57','creditScore58','creditScore59','creditScore61','creditScore62','creditScore63']
    ##京东
    score_1006=['creditScore52']
#     time_controls=["'"'2020-06-19'"'","'"'2021-05-03'"'","'"'2020-10-20'"'","'"'2020-10-20'"'","'"'2020-10-20'"'","'"'2020-10-18'"'","'"'2020-12-15'"'","'"'2020-10-27'"'","'"'2021-01-20'"'","'"'2021-01-30'"'","'"'2021-02-21'"'","'"'2020-02-21'"'","'"'2021-03-11'"'"
#                    ,"'"'2021-03-01'"'","'"'2021-04-03'"'","'"'2021-04-08'"'"]
#     inputdate_control1s=["'"'20200119'"'","'"'20201203'"'","'"'20200520'"'","'"'20200520'"'","'"'20200520'"'","'"'20200518'"'","'"'20200715'"'","'"'20200820'"'","'"'20200820'"'","'"'20200831'"'","'"'20200921'"'","'"'20200921'"'","'"'20201011'"'","'"'20201011'"'"
#                          ,"'"'20201203'"'","'"'20201208'"'"]
#     inputdate_control2s=["'"'20200219'"'","'"'20210103'"'","'"'20200620'"'","'"'20200620'"'","'"'20200620'"'","'"'20200618'"'","'"'20200815'"'","'"'20200920'"'","'"'20200920'"'","'"'20200930'"'","'"'20201021'"'","'"'20201021'"'","'"'20201111'"'","'"'20201111'"'"
#                          ,"'"'20210103'"'","'"'20210108'"'"]
    tab = Tab()
    for i,creditscore in enumerate(creditscores):
        score_cut=score_cuts[i]
        chinese_name=chinese_names[i]
        time_control=time_controls[i]
        inputdate_control1=inputdate_control1s[i]
        inputdate_control2=inputdate_control2s[i]
        if creditscore in score_1010:
            businesstype="'"'1010'"'"
        elif creditscore in score_1006:
            businesstype="'"'1006'"'"
        else:
            businesstype="'"'2006'"'"
        # 定义  显示运行时间 的小功能
        def runtime(func):
            @functools.wraps(func)
            def inner(sql_Strip):
                t1 = time.time()
                data = func(sql_Strip)
                t2 = time.time()
                run_time = t2 - t1
                print(creditscore+'运行时间为:',run_time)
                return data
            return inner
        # 输出SQL运行结果
        @runtime # 能展示时间
        def run_sql(sql_Strip):
            cursor.execute(sql_Strip) #运行SQL
            data = as_pandas(cursor)  ##  输出跑SQL结果
            return data

        ###sql
        sql_badrate='''
        SELECT 
        case 
        {score_cut}
        end as score ,
        count( DISTINCT case when b.th>={bad} THEN b.customerid  END ) AS '坏样本',
        count( DISTINCT case when b.th>=0 THEN b.customerid  END ) AS '总数'
        FROM 
        (select customerid,serialno
        from fk.v_g_aprvadt_appl_bsc_inf
        where data_dt={time_control}
        and inputdate between {inputdate_control1} and {inputdate_control2}
        and decisionstage='Apply'--申请
        and businesstype={businesstype}
        )a
        JOIN
        (SELECT customerid, max(bd_hist_high_odue_days) AS th
        FROM fk.v_g_loanlend_loan_inf_his
        WHERE data_dt={time_control}
        AND {time_control}> date(bd_fst_maturity_dt)+{day}
        GROUP BY 1
        ) b
        ON a.customerid=b.customerid
        right JOIN
        (SELECT objectno,{creditscore} AS score
        FROM fk.v_r_risk_fico_xml_format
        WHERE data_dt<={time_control}
        AND {creditscore}>0
        ) c
        ON a.serialno=c.objectno
        GROUP BY score
        ORDER BY score
        '''
        sql_badrate1='''
        SELECT 
        case 
        {score_cut}
        end as score ,
        count( DISTINCT case when b.th>={bad} THEN b.customerid  END ) AS '坏样本',
        count( DISTINCT case when b.th>=0 THEN b.customerid  END ) AS '总数'
        FROM 
        (select customerid,serialno
        from fk.v_g_aprvadt_appl_bsc_inf
        where data_dt={time1}
        and inputdate between {inputdate1_1} and {inputdate1_2}
        and decisionstage='Apply'--申请
        and businesstype={businesstype}
        )a
        JOIN
        (SELECT customerid, max(bd_hist_high_odue_days) AS th
        FROM fk.v_g_loanlend_loan_inf_his
        WHERE data_dt={time1}
        AND {time1}> date(bd_fst_maturity_dt)+{day}
        GROUP BY 1
        ) b
        ON a.customerid=b.customerid
        right JOIN
        (SELECT objectno,{creditscore} AS score
        FROM fk.v_r_risk_fico_xml_format
        WHERE data_dt<={time1}
        AND {creditscore}>0
        ) c
        ON a.serialno=c.objectno
        GROUP BY score
        ORDER BY score
        '''
        sql_badrate2='''
        SELECT 
        case 
        {score_cut}
        end as score ,
        count( DISTINCT case when b.th>={bad} THEN b.customerid  END ) AS '坏样本',
        count( DISTINCT case when b.th>=0 THEN b.customerid  END ) AS '总数'
        FROM 
        (select customerid,serialno
        from fk.v_g_aprvadt_appl_bsc_inf
        where data_dt={time2}
        and inputdate between {inputdate2_1} and {inputdate2_2}
        and decisionstage='Apply'--申请
        and businesstype={businesstype} 
        )a
        JOIN
        (SELECT customerid, max(bd_hist_high_odue_days) AS th
        FROM fk.v_g_loanlend_loan_inf_his
        WHERE data_dt={time2}
        AND {time2}> date(bd_fst_maturity_dt)+{day}
        GROUP BY 1
        ) b
        ON a.customerid=b.customerid
        right JOIN
        (SELECT objectno,{creditscore} AS score
        FROM fk.v_r_risk_fico_xml_format
        WHERE data_dt<={time2}
        AND {creditscore}>0
        ) c
        ON a.serialno=c.objectno
        GROUP BY score
        ORDER BY score
        '''
        sql_badrate3='''
        SELECT 
        case 
        {score_cut}
        end as score ,
        count( DISTINCT case when b.th>={bad} THEN b.customerid  END ) AS '坏样本',
        count( DISTINCT case when b.th>=0 THEN b.customerid  END ) AS '总数'
        FROM 
        (select customerid,serialno
        from fk.v_g_aprvadt_appl_bsc_inf
        where data_dt={time3}
        and inputdate between {inputdate3_1} and {inputdate3_2}
        and decisionstage='Apply'--申请
        and businesstype={businesstype}
        )a
        JOIN
        (SELECT customerid, max(bd_hist_high_odue_days) AS th
        FROM fk.v_g_loanlend_loan_inf_his
        WHERE data_dt={time3}
        AND {time3}> date(bd_fst_maturity_dt)+{day}
        GROUP BY 1
        ) b
        ON a.customerid=b.customerid
        right JOIN
        (SELECT objectno,{creditscore} AS score
        FROM fk.v_r_risk_fico_xml_format
        WHERE data_dt<={time3}
        AND {creditscore}>0
        ) c
        ON a.serialno=c.objectno
        GROUP BY score
        ORDER BY score
        '''
        if creditscore!='creditScore55':
            sql_badrate_end= sql_badrate.format(
            score_cut=score_cut,
            time_control=time_control,
            inputdate_control1=inputdate_control1,
            inputdate_control2=inputdate_control2 ,
            creditscore=creditscore,
            businesstype=businesstype,
            bad='30',
            day='90'  
            )
            badrate_control=run_sql(sql_badrate_end)
            print(badrate_control)
            if badrate_control['总数'].sum()<=20:
                print(creditscore+'控制组没有数据')
            else:
                if len(badrate_control)<10:
                    print(creditscore+'数据有问题')
                else:
                    sql_badrate_end1 = sql_badrate1.format(
                    score_cut=score_cut,
                    time1=time1,
                    inputdate1_1=inputdate1_1,
                    inputdate1_2=inputdate1_2 ,
                    creditscore=creditscore,
                    businesstype=businesstype,
                    bad='30',
                    day='90'  
                    )
                    badrate1=run_sql(sql_badrate_end1)
                    print(badrate1)
                    if badrate1['总数'].sum()<=20:
                        print(creditscore+'第一组没有数据')
                    else:
                        if len(badrate1)<10:
                            print(creditscore+'数据有问题')
                        else:
                            sql_badrate_end2 = sql_badrate2.format(
                            score_cut=score_cut,
                            time2=time2,
                            inputdate2_1=inputdate2_1,
                            inputdate2_2=inputdate2_2 ,
                            creditscore=creditscore,
                            businesstype=businesstype,
                            bad='30',
                            day='90'
                            )
                            badrate2=run_sql(sql_badrate_end2)
                            print(badrate2)
                            if badrate2['总数'].sum()<=0:
                                print(creditscore+'第二组没有数据')
                            else:
                                if len(badrate2)<10:
                                    print(creditscore+'数据有问题')
                                else:
                                    sql_badrate_end3 = sql_badrate3.format(
                                    score_cut=score_cut,
                                    time3=time3,
                                    inputdate3_1=inputdate3_1,
                                    inputdate3_2=inputdate3_2 ,
                                    creditscore=creditscore,
                                    businesstype=businesstype,
                                    bad='30',
                                    day='90'
                                    )
                                    badrate3=run_sql(sql_badrate_end3)
                                    print(badrate3)
                                    if badrate3['总数'].sum()<=0:
                                        print(creditscore+'第三组没有数据')
                                    else:
                                        if len(badrate3)<10:
                                            print(creditscore+'数据有问题')
                                        else:
                                            ##计算Badrate
                                            badrate_control['badrate']=round(badrate_control['坏样本']/badrate_control['总数'],4)
                                            where_are_inf_control = np.isnan(badrate_control['badrate'])  ##查找是inf的PSI
                                            badrate_control['badrate'][where_are_inf_control] = 0.0000
                                            
                                            badrate1['badrate']=round(badrate1['坏样本']/badrate1['总数'],4)
                                            where_are_inf1 = np.isnan(badrate1['badrate'])  ##查找是inf的PSI
                                            badrate1['badrate'][where_are_inf1] = 0.0000

                                            badrate2['badrate']=round(badrate2['坏样本']/badrate2['总数'],4)
                                            where_are_inf2 = np.isnan(badrate2['badrate'])  ##查找是inf的PSI
                                            badrate2['badrate'][where_are_inf2] = 0.0000

                                            badrate3['badrate']=round(badrate3['坏样本']/badrate3['总数'],4)
                                            where_are_inf3 = np.isnan(badrate3['badrate'])  ##查找是inf的PSI
                                            badrate3['badrate'][where_are_inf3] = 0.0000
                                            
                                            Badrate_control=badrate_control['badrate'].tolist()
                                            Badrate1=badrate1['badrate'].tolist()
                                            Badrate2=badrate2['badrate'].tolist()
                                            Badrate3=badrate3['badrate'].tolist()

                                            ##最小二乘法
                                            #####采样点(Xi,Yi)###
                                            Badrate_Xi=np.array([2,3,4,5,6,7,8,9,10])
                                            Badrate_control_Yi=np.array(Badrate_control[1:])
                                            Badrate1_Yi=np.array(Badrate1[1:])
                                            Badrate2_Yi=np.array(Badrate2[1:])
                                            Badrate3_Yi=np.array(Badrate3[1:])
                                            ###需要拟合的函数func及误差error###
                                            def func(p,x):
                                                k,b=p
                                                return k*x+b
                                            def error(p,x,y):
                                                return func(p,x)-y #x、y都是列表，故返回值也是个列表
                                            #TEST
                                            Badrate_control_p0=[1,Badrate_control[0]]
                                            Badrate1_p0=[1,Badrate1[0]]
                                            Badrate2_p0=[1,Badrate2[0]]
                                            Badrate3_p0=[1,Badrate3[0]]
                                            ###主函数从此开始###

                                            Para_control=leastsq(error,Badrate_control_p0,args=(Badrate_Xi,Badrate_control_Yi)) #把error函数中除了p以外的参数打包到args中
                                            k0,b0=Para_control[0]
                                            Para1=leastsq(error,Badrate1_p0,args=(Badrate_Xi,Badrate1_Yi)) #把error函数中除了p以外的参数打包到args中
                                            k1,b1=Para1[0]
                                            Para2=leastsq(error,Badrate2_p0,args=(Badrate_Xi,Badrate2_Yi)) #把error函数中除了p以外的参数打包到args中
                                            k2,b2=Para2[0]
                                            Para3=leastsq(error,Badrate3_p0,args=(Badrate_Xi,Badrate3_Yi)) #把error函数中除了p以外的参数打包到args中
                                            k3,b3=Para3[0]
                                            print("k0=",k0,'\n',"b0=",b0)
                                            print("k1=",k1,'\n',"b1=",b1) 
                                            print("k2=",k2,'\n',"b2=",b2)
                                            print("k3=",k3,'\n',"b3=",b3)
                                            I=list(range(1,11,1))
                                            y0=[]
                                            for i in I:
                                                y0.append(k0*i+b0)
                                            y1=[]
                                            for i in I:
                                                y1.append(k1*i+b1)
                                            y2=[]
                                            for i in I:
                                                y2.append(k2*i+b2)
                                            y3=[]
                                            for i in I:
                                                y3.append(k3*i+b3)
                                            k0=str(round(k0,4))   
                                            k1=str(round(k1,4))
                                            k2=str(round(k2,4))
                                            k3=str(round(k3,4))
                    
                                            def line() -> Line:
                                                c = (
                                                    Line(init_opts=opts.InitOpts(width="1200px"))
                                                    .add_xaxis(xaxis_data=list(badrate_control['score']))
                                                    .add_yaxis(
                                                        series_name="Badrate0",
                                                        y_axis=Badrate_control,
                                                        color="red",
                                                        label_opts=opts.LabelOpts(is_show=False),
                                                        is_smooth=True
                                                        )
                                                    .add_yaxis(
                                                        series_name="Badrate1",
                                                        y_axis=Badrate1,
                                                        color="#9932CC",
                                                        label_opts=opts.LabelOpts(is_show=False),
                                                        is_smooth=True
                                                        )
                                                    .add_yaxis(
                                                        series_name="Badrate2",
                                                        y_axis=Badrate2,
                                                        color="green",
                                                        label_opts=opts.LabelOpts(is_show=False),
                                                        is_smooth=True)
                                                    .add_yaxis(
                                                        series_name="Badrate3",
                                                        y_axis=Badrate3,
                                                        color="blue",
                                                        label_opts=opts.LabelOpts(is_show=False),
                                                        is_smooth=True
                                                        )
                                                    .add_yaxis(
                                                        series_name="Badrate0趋势"+k0,
                                                        y_axis=y0,
                                                        #color="red",
                                                        symbol="triangle",
                                                        linestyle_opts=opts.LineStyleOpts(color="red", width=2, type_="dashed"),
                                                        label_opts=opts.LabelOpts(is_show=False)##不主动显示数值
                                                        )
                                                    .add_yaxis(
                                                        series_name="Badrate1趋势"+k1,
                                                        y_axis=y1,
                                                        symbol="triangle",
                                                        #color="#9932CC",
                                                        linestyle_opts=opts.LineStyleOpts(color="#9932CC", width=2, type_="dashed"),
                                                        label_opts=opts.LabelOpts(is_show=False)##不主动显示数值
                                                        )
                                                    .add_yaxis(
                                                        series_name="Badrate2趋势"+k2,
                                                        y_axis=y2,
                                                        #color="green",
                                                        symbol="triangle",
                                                        linestyle_opts=opts.LineStyleOpts(color="green", width=2, type_="dashed"),
                                                        label_opts=opts.LabelOpts(is_show=False)##不主动显示数值
                                                        )
                                                    .add_yaxis(
                                                        series_name="Badrate3趋势"+k3,
                                                        y_axis=y3,
                                                        #color="blue",
                                                        label_opts=opts.LabelOpts(is_show=False),##不主动显示数值
                                                        symbol="triangle",
                                                        linestyle_opts=opts.LineStyleOpts(color="blue", width=2, type_="dashed"),
                                                        )
                                                    .set_global_opts(title_opts=opts.TitleOpts(title="效力图",
                                                                                               subtitle=chinese_name+":"+" 控制组授信:"+inputdate_control1[1:10]+"-"+inputdate_control2[1:10]+
                                                                                              " 1组授信:"+inputdate1_1[1:10]+"-"+inputdate1_2[1:10]+
                                                                                              " 2组授信:"+inputdate2_1[1:10]+"-"+inputdate2_2[1:10]+
                                                                                              " 3组授信:"+inputdate3_1[1:10]+"-"+inputdate3_2[1:10])))
                                                return c
                                            tab.add(line(), creditscore)
        else:
            sql_badrate_end= sql_badrate.format(
            score_cut=score_cut,
            time_control=time_control,
            inputdate_control1=inputdate_control1,
            inputdate_control2=inputdate_control2 ,
            creditscore=creditscore,
            businesstype=businesstype,
            bad='1',
            day='7'  
            )
            badrate_control=run_sql(sql_badrate_end)
            print(badrate_control)
            if badrate_control['总数'].sum()<=20:
                print(creditscore+'控制组没有数据')
            else:
                if len(badrate_control)<10:
                    print(creditscore+'数据有问题')
                else:
                    sql_badrate_end1 = sql_badrate1.format(
                    score_cut=score_cut,
                    time1=time55_1,
                    inputdate1_1=inputdate1_1,
                    inputdate1_2=inputdate1_2 ,
                    creditscore=creditscore,
                    businesstype=businesstype,
                    bad='1',
                    day='7'  
                    )
                    badrate1=run_sql(sql_badrate_end1)
                    print(badrate1)
                    if badrate1['总数'].sum()<=20:
                        print(creditscore+'第一组没有数据')
                    else:
                        if len(badrate1)<10:
                            print(creditscore+'数据有问题')
                        else:
                            sql_badrate_end2 = sql_badrate2.format(
                            score_cut=score_cut,
                            time2=time55_2,
                            inputdate2_1=inputdate2_1,
                            inputdate2_2=inputdate2_2 ,
                            creditscore=creditscore,
                            businesstype=businesstype,
                            bad='1',
                            day='7'
                            )
                            badrate2=run_sql(sql_badrate_end2)
                            print(badrate2)
                            if badrate2['总数'].sum()<=0:
                                print(creditscore+'第二组没有数据')
                            else:
                                if len(badrate2)<10:
                                    print(creditscore+'数据有问题')
                                else:
                                    sql_badrate_end3 = sql_badrate3.format(
                                    score_cut=score_cut,
                                    time3=time55_3,
                                    inputdate3_1=inputdate3_1,
                                    inputdate3_2=inputdate3_2 ,
                                    creditscore=creditscore,
                                    businesstype=businesstype,
                                    bad='1',
                                    day='7'
                                    )
                                    badrate3=run_sql(sql_badrate_end3)
                                    print(badrate3)
                                    if badrate3['总数'].sum()<=0:
                                        print(creditscore+'第三组没有数据')
                                    else:
                                        if len(badrate3)<10:
                                            print(creditscore+'数据有问题')
                                        else:
                                    ##计算Badrate
                                            badrate_control['badrate']=round(badrate_control['坏样本']/badrate_control['总数'],4)
                                            where_are_inf_control = np.isnan(badrate_control['badrate'])  ##查找是inf的PSI
                                            badrate_control['badrate'][where_are_inf_control] = 0.0000
                                            
                                            badrate1['badrate']=round(badrate1['坏样本']/badrate1['总数'],4)
                                            where_are_inf1 = np.isnan(badrate1['badrate'])  ##查找是inf的PSI
                                            badrate1['badrate'][where_are_inf1] = 0.0000

                                            badrate2['badrate']=round(badrate2['坏样本']/badrate2['总数'],4)
                                            where_are_inf2 = np.isnan(badrate2['badrate'])  ##查找是inf的PSI
                                            badrate2['badrate'][where_are_inf2] = 0.0000

                                            badrate3['badrate']=round(badrate3['坏样本']/badrate3['总数'],4)
                                            where_are_inf3 = np.isnan(badrate3['badrate'])  ##查找是inf的PSI
                                            badrate3['badrate'][where_are_inf3] = 0.0000
                                            
                                            Badrate_control=badrate_control['badrate'].tolist()
                                            Badrate1=badrate1['badrate'].tolist()
                                            Badrate2=badrate2['badrate'].tolist()
                                            Badrate3=badrate3['badrate'].tolist()

                                            ##最小二乘法
                                            #####采样点(Xi,Yi)###
                                            Badrate_Xi=np.array([2,3,4,5,6,7,8,9,10])
                                            Badrate_control_Yi=np.array(Badrate_control[1:])
                                            Badrate1_Yi=np.array(Badrate1[1:])
                                            Badrate2_Yi=np.array(Badrate2[1:])
                                            Badrate3_Yi=np.array(Badrate3[1:])
                                            ###需要拟合的函数func及误差error###
                                            def func(p,x):
                                                k,b=p
                                                return k*x+b
                                            def error(p,x,y):
                                                return func(p,x)-y #x、y都是列表，故返回值也是个列表
                                            #TEST
                                            Badrate_control_p0=[1,Badrate_control[0]]
                                            Badrate1_p0=[1,Badrate1[0]]
                                            Badrate2_p0=[1,Badrate2[0]]
                                            Badrate3_p0=[1,Badrate3[0]]
                                            ###主函数从此开始###

                                            Para_control=leastsq(error,Badrate_control_p0,args=(Badrate_Xi,Badrate_control_Yi)) #把error函数中除了p以外的参数打包到args中
                                            k0,b0=Para_control[0]
                                            Para1=leastsq(error,Badrate1_p0,args=(Badrate_Xi,Badrate1_Yi)) #把error函数中除了p以外的参数打包到args中
                                            k1,b1=Para1[0]
                                            Para2=leastsq(error,Badrate2_p0,args=(Badrate_Xi,Badrate2_Yi)) #把error函数中除了p以外的参数打包到args中
                                            k2,b2=Para2[0]
                                            Para3=leastsq(error,Badrate3_p0,args=(Badrate_Xi,Badrate3_Yi)) #把error函数中除了p以外的参数打包到args中
                                            k3,b3=Para3[0]
                                            print("k0=",k0,'\n',"b0=",b0)
                                            print("k1=",k1,'\n',"b1=",b1) 
                                            print("k2=",k2,'\n',"b2=",b2)
                                            print("k3=",k3,'\n',"b3=",b3)
                                            I=list(range(1,11,1))
                                            y0=[]
                                            for i in I:
                                                y0.append(k0*i+b0)
                                            y1=[]
                                            for i in I:
                                                y1.append(k1*i+b1)
                                            y2=[]
                                            for i in I:
                                                y2.append(k2*i+b2)
                                            y3=[]
                                            for i in I:
                                                y3.append(k3*i+b3)
                                            k0=str(round(k0,4))   
                                            k1=str(round(k1,4))
                                            k2=str(round(k2,4))
                                            k3=str(round(k3,4))
                    
                                            def line() -> Line:
                                                c = (
                                                    Line(init_opts=opts.InitOpts(width="1200px"))
                                                    .add_xaxis(xaxis_data=list(badrate_control['score']))
                                                    .add_yaxis(
                                                        series_name="Badrate0",
                                                        y_axis=Badrate_control,
                                                        color="red",
                                                        label_opts=opts.LabelOpts(is_show=False),
                                                        is_smooth=True
                                                        )
                                                    .add_yaxis(
                                                        series_name="Badrate1",
                                                        y_axis=Badrate1,
                                                        color="#9932CC",
                                                        label_opts=opts.LabelOpts(is_show=False),
                                                        is_smooth=True
                                                        )
                                                    .add_yaxis(
                                                        series_name="Badrate2",
                                                        y_axis=Badrate2,
                                                        color="green",
                                                        label_opts=opts.LabelOpts(is_show=False),
                                                        is_smooth=True)
                                                    .add_yaxis(
                                                        series_name="Badrate3",
                                                        y_axis=Badrate3,
                                                        color="blue",
                                                        label_opts=opts.LabelOpts(is_show=False),
                                                        is_smooth=True
                                                        )
                                                    .add_yaxis(
                                                        series_name="Badrate0趋势"+k0,
                                                        y_axis=y0,
                                                        #color="red",
                                                        symbol="triangle",
                                                        linestyle_opts=opts.LineStyleOpts(color="red",width=2, type_="dashed"),
                                                        label_opts=opts.LabelOpts(is_show=False)##不主动显示数值
                                                        )
                                                    .add_yaxis(
                                                        series_name="Badrate1趋势"+k1,
                                                        y_axis=y1,
                                                        #color="#9932CC",
                                                        symbol="triangle",
                                                        linestyle_opts=opts.LineStyleOpts(color="#9932CC",width=2, type_="dashed"),
                                                        label_opts=opts.LabelOpts(is_show=False)##不主动显示数值
                                                        )
                                                    .add_yaxis(
                                                        series_name="Badrate2趋势"+k2,
                                                        y_axis=y2,
                                                        #color="green",
                                                        symbol="triangle",
                                                        linestyle_opts=opts.LineStyleOpts(color="green",width=2, type_="dashed"),
                                                        label_opts=opts.LabelOpts(is_show=False)##不主动显示数值
                                                        )
                                                    .add_yaxis(
                                                        series_name="Badrate3趋势"+k3,
                                                        y_axis=y3,
                                                        #color="blue",
                                                        label_opts=opts.LabelOpts(is_show=False),##不主动显示数值
                                                        symbol="triangle",
                                                        linestyle_opts=opts.LineStyleOpts(color="blue",width=2, type_="dashed"),
                                                        )
                                                    .set_global_opts(title_opts=opts.TitleOpts(title="效力图",
                                                                                               subtitle=chinese_name+":"+" 控制组授信:"+inputdate_control1[1:10]+"-"+inputdate_control2[1:10]+
                                                                                              " 1组授信:"+inputdate1_1[1:10]+"-"+inputdate1_2[1:10]+
                                                                                              " 2组授信:"+inputdate2_1[1:10]+"-"+inputdate2_2[1:10]+
                                                                                              " 3组授信:"+inputdate3_1[1:10]+"-"+inputdate3_2[1:10])))
                                                return c
                                            tab.add(line(), creditscore)

    return tab.render("模型效力展示（添加趋势线）.html")

## 运行部分

In [14]:
effect(user='fk_yanjintong'##用户
       ,password = 'Yjt@0828$$'
       ,inputdate1_1="'"'20201001'"'"
       ,inputdate1_2="'"'20201101'"'"
       ,inputdate2_1="'"'20201101'"'"
       ,inputdate2_2="'"'20201201'"'"
       ,inputdate3_1="'"'20201201'"'"
       ,inputdate3_2="'"'20210101'"'"
       ,time1="'"'2021-03-01'"'"
       ,time2="'"'2021-04-01'"'"
       ,time3="'"'2021-05-01'"'"
       ,time55_1="'"'2020-12-08'"'"
       ,time55_2="'"'2021-01-08'"'"
       ,time55_3="'"'2021-02-08'"'"
      )

creditScore55运行时间为: 229.56065320968628
       score  坏样本    总数
0    (0,456]  432  7357
1  (456,477]   46  1650
2  (477,498]   25   928
3  (498,514]   12   531
4  (514,529]   11   363
5  (529,544]    1   116
6  (544,561]    1    90
7  (561,579]    1    22
8  (579,601]    0     8
9    (601,-)    0     1
creditScore55运行时间为: 289.852411031723
       score  坏样本     总数
0    (0,456]  740  13053
1  (456,477]  112   3495
2  (477,498]   56   2040
3  (498,514]   28   1301
4  (514,529]   15    974
5  (529,544]    5    306
6  (544,561]    2    183
7  (561,579]    0     47
8  (579,601]    0     14
9    (601,-)    0      0
creditScore55运行时间为: 247.62899613380432
       score  坏样本    总数
0    (0,456]  556  8010
1  (456,477]   98  2120
2  (477,498]   39  1198
3  (498,514]   19   721
4  (514,529]   14   495
5  (529,544]    3   194
6  (544,561]    2    96
7  (561,579]    0    21
8  (579,601]    0     9
9    (601,-)    0     2
creditScore55运行时间为: 234.37947130203247
       score  坏样本    总数
0    (0,456]  547  

D:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:544: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:548: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:552: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:556: SettingWithCopyWarning

'C:\\Users\\jintong.yan\\模型效力展示.html'

# badrate模型效力-不添加趋势线（最终使用）

In [16]:
##所用到的包
import numpy as np
from scipy.optimize import leastsq
from impala.util import as_pandas
from impala.dbapi import connect
import time
import functools
from pyecharts import options as opts
from pyecharts.charts import Line,Tab

##定义函数
def effect(user,password,inputdate1_1,inputdate1_2,inputdate2_1,inputdate2_2,inputdate3_1,inputdate3_2,time1,time2,time3,time55_1,time55_2,time55_3):
    ## 输入数据库账号
    conn = connect(host='10.1.57.8', 
                   port=10000, 
                   auth_mechanism='PLAIN',
                   user=user,
                   password = password,
                   database='fk')
    #连接数据库
    cursor=conn.cursor()
    
    tab = Tab()
    #确定需要监控的模型
    creditscores=['creditScore46','creditScore48','creditScore49','creditScore50','creditScore51',
                  'creditScore52','creditScore54','creditScore55' ,'creditScore56','creditScore57',
                  'creditScore58','creditScore59','creditScore60','creditScore61','creditScore62',
                  'creditScore63'
                  #,'creditScore64','creditScore65'
                 ]

#     #需要监控模型对应的中文名（注意：要一一对应）
    chinese_names= ['借呗阿波罗模型V2',
                    '借呗联合模型V2',
                    '借呗阿波罗模型V3（XGB）',
                    '借呗阿波罗模型V3（LGB）',
                    '借呗阿波罗模型V3（权重版）',
                    '京东A卡',
                    '借呗三方征信模型',
                    '借呗阿波罗入催模型',
                    '借呗负债压力模型',
                    '借呗人行二代模型',
                    '借呗融合模型-全变量'
                    ,'借呗融合模型-去除score56',
                    '京东plusA卡V2',
                    '借呗阿波罗V4',
                    '借呗三方征信V2',
                    '借呗阿波罗模型V1'
#                     ,'京东plus卡A卡V3.1',
#                     '京东plus卡A卡V3.2'
                   ]
    
#     creditscores=['creditScore55']
#     chinese_names= ['借呗阿波罗模型V2']
#     score_cuts=["when score >0  and score <=456 then '(0,456]' when score >456 and score <=477 then '(456,477]' when score >477 and score <=498 then '(477,498]' when score >498 and score <=514 then '(498,514]' when score >514 and score <=529 then '(514,529]' when score >529 and score <=544 then '(529,544]' when score >544 and score <=561 then '(544,561]' when score >561 and score <=579 then '(561,579]' when score >579 and score <=601 then '(579,601]' when score >601  then '(601,-)'" ]
#     time_controls=["'"'2020-10-27'"'"]
#     inputdate_control1s=["'"'20200820'"'"]
#     inputdate_control2s=["'"'20200920'"'"]

    score_cuts=["when score >0  and score <=456 then '(0,456]' when score >456 and score <=477 then '(456,477]' when score >477 and score <=498 then '(477,498]' when score >498 and score <=514 then '(498,514]' when score >514 and score <=529 then '(514,529]' when score >529 and score <=544 then '(529,544]' when score >544 and score <=561 then '(544,561]' when score >561 and score <=579 then '(561,579]' when score >579 and score <=601 then '(579,601]' when score >601  then '(601,-)'" 
    ,"when score >0  and score <=0.0031 then  '(0,0.0031]' when score >0.0031 and score <=0.0061 then '(0.0031,0.0061]' when score >0.0061 and score <=0.0091 then '(0.0061,0.0091]' when score >0.0091 and score <=0.0121 then '(0.0091,0.0121]' when score >0.0121 and score <=0.0151 then '(0.0121,0.0151]' when score >0.0151 and score <=0.0181 then '(0.0151,0.0181]' when score >0.0181 and score <=0.0211 then '(0.0181,0.0211]' when score >0.0211 and score <=0.0241 then '(0.0211,0.0241]' when score >0.0241 and score <=0.0271 then '(0.0241,0.0271]' when score >0.0271 then '(0.0271,-]'"
    ,"when score >0  and score <=555 then  '(0,555]' when score >555 and score <=569 then '(555,569]' when score >569 and score <=578 then '(569,578]' when score >578 and score <=582 then '(578,582]' when score >582 and score <=587 then '(582,587]' when score >587 and score <=595 then '(587,595]' when score >595 and score <=605 then '(595,605]' when score >605 and score <=619 then '(605,619]' when score >619 and score <=629 then '(619,629]' when score >629  then '(629,-)'"
    ,"when score >0  and score <=470 then  '(0,470]' when score >470 and score <=484 then '(470,484]' when score >484 and score <=495 then '(484,495]' when score >495 and score <=503 then '(495,503]' when score >503 and score <=528 then '(503,528]' when score >528 and score <=539 then '(528,539]' when score >539 and score <=546 then '(539,546]' when score >546 and score <=612 then '(546,612]' when score >612 and score <=632 then '(612,632]' when score >632  then '(632,-)'"
    ,"when score >0  and score <=425 then  '(0,425]' when score >425 and score <=441 then '(425,441]' when score >441 and score <=449 then '(441,449]' when score >449 and score <=461 then '(449,461]' when score >461 and score <=474 then '(461,474]' when score >474 and score <=490 then '(474,490]' when score >490 and score <=511 then '(490,511]' when score >511 and score <=532 then '(511,532]' when score >532 and score <=557 then '(532,557]' when score >557  then '(557,-)'"
    ,"when score >0  and score <=672 then  '(0,672]' when score >672 and score <=685 then '(672,685]' when score >685 and score <=695 then '(685,695]' when score >695 and score <=704 then '(695,704]' when score >704 and score <=713 then '(704,713]' when score >713 and score <=723 then '(713,723]' when score >723 and score <=735 then '(723,735]' when score >735 and score <=750 then '(735,750]' when score >750 and score <=771 then '(750,771]' when score >771  then '(771,-)'"
    ,"when score >0  and score <=462 then  '(0,462]' when score >462 and score <=469 then '(462,469]' when score >469 and score <=475 then '(469,475]' when score >475 and score <=480 then '(475,480]' when score >480 and score <=485 then '(480,485]' when score >485 and score <=489 then '(485,489]' when score >489 and score <=496 then '(489,496]' when score >496 and score <=503 then '(496,503]' when score >503 and score <=514 then '(503,514]' when score >514  then '(514,-)'"
    ,"when score >0  and score <=379 then  '(0,379]' when score >379 and score <=390 then '(379,390]' when score >390 and score <=398 then '(390,398]' when score >398 and score <=409 then '(398,409]' when score >409 and score <=415 then '(409,415]' when score >415 and score <=425 then '(415,425]' when score >425 and score <=437 then '(425,437]' when score >437 and score <=451 then '(437,451]' when score >451 and score <=472 then '(451,472]' when score >472  then '(472,-)'"
    ,"when score >0  and score <=511 then  '(0,511]' when score >511 and score <=520 then '(511,520]' when score >520 and score <=525 then '(520,525]' when score >525 and score <=535 then '(525,435]' when score >535 and score <=541 then '(535,541]' when score >541 and score <=551 then '(541,551]' when score >551 and score <=558 then '(551,558]' when score >558 and score <=568 then '(558,568]' when score >568 and score <=578 then '(568,578]' when score >578  then '(578,-)'"
    ,"when score >0  and score <=539 then  '(0,539]' when score >539 and score <=563 then '(539,563]' when score >563 and score <=587 then '(563,587]' when score >587 and score <=611 then '(587,611]' when score >611 and score <=635 then '(611,635]' when score >635 and score <=659 then '(635,659]' when score >659 and score <=683 then '(659,683]' when score >683 and score <=707 then '(683,707]' when score >707 and score <=731 then '(707,731]' when score >731  then '(731,-)'"
    ,"when score >0  and score <=412 then  '(0,412]' when score >412 and score <=460 then '(412,460]' when score >460 and score <=508 then '(460,508]' when score >508 and score <=556 then '(508,556]' when score >556 and score <=604 then '(556,604]' when score >604 and score <=652 then '(604,652]' when score >652 and score <=700 then '(652,700]' when score >700 and score <=748 then '(700,748]' when score >748 and score <=796 then '(748,796]' when score >796  then '(796,-)'"
    ,"when score >0  and score <=413 then  '(0,413]' when score >413 and score <=463 then '(413,463]' when score >463 and score <=513 then '(463,513]' when score >513 and score <=562 then '(513,562]' when score >562 and score <=612 then '(562,612]' when score >612 and score <=662 then '(612,662]' when score >662 and score <=712 then '(662,712]' when score >712 and score <=761 then '(712,761]' when score >761 and score <=796 then '(761,796]' when score >796  then '(796,-)'"
    ,"when score >0  and score <=584 then  '(0,584]' when score >584 and score <=592 then '(584,592]' when score >592 and score <=596 then '(592,596]' when score >596 and score <=600 then '(596,600]' when score >600 and score <=604 then '(600,604]' when score >604 and score <=608 then '(604,608]' when score >608 and score <=613 then '(608,613]' when score >613 and score <=618 then '(613,618]' when score >618 and score <=624 then '(618,624]' when score >624  then '(624,-)'"
    ,"when score >0  and score <=430 then  '(0,430]' when score >430 and score <=442 then '(430,442]' when score >442 and score <=454 then '(442,454]' when score >454 and score <=461 then '(454,461]' when score >461 and score <=466 then '(461,466]' when score >466 and score <=476 then '(466,476]' when score >476 and score <=498 then '(476,498]' when score >498 and score <=521 then '(498,521]' when score >521 and score <=536 then '(521,536]' when score >536  then '(536,-)'"
    ,"when score >0  and score <=153 then  '(0,153]' when score >153 and score <=170 then '(153,170]' when score >170 and score <=184 then '(170,184]' when score >184 and score <=197 then '(184,197]' when score >197 and score <=209 then '(197,209]' when score >209 and score <=220 then '(209,220]' when score >220 and score <=234 then '(220,234]' when score >234 and score <=249 then '(234,249]' when score >249 and score <=269 then '(249,269]' when score >269  then '(269,-)'"
    ,"when score >0  and score <=545 then  '(0,545]' when score >545 and score <=579 then '(545,579]' when score >579 and score <=603 then '(579,603]' when score >603 and score <=629 then '(603,629]' when score >629 and score <=647 then '(629,647]' when score >647 and score <=673 then '(647,673]' when score >673 and score <=693 then '(673,693]' when score >693 and score <=723 then '(693,723]' when score >723 and score <=751 then '(723,751]' when score >751  then '(751,-)'"
#     ,"when score >0  and score <=583 then  '(0,583]' when score >583 and score <=590 then '(583,590]' when score >590 and score <=595 then '(590,595]' when score >595 and score <=600 then '(595,600]' when score >600 and score <=604 then '(600,604]' when score >604 and score <=608 then '(604,608]' when score >608 and score <=612 then '(608,612]' when score >612 and score <=617 then '(612,617]' when score >617 and score <=626 then '(617,626]' when score >626  then '(626,-)'"
#     ,"when score >0  and score <=581 then  '(0,581]' when score >581 and score <=589 then '(581,589]' when score >589 and score <=595 then '(589,595]' when score >595 and score <=600 then '(595,600]' when score >600 and score <=604 then '(600,604]' when score >604 and score <=609 then '(604,609]' when score >609 and score <=614 then '(609,614]' when score >614 and score <=619 then '(614,619]' when score >619 and score <=627 then '(619,627]' when score >627  then '(627,-)'"
                   ]
    
    ##借呗
    score_1010=['creditScore46','creditScore48','creditScore49','creditScore50','creditScore51','creditScore54','creditScore55','creditScore56',
                'creditScore57','creditScore58','creditScore59','creditScore61','creditScore62','creditScore63']
    ##京东
    score_1006=['creditScore52']
    time_controls=["'"'2020-06-19'"'","'"'2021-05-03'"'","'"'2020-10-20'"'","'"'2020-10-20'"'","'"'2020-10-20'"'","'"'2020-10-18'"'","'"'2020-12-15'"'","'"'2020-10-27'"'","'"'2021-01-20'"'","'"'2021-01-30'"'","'"'2021-02-21'"'","'"'2020-02-21'"'","'"'2021-03-11'"'"
                   ,"'"'2021-03-01'"'","'"'2021-04-03'"'","'"'2021-04-08'"'"]
    inputdate_control1s=["'"'20200119'"'","'"'20201203'"'","'"'20200520'"'","'"'20200520'"'","'"'20200520'"'","'"'20200518'"'","'"'20200715'"'","'"'20200820'"'","'"'20200820'"'","'"'20200831'"'","'"'20200921'"'","'"'20200921'"'","'"'20201011'"'","'"'20201011'"'"
                         ,"'"'20201203'"'","'"'20201208'"'"]
    inputdate_control2s=["'"'20200219'"'","'"'20210103'"'","'"'20200620'"'","'"'20200620'"'","'"'20200620'"'","'"'20200618'"'","'"'20200815'"'","'"'20200920'"'","'"'20200920'"'","'"'20200930'"'","'"'20201021'"'","'"'20201021'"'","'"'20201111'"'","'"'20201111'"'"
                         ,"'"'20210103'"'","'"'20210108'"'"]
    tab = Tab()
    for i,creditscore in enumerate(creditscores):
        score_cut=score_cuts[i]
        chinese_name=chinese_names[i]
        time_control=time_controls[i]
        inputdate_control1=inputdate_control1s[i]
        inputdate_control2=inputdate_control2s[i]
        if creditscore in score_1010:
            businesstype="'"'1010'"'"
        elif creditscore in score_1006:
            businesstype="'"'1006'"'"
        else:
            businesstype="'"'2006'"'"
        # 定义  显示运行时间 的小功能
        def runtime(func):
            @functools.wraps(func)
            def inner(sql_Strip):
                t1 = time.time()
                data = func(sql_Strip)
                t2 = time.time()
                run_time = t2 - t1
                print(creditscore+'运行时间为:',run_time)
                return data
            return inner
        # 输出SQL运行结果
        @runtime # 能展示时间
        def run_sql(sql_Strip):
            cursor.execute(sql_Strip) #运行SQL
            data = as_pandas(cursor)  ##  输出跑SQL结果
            return data

        ###sql
        sql_badrate='''
        SELECT 
        case 
        {score_cut}
        end as score ,
        count( DISTINCT case when b.th>={bad} THEN b.customerid  END ) AS '坏样本',
        count( DISTINCT case when b.th>=0 THEN b.customerid  END ) AS '总数'
        FROM 
        (select customerid,serialno
        from fk.v_g_aprvadt_appl_bsc_inf
        where data_dt={time_control}
        and inputdate between {inputdate_control1} and {inputdate_control2}
        and decisionstage='Apply'--申请
        and businesstype={businesstype}
        )a
        JOIN
        (SELECT customerid, max(bd_hist_high_odue_days) AS th
        FROM fk.v_g_loanlend_loan_inf_his
        WHERE data_dt={time_control}
        AND {time_control}> date(bd_fst_maturity_dt)+{day}
        GROUP BY 1
        ) b
        ON a.customerid=b.customerid
        right JOIN
        (SELECT objectno,{creditscore} AS score
        FROM fk.v_r_risk_fico_xml_format
        WHERE data_dt<={time_control}
        AND {creditscore}>0
        ) c
        ON a.serialno=c.objectno
        GROUP BY score
        ORDER BY score
        '''
        sql_badrate1='''
        SELECT 
        case 
        {score_cut}
        end as score ,
        count( DISTINCT case when b.th>={bad} THEN b.customerid  END ) AS '坏样本',
        count( DISTINCT case when b.th>=0 THEN b.customerid  END ) AS '总数'
        FROM 
        (select customerid,serialno
        from fk.v_g_aprvadt_appl_bsc_inf
        where data_dt={time1}
        and inputdate between {inputdate1_1} and {inputdate1_2}
        and decisionstage='Apply'--申请
        and businesstype={businesstype}
        )a
        JOIN
        (SELECT customerid, max(bd_hist_high_odue_days) AS th
        FROM fk.v_g_loanlend_loan_inf_his
        WHERE data_dt={time1}
        AND {time1}> date(bd_fst_maturity_dt)+{day}
        GROUP BY 1
        ) b
        ON a.customerid=b.customerid
        right JOIN
        (SELECT objectno,{creditscore} AS score
        FROM fk.v_r_risk_fico_xml_format
        WHERE data_dt<={time1}
        AND {creditscore}>0
        ) c
        ON a.serialno=c.objectno
        GROUP BY score
        ORDER BY score
        '''
        sql_badrate2='''
        SELECT 
        case 
        {score_cut}
        end as score ,
        count( DISTINCT case when b.th>={bad} THEN b.customerid  END ) AS '坏样本',
        count( DISTINCT case when b.th>=0 THEN b.customerid  END ) AS '总数'
        FROM 
        (select customerid,serialno
        from fk.v_g_aprvadt_appl_bsc_inf
        where data_dt={time2}
        and inputdate between {inputdate2_1} and {inputdate2_2}
        and decisionstage='Apply'--申请
        and businesstype={businesstype} 
        )a
        JOIN
        (SELECT customerid, max(bd_hist_high_odue_days) AS th
        FROM fk.v_g_loanlend_loan_inf_his
        WHERE data_dt={time2}
        AND {time2}> date(bd_fst_maturity_dt)+{day}
        GROUP BY 1
        ) b
        ON a.customerid=b.customerid
        right JOIN
        (SELECT objectno,{creditscore} AS score
        FROM fk.v_r_risk_fico_xml_format
        WHERE data_dt<={time2}
        AND {creditscore}>0
        ) c
        ON a.serialno=c.objectno
        GROUP BY score
        ORDER BY score
        '''
        sql_badrate3='''
        SELECT 
        case 
        {score_cut}
        end as score ,
        count( DISTINCT case when b.th>={bad} THEN b.customerid  END ) AS '坏样本',
        count( DISTINCT case when b.th>=0 THEN b.customerid  END ) AS '总数'
        FROM 
        (select customerid,serialno
        from fk.v_g_aprvadt_appl_bsc_inf
        where data_dt={time3}
        and inputdate between {inputdate3_1} and {inputdate3_2}
        and decisionstage='Apply'--申请
        and businesstype={businesstype}
        )a
        JOIN
        (SELECT customerid, max(bd_hist_high_odue_days) AS th
        FROM fk.v_g_loanlend_loan_inf_his
        WHERE data_dt={time3}
        AND {time3}> date(bd_fst_maturity_dt)+{day}
        GROUP BY 1
        ) b
        ON a.customerid=b.customerid
        right JOIN
        (SELECT objectno,{creditscore} AS score
        FROM fk.v_r_risk_fico_xml_format
        WHERE data_dt<={time3}
        AND {creditscore}>0
        ) c
        ON a.serialno=c.objectno
        GROUP BY score
        ORDER BY score
        '''
        if creditscore!='creditScore55':
            sql_badrate_end= sql_badrate.format(
            score_cut=score_cut,
            time_control=time_control,
            inputdate_control1=inputdate_control1,
            inputdate_control2=inputdate_control2 ,
            creditscore=creditscore,
            businesstype=businesstype,
            bad='30',
            day='90'  
            )
            badrate_control=run_sql(sql_badrate_end)
            print(badrate_control)
            if badrate_control['总数'].sum()<=20:
                print(creditscore+'控制组没有数据')
            else:
                if len(badrate_control)<10:
                    print(creditscore+'数据有问题')
                else:
                    sql_badrate_end1 = sql_badrate1.format(
                    score_cut=score_cut,
                    time1=time1,
                    inputdate1_1=inputdate1_1,
                    inputdate1_2=inputdate1_2 ,
                    creditscore=creditscore,
                    businesstype=businesstype,
                    bad='30',
                    day='90'  
                    )
                    badrate1=run_sql(sql_badrate_end1)
                    print(badrate1)
                    if badrate1['总数'].sum()<=20:
                        print(creditscore+'第一组没有数据')
                    else:
                        if len(badrate1)<10:
                            print(creditscore+'数据有问题')
                        else:
                            sql_badrate_end2 = sql_badrate2.format(
                            score_cut=score_cut,
                            time2=time2,
                            inputdate2_1=inputdate2_1,
                            inputdate2_2=inputdate2_2 ,
                            creditscore=creditscore,
                            businesstype=businesstype,
                            bad='30',
                            day='90'
                            )
                            badrate2=run_sql(sql_badrate_end2)
                            print(badrate2)
                            if badrate2['总数'].sum()<=0:
                                print(creditscore+'第二组没有数据')
                            else:
                                if len(badrate2)<10:
                                    print(creditscore+'数据有问题')
                                else:
                                    sql_badrate_end3 = sql_badrate3.format(
                                    score_cut=score_cut,
                                    time3=time3,
                                    inputdate3_1=inputdate3_1,
                                    inputdate3_2=inputdate3_2 ,
                                    creditscore=creditscore,
                                    businesstype=businesstype,
                                    bad='30',
                                    day='90'
                                    )
                                    badrate3=run_sql(sql_badrate_end3)
                                    print(badrate3)
                                    if badrate3['总数'].sum()<=0:
                                        print(creditscore+'第三组没有数据')
                                    else:
                                        if len(badrate3)<10:
                                            print(creditscore+'数据有问题')
                                        else:
                                            ##计算Badrate
                                            badrate_control['badrate']=round(badrate_control['坏样本']/badrate_control['总数'],4)
                                            where_are_inf_control = np.isnan(badrate_control['badrate'])  ##查找是inf的PSI
                                            badrate_control['badrate'][where_are_inf_control] = 0.0000
                                            
                                            badrate1['badrate']=round(badrate1['坏样本']/badrate1['总数'],4)
                                            where_are_inf1 = np.isnan(badrate1['badrate'])  ##查找是inf的PSI
                                            badrate1['badrate'][where_are_inf1] = 0.0000

                                            badrate2['badrate']=round(badrate2['坏样本']/badrate2['总数'],4)
                                            where_are_inf2 = np.isnan(badrate2['badrate'])  ##查找是inf的PSI
                                            badrate2['badrate'][where_are_inf2] = 0.0000

                                            badrate3['badrate']=round(badrate3['坏样本']/badrate3['总数'],4)
                                            where_are_inf3 = np.isnan(badrate3['badrate'])  ##查找是inf的PSI
                                            badrate3['badrate'][where_are_inf3] = 0.0000
                                            
                                            Badrate_control=badrate_control['badrate'].tolist()
                                            Badrate1=badrate1['badrate'].tolist()
                                            Badrate2=badrate2['badrate'].tolist()
                                            Badrate3=badrate3['badrate'].tolist()

                                            ##最小二乘法
                                            #####采样点(Xi,Yi)###
                                            Badrate_Xi=np.array([2,3,4,5,6,7,8,9,10])
                                            Badrate_control_Yi=np.array(Badrate_control[1:])
                                            Badrate1_Yi=np.array(Badrate1[1:])
                                            Badrate2_Yi=np.array(Badrate2[1:])
                                            Badrate3_Yi=np.array(Badrate3[1:])
                                            ###需要拟合的函数func及误差error###
                                            def func(p,x):
                                                k,b=p
                                                return k*x+b
                                            def error(p,x,y):
                                                return func(p,x)-y #x、y都是列表，故返回值也是个列表
                                            #TEST
                                            Badrate_control_p0=[1,Badrate_control[0]]
                                            Badrate1_p0=[1,Badrate1[0]]
                                            Badrate2_p0=[1,Badrate2[0]]
                                            Badrate3_p0=[1,Badrate3[0]]
                                            ###主函数从此开始###

                                            Para_control=leastsq(error,Badrate_control_p0,args=(Badrate_Xi,Badrate_control_Yi)) #把error函数中除了p以外的参数打包到args中
                                            k0,b0=Para_control[0]
                                            Para1=leastsq(error,Badrate1_p0,args=(Badrate_Xi,Badrate1_Yi)) #把error函数中除了p以外的参数打包到args中
                                            k1,b1=Para1[0]
                                            Para2=leastsq(error,Badrate2_p0,args=(Badrate_Xi,Badrate2_Yi)) #把error函数中除了p以外的参数打包到args中
                                            k2,b2=Para2[0]
                                            Para3=leastsq(error,Badrate3_p0,args=(Badrate_Xi,Badrate3_Yi)) #把error函数中除了p以外的参数打包到args中
                                            k3,b3=Para3[0]
                                            print("k0=",k0,'\n',"b0=",b0)
                                            print("k1=",k1,'\n',"b1=",b1) 
                                            print("k2=",k2,'\n',"b2=",b2)
                                            print("k3=",k3,'\n',"b3=",b3)
                                            I=list(range(1,11,1))
                                            y0=[]
                                            for i in I:
                                                y0.append(k0*i+b0)
                                            y1=[]
                                            for i in I:
                                                y1.append(k1*i+b1)
                                            y2=[]
                                            for i in I:
                                                y2.append(k2*i+b2)
                                            y3=[]
                                            for i in I:
                                                y3.append(k3*i+b3)
                                            k0=str(round(k0,4))   
                                            k1=str(round(k1,4))
                                            k2=str(round(k2,4))
                                            k3=str(round(k3,4))
                    
                                            def line() -> Line:
                                                c = (
                                                    Line(init_opts=opts.InitOpts(width="1200px"))
                                                    .add_xaxis(xaxis_data=list(badrate_control['score']))
                                                    .add_yaxis(
                                                        series_name="Badrate0-趋势线为："+k0,
                                                        y_axis=Badrate_control,
                                                        color="red",
                                                        label_opts=opts.LabelOpts(is_show=False),
                                                        is_smooth=True
                                                        )
                                                    .add_yaxis(
                                                        series_name="Badrate1-趋势线为："+k1,
                                                        y_axis=Badrate1,
                                                        color="#9932CC",
                                                        label_opts=opts.LabelOpts(is_show=False),
                                                        is_smooth=True
                                                        )
                                                    .add_yaxis(
                                                        series_name="Badrate2-趋势线为："+k2,
                                                        y_axis=Badrate2,
                                                        color="green",
                                                        label_opts=opts.LabelOpts(is_show=False),
                                                        is_smooth=True)
                                                    .add_yaxis(
                                                        series_name="Badrate3-趋势线为："+k3,
                                                        y_axis=Badrate3,
                                                        color="blue",
                                                        label_opts=opts.LabelOpts(is_show=False),
                                                        is_smooth=True
                                                        )
#                                                     .add_yaxis(
#                                                         series_name="Badrate0趋势"+k0,
#                                                         y_axis=y0,
#                                                         #color="red",
#                                                         symbol="triangle",
#                                                         linestyle_opts=opts.LineStyleOpts(color="red", width=2, type_="dashed"),
#                                                         label_opts=opts.LabelOpts(is_show=False)##不主动显示数值
#                                                         )
#                                                     .add_yaxis(
#                                                         series_name="Badrate1趋势"+k1,
#                                                         y_axis=y1,
#                                                         symbol="triangle",
#                                                         #color="#9932CC",
#                                                         linestyle_opts=opts.LineStyleOpts(color="#9932CC", width=2, type_="dashed"),
#                                                         label_opts=opts.LabelOpts(is_show=False)##不主动显示数值
#                                                         )
#                                                     .add_yaxis(
#                                                         series_name="Badrate2趋势"+k2,
#                                                         y_axis=y2,
#                                                         #color="green",
#                                                         symbol="triangle",
#                                                         linestyle_opts=opts.LineStyleOpts(color="green", width=2, type_="dashed"),
#                                                         label_opts=opts.LabelOpts(is_show=False)##不主动显示数值
#                                                         )
#                                                     .add_yaxis(
#                                                         series_name="Badrate3趋势"+k3,
#                                                         y_axis=y3,
#                                                         #color="blue",
#                                                         label_opts=opts.LabelOpts(is_show=False),##不主动显示数值
#                                                         symbol="triangle",
#                                                         linestyle_opts=opts.LineStyleOpts(color="blue", width=2, type_="dashed"),
#                                                         )
                                                    .set_global_opts(title_opts=opts.TitleOpts(title="效力图",
                                                                                               subtitle=chinese_name+":"+" 控制组授信:"+inputdate_control1[1:10]+"-"+inputdate_control2[1:10]+
                                                                                              " 1组授信:"+inputdate1_1[1:10]+"-"+inputdate1_2[1:10]+
                                                                                              " 2组授信:"+inputdate2_1[1:10]+"-"+inputdate2_2[1:10]+
                                                                                              " 3组授信:"+inputdate3_1[1:10]+"-"+inputdate3_2[1:10])))
                                                return c
                                            tab.add(line(), creditscore)
        else:
            sql_badrate_end= sql_badrate.format(
            score_cut=score_cut,
            time_control=time_control,
            inputdate_control1=inputdate_control1,
            inputdate_control2=inputdate_control2 ,
            creditscore=creditscore,
            businesstype=businesstype,
            bad='1',
            day='7'  
            )
            badrate_control=run_sql(sql_badrate_end)
            print(badrate_control)
            if badrate_control['总数'].sum()<=20:
                print(creditscore+'控制组没有数据')
            else:
                if len(badrate_control)<10:
                    print(creditscore+'数据有问题')
                else:
                    sql_badrate_end1 = sql_badrate1.format(
                    score_cut=score_cut,
                    time1=time55_1,
                    inputdate1_1=inputdate1_1,
                    inputdate1_2=inputdate1_2 ,
                    creditscore=creditscore,
                    businesstype=businesstype,
                    bad='1',
                    day='7'  
                    )
                    badrate1=run_sql(sql_badrate_end1)
                    print(badrate1)
                    if badrate1['总数'].sum()<=20:
                        print(creditscore+'第一组没有数据')
                    else:
                        if len(badrate1)<10:
                            print(creditscore+'数据有问题')
                        else:
                            sql_badrate_end2 = sql_badrate2.format(
                            score_cut=score_cut,
                            time2=time55_2,
                            inputdate2_1=inputdate2_1,
                            inputdate2_2=inputdate2_2 ,
                            creditscore=creditscore,
                            businesstype=businesstype,
                            bad='1',
                            day='7'
                            )
                            badrate2=run_sql(sql_badrate_end2)
                            print(badrate2)
                            if badrate2['总数'].sum()<=0:
                                print(creditscore+'第二组没有数据')
                            else:
                                if len(badrate2)<10:
                                    print(creditscore+'数据有问题')
                                else:
                                    sql_badrate_end3 = sql_badrate3.format(
                                    score_cut=score_cut,
                                    time3=time55_3,
                                    inputdate3_1=inputdate3_1,
                                    inputdate3_2=inputdate3_2 ,
                                    creditscore=creditscore,
                                    businesstype=businesstype,
                                    bad='1',
                                    day='7'
                                    )
                                    badrate3=run_sql(sql_badrate_end3)
                                    print(badrate3)
                                    if badrate3['总数'].sum()<=0:
                                        print(creditscore+'第三组没有数据')
                                    else:
                                        if len(badrate3)<10:
                                            print(creditscore+'数据有问题')
                                        else:
                                    ##计算Badrate
                                            badrate_control['badrate']=round(badrate_control['坏样本']/badrate_control['总数'],4)
                                            where_are_inf_control = np.isnan(badrate_control['badrate'])  ##查找是inf的PSI
                                            badrate_control['badrate'][where_are_inf_control] = 0.0000
                                            
                                            badrate1['badrate']=round(badrate1['坏样本']/badrate1['总数'],4)
                                            where_are_inf1 = np.isnan(badrate1['badrate'])  ##查找是inf的PSI
                                            badrate1['badrate'][where_are_inf1] = 0.0000

                                            badrate2['badrate']=round(badrate2['坏样本']/badrate2['总数'],4)
                                            where_are_inf2 = np.isnan(badrate2['badrate'])  ##查找是inf的PSI
                                            badrate2['badrate'][where_are_inf2] = 0.0000

                                            badrate3['badrate']=round(badrate3['坏样本']/badrate3['总数'],4)
                                            where_are_inf3 = np.isnan(badrate3['badrate'])  ##查找是inf的PSI
                                            badrate3['badrate'][where_are_inf3] = 0.0000
                                            
                                            Badrate_control=badrate_control['badrate'].tolist()
                                            Badrate1=badrate1['badrate'].tolist()
                                            Badrate2=badrate2['badrate'].tolist()
                                            Badrate3=badrate3['badrate'].tolist()

                                            ##最小二乘法
                                            #####采样点(Xi,Yi)###
                                            Badrate_Xi=np.array([2,3,4,5,6,7,8,9,10])
                                            Badrate_control_Yi=np.array(Badrate_control[1:])
                                            Badrate1_Yi=np.array(Badrate1[1:])
                                            Badrate2_Yi=np.array(Badrate2[1:])
                                            Badrate3_Yi=np.array(Badrate3[1:])
                                            ###需要拟合的函数func及误差error###
                                            def func(p,x):
                                                k,b=p
                                                return k*x+b
                                            def error(p,x,y):
                                                return func(p,x)-y #x、y都是列表，故返回值也是个列表
                                            #TEST
                                            Badrate_control_p0=[1,Badrate_control[0]]
                                            Badrate1_p0=[1,Badrate1[0]]
                                            Badrate2_p0=[1,Badrate2[0]]
                                            Badrate3_p0=[1,Badrate3[0]]
                                            ###主函数从此开始###

                                            Para_control=leastsq(error,Badrate_control_p0,args=(Badrate_Xi,Badrate_control_Yi)) #把error函数中除了p以外的参数打包到args中
                                            k0,b0=Para_control[0]
                                            Para1=leastsq(error,Badrate1_p0,args=(Badrate_Xi,Badrate1_Yi)) #把error函数中除了p以外的参数打包到args中
                                            k1,b1=Para1[0]
                                            Para2=leastsq(error,Badrate2_p0,args=(Badrate_Xi,Badrate2_Yi)) #把error函数中除了p以外的参数打包到args中
                                            k2,b2=Para2[0]
                                            Para3=leastsq(error,Badrate3_p0,args=(Badrate_Xi,Badrate3_Yi)) #把error函数中除了p以外的参数打包到args中
                                            k3,b3=Para3[0]
                                            print("k0=",k0,'\n',"b0=",b0)
                                            print("k1=",k1,'\n',"b1=",b1) 
                                            print("k2=",k2,'\n',"b2=",b2)
                                            print("k3=",k3,'\n',"b3=",b3)
                                            I=list(range(1,11,1))
                                            y0=[]
                                            for i in I:
                                                y0.append(k0*i+b0)
                                            y1=[]
                                            for i in I:
                                                y1.append(k1*i+b1)
                                            y2=[]
                                            for i in I:
                                                y2.append(k2*i+b2)
                                            y3=[]
                                            for i in I:
                                                y3.append(k3*i+b3)
                                            k0=str(round(k0,4))   
                                            k1=str(round(k1,4))
                                            k2=str(round(k2,4))
                                            k3=str(round(k3,4))
                    
                                            def line() -> Line:
                                                c = (
                                                    Line(init_opts=opts.InitOpts(width="1200px"))
                                                    .add_xaxis(xaxis_data=list(badrate_control['score']))
                                                    .add_yaxis(
                                                        series_name="Badrate0-趋势线为："+k0,
                                                        y_axis=Badrate_control,
                                                        color="red",
                                                        label_opts=opts.LabelOpts(is_show=False),
                                                        is_smooth=True
                                                        )
                                                    .add_yaxis(
                                                        series_name="Badrate1-趋势线为："+k1,
                                                        y_axis=Badrate1,
                                                        color="#9932CC",
                                                        label_opts=opts.LabelOpts(is_show=False),
                                                        is_smooth=True
                                                        )
                                                    .add_yaxis(
                                                        series_name="Badrate2-趋势线为："+k2,
                                                        y_axis=Badrate2,
                                                        color="green",
                                                        label_opts=opts.LabelOpts(is_show=False),
                                                        is_smooth=True)
                                                    .add_yaxis(
                                                        series_name="Badrate3-趋势线为："+k3,
                                                        y_axis=Badrate3,
                                                        color="blue",
                                                        label_opts=opts.LabelOpts(is_show=False),
                                                        is_smooth=True
                                                        )
#                                                     .add_yaxis(
#                                                         series_name="Badrate0趋势"+k0,
#                                                         y_axis=y0,
#                                                         #color="red",
#                                                         symbol="triangle",
#                                                         linestyle_opts=opts.LineStyleOpts(color="red",width=2, type_="dashed"),
#                                                         label_opts=opts.LabelOpts(is_show=False)##不主动显示数值
#                                                         )
#                                                     .add_yaxis(
#                                                         series_name="Badrate1趋势"+k1,
#                                                         y_axis=y1,
#                                                         #color="#9932CC",
#                                                         symbol="triangle",
#                                                         linestyle_opts=opts.LineStyleOpts(color="#9932CC",width=2, type_="dashed"),
#                                                         label_opts=opts.LabelOpts(is_show=False)##不主动显示数值
#                                                         )
#                                                     .add_yaxis(
#                                                         series_name="Badrate2趋势"+k2,
#                                                         y_axis=y2,
#                                                         #color="green",
#                                                         symbol="triangle",
#                                                         linestyle_opts=opts.LineStyleOpts(color="green",width=2, type_="dashed"),
#                                                         label_opts=opts.LabelOpts(is_show=False)##不主动显示数值
#                                                         )
#                                                     .add_yaxis(
#                                                         series_name="Badrate3趋势"+k3,
#                                                         y_axis=y3,
#                                                         #color="blue",
#                                                         label_opts=opts.LabelOpts(is_show=False),##不主动显示数值
#                                                         symbol="triangle",
#                                                         linestyle_opts=opts.LineStyleOpts(color="blue",width=2, type_="dashed"),
#                                                         )
                                                    .set_global_opts(title_opts=opts.TitleOpts(title="效力图",
                                                                                               subtitle=chinese_name+":"+" 控制组授信:"+inputdate_control1[1:10]+"-"+inputdate_control2[1:10]+
                                                                                              " 1组授信:"+inputdate1_1[1:10]+"-"+inputdate1_2[1:10]+
                                                                                              " 2组授信:"+inputdate2_1[1:10]+"-"+inputdate2_2[1:10]+
                                                                                              " 3组授信:"+inputdate3_1[1:10]+"-"+inputdate3_2[1:10])))
                                                return c
                                            tab.add(line(), creditscore)

    return tab.render("模型效力展示.html")

## 运行部分

In [17]:
effect(user='fk_yanjintong'##用户
       ,password = 'Yjt@0828$$'
       ,inputdate1_1="'"'20201001'"'"
       ,inputdate1_2="'"'20201101'"'"
       ,inputdate2_1="'"'20201101'"'"
       ,inputdate2_2="'"'20201201'"'"
       ,inputdate3_1="'"'20201201'"'"
       ,inputdate3_2="'"'20210101'"'"
       ,time1="'"'2021-03-01'"'"
       ,time2="'"'2021-04-01'"'"
       ,time3="'"'2021-05-01'"'"
       ,time55_1="'"'2020-12-08'"'"
       ,time55_2="'"'2021-01-08'"'"
       ,time55_3="'"'2021-02-08'"'"
      )

creditScore46运行时间为: 229.7585084438324
       score  坏样本    总数
0    (0,456]   58   920
1  (456,477]  114  2457
2  (477,498]  171  3807
3  (498,514]  189  4337
4  (514,529]  213  4976
5  (529,544]  231  5642
6  (544,561]  157  5149
7  (561,579]  117  4705
8  (579,601]   87  4632
9    (601,-)  100  5482
creditScore46运行时间为: 342.41893219947815
       score  坏样本    总数
0    (0,456]    0     9
1  (456,477]    0   116
2  (477,498]   19   462
3  (498,514]   12   914
4  (514,529]   24  1647
5  (529,544]   12  1681
6  (544,561]   37  3024
7  (561,579]   20  2957
8  (579,601]   34  6931
9    (601,-)   15  3672
creditScore46运行时间为: 240.96742868423462
       score  坏样本    总数
0    (0,456]    0     8
1  (456,477]    0    96
2  (477,498]    7   320
3  (498,514]   12   591
4  (514,529]   12  1100
5  (529,544]   18  1085
6  (544,561]   24  1974
7  (561,579]   12  1849
8  (579,601]   15  3861
9    (601,-)    6  1983
creditScore46运行时间为: 228.4096736907959
       score  坏样本    总数
0    (0,456]    0     7
1  (45

D:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:331: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:335: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:339: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:343: SettingWithCopyWarning

creditScore48运行时间为: 231.1985261440277
             score  坏样本    总数
0       (0,0.0031]    4   416
1  (0.0031,0.0061]    5  1137
2  (0.0061,0.0091]    4  1069
3  (0.0091,0.0121]    8   772
4  (0.0121,0.0151]    8   493
5  (0.0151,0.0181]    3   318
6  (0.0181,0.0211]    2   201
7  (0.0211,0.0241]    3   129
8  (0.0241,0.0271]    0    65
9       (0.0271,-]    1   128
creditScore48运行时间为: 243.93806958198547
             score  坏样本    总数
0       (0,0.0031]    6  2024
1  (0.0031,0.0061]   22  4424
2  (0.0061,0.0091]   36  4222
3  (0.0091,0.0121]   30  2964
4  (0.0121,0.0151]   18  2048
5  (0.0151,0.0181]   16  1265
6  (0.0181,0.0211]   11   738
7  (0.0211,0.0241]    6   403
8  (0.0241,0.0271]    4   266
9       (0.0271,-]    5   342
creditScore48运行时间为: 230.43996143341064
             score  坏样本    总数
0       (0,0.0031]    2   708
1  (0.0031,0.0061]   12  1770
2  (0.0061,0.0091]   13  1698
3  (0.0091,0.0121]   12  1219
4  (0.0121,0.0151]    6   845
5  (0.0151,0.0181]    2   554
6  (0.0181,0.0

D:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:546: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:550: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:554: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:558: SettingWithCopyWarning

creditScore56运行时间为: 929.908433675766
       score  坏样本    总数
0    (0,511]   28  1854
1  (511,520]    6   927
2  (520,525]    5   517
3  (525,435]   10   985
4  (535,541]   16  1413
5  (541,551]   17  2313
6  (551,558]    6  1236
7  (558,568]    5  1606
8  (568,578]    7   753
9    (578,-)   10  1323
creditScore56运行时间为: 555.6202392578125
       score  坏样本    总数
0    (0,511]   30  1694
1  (511,520]   15  1157
2  (520,525]    5   673
3  (525,435]   15  1345
4  (535,541]   18  2526
5  (541,551]   35  4564
6  (551,558]   15  2401
7  (558,568]   18  3147
8  (568,578]   13  1419
9    (578,-)    9  2487
creditScore56运行时间为: 437.45431089401245
       score  坏样本    总数
0    (0,511]   19  1208
1  (511,520]   10   716
2  (520,525]    3   393
3  (525,435]    8   770
4  (535,541]   13  1364
5  (541,551]   21  2919
6  (551,558]    9  1511
7  (558,568]   18  1695
8  (568,578]    1   840
9    (578,-)    4  1454
creditScore56运行时间为: 347.6019446849823
       score  坏样本    总数
0    (0,511]   24  1769
1  (511,

'C:\\Users\\jintong.yan\\模型效力展示.html'

# Badrate四个月数据版（最终使用）

In [ ]:
##所用到的包
import numpy as np
from impala.util import as_pandas
from impala.dbapi import connect
import time
import functools
from pyecharts.charts import Tab
import pandas as pd
from pyecharts.components import Table
from pyecharts.options import ComponentTitleOpts
##定义函数
def effect(user,password,inputdate1_1,inputdate1_2,inputdate2_1,inputdate2_2,inputdate3_1,inputdate3_2,time1,time2,time3,time55_1,time55_2,time55_3):
    ## 输入数据库账号
    conn = connect(host='10.1.57.8', 
                   port=10000, 
                   auth_mechanism='PLAIN',
                   user=user,
                   password = password,
                   database='fk')
    #连接数据库
    cursor=conn.cursor()
    
    tab = Tab()
    ks_dic={}
    #确定需要监控的模型
    creditscores=['creditScore46','creditScore48','creditScore49','creditScore50','creditScore51',
                  'creditScore52','creditScore54','creditScore55' ,'creditScore56','creditScore57',
                  'creditScore58','creditScore59','creditScore60','creditScore61','creditScore62',
                  'creditScore63'
#                   ,'creditScore64','creditScore65'
                 ]
#     creditscores=['creditScore55']
#     chinese_names= ['借呗阿波罗模型V2']
#     score_cuts=["when score >0  and score <=456 then '(0,456]' when score >456 and score <=477 then '(456,477]' when score >477 and score <=498 then '(477,498]' when score >498 and score <=514 then '(498,514]' when score >514 and score <=529 then '(514,529]' when score >529 and score <=544 then '(529,544]' when score >544 and score <=561 then '(544,561]' when score >561 and score <=579 then '(561,579]' when score >579 and score <=601 then '(579,601]' when score >601  then '(601,-)'" ]
#     time_controls=["'"'2020-10-27'"'"]
#     inputdate_control1s=["'"'20200820'"'"]
#     inputdate_control2s=["'"'20200920'"'"]
    #需要监控模型对应的中文名（注意：要一一对应）
    chinese_names= ['借呗阿波罗模型V2',
                    '借呗联合模型V2',
                    '借呗阿波罗模型V3（XGB）',
                    '借呗阿波罗模型V3（LGB）',
                    '借呗阿波罗模型V3（权重版）',
                    '京东A卡',
                    '借呗三方征信模型',
                    '借呗阿波罗入催模型',
                    '借呗负债压力模型',
                    '借呗人行二代模型',
                    '借呗融合模型-全变量'
                    ,'借呗融合模型-去除score56',
                    '京东plusA卡V2',
                    '借呗阿波罗V4',
                    '借呗三方征信V2',
                    '借呗阿波罗模型V1'
#                     ,'京东plus卡A卡V3.1',
#                     '京东plus卡A卡V3.2'
                   ]
    
    score_cuts=["when score >0  and score <=456 then '(0,456]' when score >456 and score <=477 then '(456,477]' when score >477 and score <=498 then '(477,498]' when score >498 and score <=514 then '(498,514]' when score >514 and score <=529 then '(514,529]' when score >529 and score <=544 then '(529,544]' when score >544 and score <=561 then '(544,561]' when score >561 and score <=579 then '(561,579]' when score >579 and score <=601 then '(579,601]' when score >601  then '(601,-)'" 
    ,"when score >0  and score <=0.0031 then  '(0,0.0031]' when score >0.0031 and score <=0.0061 then '(0.0031,0.0061]' when score >0.0061 and score <=0.0091 then '(0.0061,0.0091]' when score >0.0091 and score <=0.0121 then '(0.0091,0.0121]' when score >0.0121 and score <=0.0151 then '(0.0121,0.0151]' when score >0.0151 and score <=0.0181 then '(0.0151,0.0181]' when score >0.0181 and score <=0.0211 then '(0.0181,0.0211]' when score >0.0211 and score <=0.0241 then '(0.0211,0.0241]' when score >0.0241 and score <=0.0271 then '(0.0241,0.0271]' when score >0.0271 then '(0.0271,-]'"
    ,"when score >0  and score <=555 then  '(0,555]' when score >555 and score <=569 then '(555,569]' when score >569 and score <=578 then '(569,578]' when score >578 and score <=582 then '(578,582]' when score >582 and score <=587 then '(582,587]' when score >587 and score <=595 then '(587,595]' when score >595 and score <=605 then '(595,605]' when score >605 and score <=619 then '(605,619]' when score >619 and score <=629 then '(619,629]' when score >629  then '(629,-)'"
    ,"when score >0  and score <=470 then  '(0,470]' when score >470 and score <=484 then '(470,484]' when score >484 and score <=495 then '(484,495]' when score >495 and score <=503 then '(495,503]' when score >503 and score <=528 then '(503,528]' when score >528 and score <=539 then '(528,539]' when score >539 and score <=546 then '(539,546]' when score >546 and score <=612 then '(546,612]' when score >612 and score <=632 then '(612,632]' when score >632  then '(632,-)'"
    ,"when score >0  and score <=425 then  '(0,425]' when score >425 and score <=441 then '(425,441]' when score >441 and score <=449 then '(441,449]' when score >449 and score <=461 then '(449,461]' when score >461 and score <=474 then '(461,474]' when score >474 and score <=490 then '(474,490]' when score >490 and score <=511 then '(490,511]' when score >511 and score <=532 then '(511,532]' when score >532 and score <=557 then '(532,557]' when score >557  then '(557,-)'"
    ,"when score >0  and score <=672 then  '(0,672]' when score >672 and score <=685 then '(672,685]' when score >685 and score <=695 then '(685,695]' when score >695 and score <=704 then '(695,704]' when score >704 and score <=713 then '(704,713]' when score >713 and score <=723 then '(713,723]' when score >723 and score <=735 then '(723,735]' when score >735 and score <=750 then '(735,750]' when score >750 and score <=771 then '(750,771]' when score >771  then '(771,-)'"
    ,"when score >0  and score <=462 then  '(0,462]' when score >462 and score <=469 then '(462,469]' when score >469 and score <=475 then '(469,475]' when score >475 and score <=480 then '(475,480]' when score >480 and score <=485 then '(480,485]' when score >485 and score <=489 then '(485,489]' when score >489 and score <=496 then '(489,496]' when score >496 and score <=503 then '(496,503]' when score >503 and score <=514 then '(503,514]' when score >514  then '(514,-)'"
    ,"when score >0  and score <=379 then  '(0,379]' when score >379 and score <=390 then '(379,390]' when score >390 and score <=398 then '(390,398]' when score >398 and score <=409 then '(398,409]' when score >409 and score <=415 then '(409,415]' when score >415 and score <=425 then '(415,425]' when score >425 and score <=437 then '(425,437]' when score >437 and score <=451 then '(437,451]' when score >451 and score <=472 then '(451,472]' when score >472  then '(472,-)'"
    ,"when score >0  and score <=511 then  '(0,511]' when score >511 and score <=520 then '(511,520]' when score >520 and score <=525 then '(520,525]' when score >525 and score <=535 then '(525,435]' when score >535 and score <=541 then '(535,541]' when score >541 and score <=551 then '(541,551]' when score >551 and score <=558 then '(551,558]' when score >558 and score <=568 then '(558,568]' when score >568 and score <=578 then '(568,578]' when score >578  then '(578,-)'"
    ,"when score >0  and score <=539 then  '(0,539]' when score >539 and score <=563 then '(539,563]' when score >563 and score <=587 then '(563,587]' when score >587 and score <=611 then '(587,611]' when score >611 and score <=635 then '(611,635]' when score >635 and score <=659 then '(635,659]' when score >659 and score <=683 then '(659,683]' when score >683 and score <=707 then '(683,707]' when score >707 and score <=731 then '(707,731]' when score >731  then '(731,-)'"
    ,"when score >0  and score <=412 then  '(0,412]' when score >412 and score <=460 then '(412,460]' when score >460 and score <=508 then '(460,508]' when score >508 and score <=556 then '(508,556]' when score >556 and score <=604 then '(556,604]' when score >604 and score <=652 then '(604,652]' when score >652 and score <=700 then '(652,700]' when score >700 and score <=748 then '(700,748]' when score >748 and score <=796 then '(748,796]' when score >796  then '(796,-)'"
    ,"when score >0  and score <=413 then  '(0,413]' when score >413 and score <=463 then '(413,463]' when score >463 and score <=513 then '(463,513]' when score >513 and score <=562 then '(513,562]' when score >562 and score <=612 then '(562,612]' when score >612 and score <=662 then '(612,662]' when score >662 and score <=712 then '(662,712]' when score >712 and score <=761 then '(712,761]' when score >761 and score <=796 then '(761,796]' when score >796  then '(796,-)'"
    ,"when score >0  and score <=584 then  '(0,584]' when score >584 and score <=592 then '(584,592]' when score >592 and score <=596 then '(592,596]' when score >596 and score <=600 then '(596,600]' when score >600 and score <=604 then '(600,604]' when score >604 and score <=608 then '(604,608]' when score >608 and score <=613 then '(608,613]' when score >613 and score <=618 then '(613,618]' when score >618 and score <=624 then '(618,624]' when score >624  then '(624,-)'"
    ,"when score >0  and score <=430 then  '(0,430]' when score >430 and score <=442 then '(430,442]' when score >442 and score <=454 then '(442,454]' when score >454 and score <=461 then '(454,461]' when score >461 and score <=466 then '(461,466]' when score >466 and score <=476 then '(466,476]' when score >476 and score <=498 then '(476,498]' when score >498 and score <=521 then '(498,521]' when score >521 and score <=536 then '(521,536]' when score >536  then '(536,-)'"
    ,"when score >0  and score <=153 then  '(0,153]' when score >153 and score <=170 then '(153,170]' when score >170 and score <=184 then '(170,184]' when score >184 and score <=197 then '(184,197]' when score >197 and score <=209 then '(197,209]' when score >209 and score <=220 then '(209,220]' when score >220 and score <=234 then '(220,234]' when score >234 and score <=249 then '(234,249]' when score >249 and score <=269 then '(249,269]' when score >269  then '(269,-)'"
    ,"when score >0  and score <=545 then  '(0,545]' when score >545 and score <=579 then '(545,579]' when score >579 and score <=603 then '(579,603]' when score >603 and score <=629 then '(603,629]' when score >629 and score <=647 then '(629,647]' when score >647 and score <=673 then '(647,673]' when score >673 and score <=693 then '(673,693]' when score >693 and score <=723 then '(693,723]' when score >723 and score <=751 then '(723,751]' when score >751  then '(751,-)'"
    #,"when score >0  and score <=583 then  '(0,583]' when score >583 and score <=590 then '(583,590]' when score >590 and score <=595 then '(590,595]' when score >595 and score <=600 then '(595,600]' when score >600 and score <=604 then '(600,604]' when score >604 and score <=608 then '(604,608]' when score >608 and score <=612 then '(608,612]' when score >612 and score <=617 then '(612,617]' when score >617 and score <=626 then '(617,626]' when score >626  then '(626,-)'"
    #,"when score >0  and score <=581 then  '(0,581]' when score >581 and score <=589 then '(581,589]' when score >589 and score <=595 then '(589,595]' when score >595 and score <=600 then '(595,600]' when score >600 and score <=604 then '(600,604]' when score >604 and score <=609 then '(604,609]' when score >609 and score <=614 then '(609,614]' when score >614 and score <=619 then '(614,619]' when score >619 and score <=627 then '(619,627]' when score >627  then '(627,-)'"
                   ]

    time_controls=["'"'2020-06-19'"'","'"'2021-05-03'"'","'"'2020-10-20'"'","'"'2020-10-20'"'","'"'2020-10-20'"'","'"'2020-10-18'"'","'"'2020-12-15'"'","'"'2020-10-27'"'","'"'2021-01-20'"'","'"'2021-01-30'"'","'"'2021-02-21'"'","'"'2020-02-21'"'","'"'2021-03-11'"'"
                   ,"'"'2021-03-01'"'","'"'2021-04-03'"'","'"'2021-04-08'"'"]
    inputdate_control1s=["'"'20200119'"'","'"'20201203'"'","'"'20200520'"'","'"'20200520'"'","'"'20200520'"'","'"'20200518'"'","'"'20200715'"'","'"'20200820'"'","'"'20200820'"'","'"'20200831'"'","'"'20200921'"'","'"'20200921'"'","'"'20201011'"'","'"'20201011'"'"
                         ,"'"'20201203'"'","'"'20201208'"'"]
    inputdate_control2s=["'"'20200219'"'","'"'20210103'"'","'"'20200620'"'","'"'20200620'"'","'"'20200620'"'","'"'20200618'"'","'"'20200815'"'","'"'20200920'"'","'"'20200920'"'","'"'20200930'"'","'"'20201021'"'","'"'20201021'"'","'"'20201111'"'","'"'20201111'"'"
                         ,"'"'20210103'"'","'"'20210108'"'"]
    ##借呗
    score_1010=['creditScore46','creditScore48','creditScore49','creditScore50','creditScore51','creditScore54','creditScore55','creditScore56',
                'creditScore57','creditScore58','creditScore59','creditScore61','creditScore62','creditScore63']
    ##京东
    score_1006=['creditScore52']
    tab = Tab()
    for i,creditscore in enumerate(creditscores):
        score_cut=score_cuts[i]
        time_control=time_controls[i]
        inputdate_control1=inputdate_control1s[i]
        inputdate_control2=inputdate_control2s[i]
        chinese_name=chinese_names[i]
        if creditscore in score_1010:
            businesstype="'"'1010'"'"
        elif creditscore in score_1006:
            businesstype="'"'1006'"'"
        else:
            businesstype="'"'2006'"'"
        # 定义  显示运行时间 的小功能
        def runtime(func):
            @functools.wraps(func)
            def inner(sql_Strip):
                t1 = time.time()
                data = func(sql_Strip)
                t2 = time.time()
                run_time = t2 - t1
                print(creditscore+'运行时间为:',run_time)
                return data
            return inner
        # 输出SQL运行结果
        @runtime # 能展示时间
        def run_sql(sql_Strip):
            cursor.execute(sql_Strip) #运行SQL
            data = as_pandas(cursor)  ##  输出跑SQL结果
            return data

        ###sql
        sql_badrate='''
        SELECT 
        case 
        {score_cut}
        end as score ,
        count( DISTINCT case when b.th>={bad} THEN b.customerid  END ) AS '坏样本',
        count( DISTINCT case when b.th=0 THEN b.customerid  END ) AS '好样本',
        count( DISTINCT case when b.th>=0 THEN b.customerid  END ) AS '总数'
        FROM 
        (select customerid,serialno
        from fk.v_g_aprvadt_appl_bsc_inf
        where data_dt={time_control}
        and inputdate between {inputdate_control1} and {inputdate_control2}
        and decisionstage='Apply'--申请
        and businesstype={businesstype}
        )a
        JOIN
        (SELECT customerid, max(bd_hist_high_odue_days) AS th
        FROM fk.v_g_loanlend_loan_inf_his
        WHERE data_dt={time_control}
        AND {time_control}> date(bd_fst_maturity_dt)+{day}
        GROUP BY 1
        ) b
        ON a.customerid=b.customerid
        right JOIN
        (SELECT objectno,{creditscore} AS score
        FROM fk.v_r_risk_fico_xml_format
        WHERE data_dt<={time_control}
        AND {creditscore}>0
        ) c
        ON a.serialno=c.objectno
        GROUP BY score
        ORDER BY score
        '''
        sql_badrate1='''
        SELECT 
        case 
        {score_cut}
        end as score ,
        count( DISTINCT case when b.th>={bad} THEN b.customerid  END ) AS '坏样本',
        count( DISTINCT case when b.th=0 THEN b.customerid  END ) AS '好样本',
        count( DISTINCT case when b.th>=0 THEN b.customerid  END ) AS '总数'
        FROM 
        (select customerid,serialno
        from fk.v_g_aprvadt_appl_bsc_inf
        where data_dt={time1}
        and inputdate between {inputdate1_1} and {inputdate1_2}
        and decisionstage='Apply'--申请
        and businesstype={businesstype}
        )a
        JOIN
        (SELECT customerid, max(bd_hist_high_odue_days) AS th
        FROM fk.v_g_loanlend_loan_inf_his
        WHERE data_dt={time1}
        AND {time1}> date(bd_fst_maturity_dt)+{day}
        GROUP BY 1
        ) b
        ON a.customerid=b.customerid
        right JOIN
        (SELECT objectno,{creditscore} AS score
        FROM fk.v_r_risk_fico_xml_format
        WHERE data_dt<={time1}
        AND {creditscore}>0
        ) c
        ON a.serialno=c.objectno
        GROUP BY score
        ORDER BY score
        '''
        sql_badrate2='''
        SELECT 
        case 
        {score_cut}
        end as score ,
        count( DISTINCT case when b.th>={bad} THEN b.customerid  END ) AS '坏样本',
        count( DISTINCT case when b.th=0 THEN b.customerid  END ) AS '好样本',
        count( DISTINCT case when b.th>=0 THEN b.customerid  END ) AS '总数'
        FROM 
        (select customerid,serialno
        from fk.v_g_aprvadt_appl_bsc_inf
        where data_dt={time2}
        and inputdate between {inputdate2_1} and {inputdate2_2}
        and decisionstage='Apply'--申请
        and businesstype={businesstype} 
        )a
        JOIN
        (SELECT customerid, max(bd_hist_high_odue_days) AS th
        FROM fk.v_g_loanlend_loan_inf_his
        WHERE data_dt={time2}
        AND {time2}> date(bd_fst_maturity_dt)+{day}
        GROUP BY 1
        ) b
        ON a.customerid=b.customerid
        right JOIN
        (SELECT objectno,{creditscore} AS score
        FROM fk.v_r_risk_fico_xml_format
        WHERE data_dt<={time2}
        AND {creditscore}>0
        ) c
        ON a.serialno=c.objectno
        GROUP BY score
        ORDER BY score
        '''
        sql_badrate3='''
        SELECT 
        case 
        {score_cut}
        end as score ,
        count( DISTINCT case when b.th>={bad} THEN b.customerid  END ) AS '坏样本',
        count( DISTINCT case when b.th=0 THEN b.customerid  END ) AS '好样本',
        count( DISTINCT case when b.th>=0 THEN b.customerid  END ) AS '总数'
        FROM 
        (select customerid,serialno
        from fk.v_g_aprvadt_appl_bsc_inf
        where data_dt={time3}
        and inputdate between {inputdate3_1} and {inputdate3_2}
        and decisionstage='Apply'--申请
        and businesstype={businesstype}
        )a
        JOIN
        (SELECT customerid, max(bd_hist_high_odue_days) AS th
        FROM fk.v_g_loanlend_loan_inf_his
        WHERE data_dt={time3}
        AND {time3}> date(bd_fst_maturity_dt)+{day}
        GROUP BY 1
        ) b
        ON a.customerid=b.customerid
        right JOIN
        (SELECT objectno,{creditscore} AS score
        FROM fk.v_r_risk_fico_xml_format
        WHERE data_dt<={time3}
        AND {creditscore}>0
        ) c
        ON a.serialno=c.objectno
        GROUP BY score
        ORDER BY score
        '''
        if creditscore!='creditScore55':
            sql_badrate_end= sql_badrate.format(
            score_cut=score_cut,
            time_control=time_control,
            inputdate_control1=inputdate_control1,
            inputdate_control2=inputdate_control2 ,
            creditscore=creditscore,
            businesstype=businesstype,
            bad='30',
            day='90'  
            )
            badrate_control=run_sql(sql_badrate_end)
            print(badrate_control)
            if badrate_control['总数'].sum()<=20:
                print(creditscore+'控制组没有数据')
            else:
                if len(badrate_control)<10:
                    print(creditscore+'数据有问题')
                else:
                    sql_badrate_end1 = sql_badrate1.format(
                    score_cut=score_cut,
                    time1=time1,
                    inputdate1_1=inputdate1_1,
                    inputdate1_2=inputdate1_2 ,
                    creditscore=creditscore,
                    businesstype=businesstype,
                    bad='30',
                    day='90'  
                    )
                    badrate1=run_sql(sql_badrate_end1)
                    print(badrate1)
                    if badrate1['总数'].sum()<=20:
                        print(creditscore+'第一组没有数据')
                    else:
                        if len(badrate1)<10:
                            print(creditscore+'数据有问题')
                        else:
                            sql_badrate_end2 = sql_badrate2.format(
                            score_cut=score_cut,
                            time2=time2,
                            inputdate2_1=inputdate2_1,
                            inputdate2_2=inputdate2_2 ,
                            creditscore=creditscore,
                            businesstype=businesstype,
                            bad='30',
                            day='90'
                            )
                            badrate2=run_sql(sql_badrate_end2)
                            print(badrate2)
                            if badrate2['总数'].sum()<=0:
                                print(creditscore+'第二组没有数据')
                            else:
                                if len(badrate2)<10:
                                    print(creditscore+'数据有问题')
                                else:
                                    sql_badrate_end3 = sql_badrate3.format(
                                    score_cut=score_cut,
                                    time3=time3,
                                    inputdate3_1=inputdate3_1,
                                    inputdate3_2=inputdate3_2 ,
                                    creditscore=creditscore,
                                    businesstype=businesstype,
                                    bad='30',
                                    day='90'
                                    )
                                    badrate3=run_sql(sql_badrate_end3)
                                    print(badrate3)
                                    if badrate3['总数'].sum()<=0:
                                        print(creditscore+'第三组没有数据')
                                    else:
                                        if len(badrate3)<10:
                                            print(creditscore+'数据有问题')
                                        else:
                                            ##计算Badrate
                                            f=lambda x: '%.2f%%' % (x*100)
                                            badrate_control['badrate']=badrate_control['坏样本']/badrate_control['总数']
                                            where_are_nan_control = np.isnan(badrate_control['badrate'])  ##查找是nan的PSI
                                            badrate_control['badrate'][where_are_nan_control] = 0.0000
                                            badrate_control[['badrate']]=badrate_control[['badrate']].applymap(f)##转变成百分数并保留两位小数
                                            
                                            
                                            badrate1['badrate']=badrate1['坏样本']/badrate1['总数']
                                            where_are_nan1 = np.isnan(badrate1['badrate'])  ##查找是nan的PSI
                                            badrate1['badrate'][where_are_nan1] = 0.0000
                                            badrate1[['badrate']]=badrate1[['badrate']].applymap(f)
                                            
                                            
                                            badrate2['badrate']=badrate2['坏样本']/badrate2['总数']
                                            where_are_nan2 = np.isnan(badrate2['badrate'])  ##查找是nan的PSI
                                            badrate2['badrate'][where_are_nan2] = 0.0000
                                            badrate2[['badrate']]=badrate2[['badrate']].applymap(f)
                                            
                                            
                                            badrate3['badrate']=badrate3['坏样本']/badrate3['总数']
                                            where_are_nan3 = np.isnan(badrate3['badrate'])  ##查找是nan的PSI
                                            badrate3['badrate'][where_are_nan3] = 0.0000
                                            badrate3[['badrate']]=badrate3[['badrate']].applymap(f)
                                            
                                            b0=badrate_control.drop(['好样本'],axis=1)
                                            b1=badrate1.drop(['score','好样本'],axis=1)
                                            b2=badrate2.drop(['score','好样本'],axis=1)
                                            b3=badrate3.drop(['score','好样本'],axis=1)
                                            
                                            badrate=pd.concat([b0,b1,b2,b3],axis=1)
                                            rows=[]
                                            line=list(range(0,len(badrate),1))
                                            for i in line:
                                                line[i]=badrate.iloc[i].tolist()
                                                rows.append(line[i])
                                                headers = ['Score', '坏样本0','总数0', 'badrate0','坏样本1','总数1', 'badrate1','坏样本2','总数2', 'badrate2','坏样本3', '总数3', 'badrate3']
                                            def bad():
                                                c=(Table().add(headers,rows)  
                                                .set_global_opts(title_opts=ComponentTitleOpts(title=creditscore+"("+chinese_name+")",
                                                                                               subtitle="控制组授信日期:"+inputdate_control1[1:10]+"-"+inputdate_control2[1:10]+
                                                                                              "    1组授信日期:"+inputdate1_1[1:10]+"-"+inputdate1_2[1:10]+
                                                                                              "    2组授信日期:"+inputdate2_1[1:10]+"-"+inputdate2_2[1:10]+
                                                                                              "    3组授信日期:"+inputdate3_1[1:10]+"-"+inputdate3_2[1:10])))               
                                                return c
                                            tab.add(bad(),creditscore)
                                    

        else:
            sql_badrate_end= sql_badrate.format(
            score_cut=score_cut,
            time_control=time_control,
            inputdate_control1=inputdate_control1,
            inputdate_control2=inputdate_control2 ,
            creditscore=creditscore,
            businesstype=businesstype,
            bad='1',
            day='7'  
            )
            badrate_control=run_sql(sql_badrate_end)
            print(badrate_control)
            if badrate_control['总数'].sum()<=20:
                print(creditscore+'控制组没有数据')
            else:
                if len(badrate_control)<10:
                    print(creditscore+'数据有问题')
                else:
                    sql_badrate_end1 = sql_badrate1.format(
                    score_cut=score_cut,
                    time1=time55_1,
                    inputdate1_1=inputdate1_1,
                    inputdate1_2=inputdate1_2 ,
                    creditscore=creditscore,
                    businesstype=businesstype,
                    bad='1',
                    day='7'  
                    )
                    badrate1=run_sql(sql_badrate_end1)
                    print(badrate1)
                    if badrate1['总数'].sum()<=20:
                        print(creditscore+'第一组没有数据')
                    else:
                        if len(badrate1)<10:
                            print(creditscore+'数据有问题')
                        else:
                            sql_badrate_end2 = sql_badrate2.format(
                            score_cut=score_cut,
                            time2=time55_2,
                            inputdate2_1=inputdate2_1,
                            inputdate2_2=inputdate2_2 ,
                            creditscore=creditscore,
                            businesstype=businesstype,
                            bad='1',
                            day='7'
                            )
                            badrate2=run_sql(sql_badrate_end2)
                            print(badrate2)
                            if badrate2['总数'].sum()<=0:
                                print(creditscore+'第二组没有数据')
                            else:
                                if len(badrate2)<10:
                                    print(creditscore+'数据有问题')
                                else:
                                    sql_badrate_end3 = sql_badrate3.format(
                                    score_cut=score_cut,
                                    time3=time55_3,
                                    inputdate3_1=inputdate3_1,
                                    inputdate3_2=inputdate3_2 ,
                                    creditscore=creditscore,
                                    businesstype=businesstype,
                                    bad='1',
                                    day='7'
                                    )
                                    badrate3=run_sql(sql_badrate_end3)
                                    print(badrate3)
                                    if badrate3['总数'].sum()<=0:
                                        print(creditscore+'第三组没有数据')
                                    else:
                                        if len(badrate3)<10:
                                            print(creditscore+'数据有问题')
                                        else:
                                        
                                             ##计算Badrate
                                            f=lambda x: '%.2f%%' % (x*100)
                                            badrate_control['badrate']=badrate_control['坏样本']/badrate_control['总数']
                                            where_are_nan_control = np.isnan(badrate_control['badrate'])  ##查找是nan的PSI
                                            badrate_control['badrate'][where_are_nan_control] = 0.0000
                                            badrate_control[['badrate']]=badrate_control[['badrate']].applymap(f)##转变成百分数并保留两位小数
                                            
                                            
                                            badrate1['badrate']=badrate1['坏样本']/badrate1['总数']
                                            where_are_nan1 = np.isnan(badrate1['badrate'])  ##查找是nan的PSI
                                            badrate1['badrate'][where_are_nan1] = 0.0000
                                            badrate1[['badrate']]=badrate1[['badrate']].applymap(f)
                                            
                                            
                                            badrate2['badrate']=badrate2['坏样本']/badrate2['总数']
                                            where_are_nan2 = np.isnan(badrate2['badrate'])  ##查找是nan的PSI
                                            badrate2['badrate'][where_are_nan2] = 0.0000
                                            badrate2[['badrate']]=badrate2[['badrate']].applymap(f)
                                            
                                            
                                            badrate3['badrate']=badrate3['坏样本']/badrate3['总数']
                                            where_are_nan3 = np.isnan(badrate3['badrate'])  ##查找是nan的PSI
                                            badrate3['badrate'][where_are_nan3] = 0.0000
                                            badrate3[['badrate']]=badrate3[['badrate']].applymap(f)
                                            
                                            b0=badrate_control.drop(['好样本'],axis=1)
                                            b1=badrate1.drop(['score','好样本'],axis=1)
                                            b2=badrate2.drop(['score','好样本'],axis=1)
                                            b3=badrate3.drop(['score','好样本'],axis=1)
                                            
                                            badrate=pd.concat([b0,b1,b2,b3],axis=1)
                                            rows=[]
                                            line=list(range(0,len(badrate),1))
                                            for i in line:
                                                line[i]=badrate.iloc[i].tolist()
                                                rows.append(line[i])
                                                headers = ['Score', '坏样本0','总数0', 'badrate0','坏样本1','总数1', 'badrate1','坏样本2','总数2', 'badrate2','坏样本3', '总数3', 'badrate3']
                                            def bad():
                                                c=(Table().add(headers,rows)  
                                                .set_global_opts(title_opts=ComponentTitleOpts(title=creditscore+"("+chinese_name+")",
                                                                                               subtitle="控制组授信日期:"+inputdate_control1[1:10]+"-"+inputdate_control2[1:10]+
                                                                                              "    1组授信日期:"+inputdate1_1[1:10]+"-"+inputdate1_2[1:10]+
                                                                                              "    2组授信日期:"+inputdate2_1[1:10]+"-"+inputdate2_2[1:10]+
                                                                                              "    3组授信日期:"+inputdate3_1[1:10]+"-"+inputdate3_2[1:10])))   
                                                return c
                                            tab.add(bad(),creditscore)
        ##计算KS
        badrate_control.loc['Sum'] = badrate_control.apply(lambda x: x.sum())
        badrate1.loc['Sum'] = badrate1.apply(lambda x: x.sum())
        badrate2.loc['Sum'] = badrate2.apply(lambda x: x.sum())
        badrate3.loc['Sum'] = badrate3.apply(lambda x: x.sum())
        #print(badrate_control)
        
        KS0=[]
        for i in list(range(len(badrate_control))) :
            KS0.append(round(abs(sum(badrate_control['坏样本'][0:i])/badrate_control['坏样本']['Sum']
                                -sum(badrate_control['好样本'][0:i])/badrate_control['好样本']['Sum']),4))
        KS1=[]
        for i in list(range(len(badrate1))) :
            KS1.append(round(abs(sum(badrate1['坏样本'][0:i])/badrate1['坏样本']['Sum']
                                -sum(badrate1['好样本'][0:i])/badrate1['好样本']['Sum']),4))
        KS2=[]
        for i in list(range(len(badrate2))) :
            KS2.append(round(abs(sum(badrate2['坏样本'][0:i])/badrate2['坏样本']['Sum']
                                -sum(badrate2['好样本'][0:i])/badrate2['好样本']['Sum']),4))
        KS3=[]
        for i in list(range(len(badrate3))) :
            KS3.append(round(abs(sum(badrate3['坏样本'][0:i])/badrate3['坏样本']['Sum']
                                -sum(badrate3['好样本'][0:i])/badrate3['好样本']['Sum']),4))
        ks_dic.update({str(creditscore):[chinese_name,max(KS0),max(KS1),max(KS2),max(KS3)]})
                                            
    return tab,ks_dic

##最终汇总的公式，用到了定义的公式effect()
def effect_and_ks(user,password,inputdate1_1,inputdate1_2,inputdate2_1,inputdate2_2,inputdate3_1,inputdate3_2,time1,time2,time3,time55_1,time55_2,time55_3):
    tab,ks_dic=effect(user=user##用户
                   ,password = password##密码
                   ,inputdate1_1=inputdate1_1
                   ,inputdate1_2=inputdate1_2
                   ,inputdate2_1=inputdate2_1
                   ,inputdate2_2=inputdate2_2
                   ,inputdate3_1=inputdate3_1
                   ,inputdate3_2=inputdate3_2
                   ,time1=time1
                   ,time2=time2
                   ,time3=time3
                   ,time55_1=time55_1
                   ,time55_2=time55_2
                   ,time55_3=time55_3
                    )
    ##将psi_dic字典转化为dataframe形式，方便处理
    ks_df=pd.DataFrame.from_dict(ks_dic,orient='index',columns=['Chinese_name','KS0','KS1','KS2','KS3'])
    ks_df = ks_df.reset_index().rename(columns = {'index':'ID'})
    
    ##所有KS以table形式展示
    headers=['ID','Chinese_name','KS0','KS1','KS2','KS3']
    rows=[]
    line=list(range(0,len(ks_df),1))
    for i in line:
        line[i]=ks_df.iloc[i].tolist()
        rows.append(line[i])
    def ks():
        c=(Table().add(headers,rows)  
        .set_global_opts(title_opts=ComponentTitleOpts(title="KS汇总"
                         ,subtitle="1组授信日期:"+inputdate1_1[1:10]+"-"+inputdate1_2[1:10]+
                                   "    2组授信日期:"+inputdate2_1[1:10]+"-"+inputdate2_2[1:10]+
                                   "    3组授信日期:"+inputdate3_1[1:10]+"-"+inputdate3_2[1:10])))              
        return c
    tab.add(ks(),'KS_all')
    return tab.render("badrate+ks.html")

## 运行部分

In [ ]:
effect_and_ks(user='fk_yanjintong'##用户
       ,password = 'Yjt@0828$$'
       ,inputdate1_1="'"'20201001'"'"
       ,inputdate1_2="'"'20201101'"'"
       ,inputdate2_1="'"'20201101'"'"
       ,inputdate2_2="'"'20201201'"'"
       ,inputdate3_1="'"'20201201'"'"
       ,inputdate3_2="'"'20210101'"'"
       ,time1="'"'2021-03-01'"'"
       ,time2="'"'2021-04-01'"'"
       ,time3="'"'2021-05-01'"'"
       ,time55_1="'"'2020-12-08'"'"
       ,time55_2="'"'2021-01-08'"'"
       ,time55_3="'"'2021-02-08'"'"
      )